# FlowSOM Analysis Pipeline - Notebook Headless

## Pipeline complète d'analyse FlowSOM pour données de cytométrie en flux

Ce notebook "miroir" de l'application FlowSOM Analyzer permet:
- **Debug & Introspection**: Visualiser les données à chaque étape
- **Tuning rapide**: Tester différents paramètres sans relancer l'app
- **Séparation des responsabilités**: Logique métier pure, sans UI

---

**Auteur**: Florian Magne
**Version**: 1.0
**Date**: Janvier 2026

## 1. Import des Librairies

Import de toutes les librairies nécessaires avec vérification de disponibilité des dépendances optionnelles.

In [ ]:
# -*- coding: utf-8 -*-

# IMPORTS début du fichier
import sys
import warnings
from pathlib import Path
from typing import Optional, List, Dict, Any, Tuple
from datetime import datetime

warnings.filterwarnings('ignore')

# Imports scientifiques de base
import numpy as np
import pandas as pd

# CONFIGURATION PANDAS: Affichage en format linéaire (jamais exponentiel)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')  # 4 décimales max
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.width', None)  # Largeur auto
pd.set_option('display.max_rows', 100)  # Max 100 lignes affichées
np.set_printoptions(suppress=True, precision=4)  # Numpy aussi en linéaire
print("[OK] Pandas configuré: affichage linéaire (pas de notation scientifique)")

# Imports visualisation
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')
plt.rcParams['figure.facecolor'] = "#ffffff"
plt.rcParams['axes.facecolor'] = "#ffffff"
plt.rcParams['text.color'] = "#000000"
plt.rcParams['axes.labelcolor'] = "#000000"
plt.rcParams['xtick.color'] = "#000000"
plt.rcParams['ytick.color'] = "#000000"
plt.rcParams['axes.edgecolor'] = "#000000"
plt.rcParams['grid.color'] = "#cccccc"
plt.rcParams['figure.figsize'] = (12, 8)

# Plotly pour visualisations interactives
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.io as pio
    # Configuration pour affichage dans les notebooks
    pio.renderers.default = 'notebook'
    PLOTLY_AVAILABLE = True
    print("[OK] Plotly disponible")
except ImportError:
    PLOTLY_AVAILABLE = False
    print("[!] Plotly non installé (optionnel): pip install plotly")

# IMPORTS flowsom et anndata, l'un est le package d'analyse du FlowSOM, l'autre est pour gérer les données dans des objets AnnData
try:
    import flowsom as fs
    import anndata as ad
    FLOWSOM_AVAILABLE = True
    print("[OK] FlowSOM disponible")
except ImportError:
    FLOWSOM_AVAILABLE = False
    print("[X] FlowSOM non installé: pip install flowsom")

# Import de Scanpy pour UMAP/t-SNE
try:
    import scanpy as sc
    SCANPY_AVAILABLE = True
    print("[OK] Scanpy disponible")
except ImportError:
    SCANPY_AVAILABLE = False
    print("[!] Scanpy non installé (optionnel): pip install scanpy")

# Import de UMAP
try:
    import umap
    UMAP_AVAILABLE = True
    print("[OK] UMAP disponible")
except ImportError:
    UMAP_AVAILABLE = False
    print("[!] UMAP non installé (optionnel): pip install umap-learn")


# Import de t-SNE via sklearn car t-SNE trop lent à être implémenté dans Scanpy (et FlowSOM)
try:
    from sklearn.manifold import TSNE
    from sklearn.metrics import silhouette_score
    from sklearn.cluster import AgglomerativeClustering
    SKLEARN_AVAILABLE = True
    print("[OK] Scikit-learn disponible")
except ImportError:
    SKLEARN_AVAILABLE = False
    print("[!] Scikit-learn non installé: pip install scikit-learn")

# FlowKit pour transformations Logicle
try:
    import flowkit as fk
    FLOWKIT_AVAILABLE = True
    # Configuration FlowKit: format linéaire pour les exports/affichages
    # FlowKit utilise pandas en interne, donc pd.set_option s'applique
    # Mais on configure aussi les options de logging/affichage si disponibles
    try:
        import logging
        logging.getLogger('flowkit').setLevel(logging.WARNING)  # Moins de logs verbose
    except:
        pass
    print("[OK] FlowKit disponible (transformations Logicle précise en 1 fonction)")
except ImportError:
    FLOWKIT_AVAILABLE = False
    print("[!] FlowKit non installé (optionnel): pip install flowkit)")

# FCSWrite pour export FCS
try:
    import fcswrite
    FCSWRITE_AVAILABLE = True
    print("[OK] FCSWrite disponible (export FCS)")
except ImportError:
    FCSWRITE_AVAILABLE = False
    print("[!] FCSWrite non installé (optionnel): pip install fcswrite")

# Scipy pour statistiques 
from scipy import stats

In [ ]:
# Import en haut de fichier des classes utilitaires permettant les transformations des fichiers ainsi que le pre-gating 

class DataTransformer:
    """
    Transformations de données de cytométrie (Logicle, Arcsinh, etc.).
    Classe statique réutilisable sans dépendance à l'UI.
    """
    
    @staticmethod
    def arcsinh_transform(data: np.ndarray, cofactor: float = 5.0) -> np.ndarray:
        """
        Transformation Arcsinh (inverse hyperbolic sine).
        
        Args en entrée:
            data: Matrice de données (n_cells, n_markers)
            cofactor: Facteur de division (5 pour flow cytometry)
        
        Returns:
            Données transformées
        """
        return np.arcsinh(data / cofactor)
    
    @staticmethod
    def arcsinh_inverse(data: np.ndarray, cofactor: float = 5.0) -> np.ndarray:
        """Inverse de la transformation Arcsinh."""
        return np.sinh(data) * cofactor
    
    @staticmethod
    def logicle_transform(data: np.ndarray, T: float = 262144.0, M: float = 4.5,
                          W: float = 0.5, A: float = 0.0) -> np.ndarray:
        """
        Transformation Logicle (biexponentielle).
        
        Args en entrée:
            data: Matrice de données
            T: Maximum de l'échelle linéaire (262144 = 2^18)
            M: Décades de largeur
            W: Linéarisation près de zéro
            A: Décades additionnelles (négatifs)
        
        Returns:
            Données transformées
        """
        if FLOWKIT_AVAILABLE:
            # Utiliser FlowKit si disponible (plus précis) avec une fonction prédéfinie
            try:
                xform = fk.transforms.LogicleTransform(T=T, M=M, W=W, A=A)
                return xform.apply(data)
            except:
                pass
        
        # Approximation si FlowKit absent: Arcsinh modifié
        w_val = W * np.log10(np.e)
        return np.arcsinh(data / (T / (10 ** M))) * (M / np.log(10))
    
    @staticmethod
    def log_transform(data: np.ndarray, base: float = 10.0,
                      min_val: float = 1.0) -> np.ndarray:
        """Transformation logarithmique standard."""
        data_clipped = np.maximum(data, min_val)
        return np.log(data_clipped) / np.log(base)
    
    @staticmethod
    def zscore_normalize(data: np.ndarray) -> np.ndarray:
        """Normalisation Z-score (moyenne=0, std=1)."""
        mean = np.nanmean(data, axis=0)
        std = np.nanstd(data, axis=0)
        std[std == 0] = 1  # Éviter division par zéro
        return (data - mean) / std
    
    @staticmethod
    def min_max_normalize(data: np.ndarray) -> np.ndarray:
        """Normalisation Min-Max [0, 1]."""
        min_val = np.nanmin(data, axis=0)
        max_val = np.nanmax(data, axis=0)
        range_val = max_val - min_val
        range_val[range_val == 0] = 1
        return (data - min_val) / range_val


class PreGating:
    """
    Pre-gating automatique pour la sélection des populations d'intérêt.
    Basé sur FSC/SSC pour exclure les débris et les doublets.
    """
    
    @staticmethod
    def find_marker_index(var_names: List[str], patterns: List[str]) -> Optional[int]:
        """Trouve l'index d'un marqueur parmi les patterns donnés."""
        var_upper = [v.upper() for v in var_names]
        for pattern in patterns:
            for i, name in enumerate(var_upper):
                if pattern.upper() in name:
                    return i
        return None
    
    @staticmethod
    def gate_viable_cells(X: np.ndarray, var_names: List[str],
                          min_percentile: float = 2.0, 
                          max_percentile: float = 98.0) -> np.ndarray:
        """
        Gate les cellules viables basé sur FSC/SSC.
        
        Args:
            X: Matrice des données (n_cells, n_markers)
            var_names: Liste des noms de marqueurs
            min_percentile: Percentile minimum (exclusion débris)
            max_percentile: Percentile maximum (exclusion doublets)
        
        Returns:
            Masque booléen des cellules viables
        """
        n_cells = X.shape[0]
        mask = np.ones(n_cells, dtype=bool)
        
        # Trouver FSC (priorité à FSC-A)
        fsc_idx = PreGating.find_marker_index(var_names, ['FSC-A', 'FSC-H', 'FSC'])
        if fsc_idx is not None:
            fsc_vals = X[:, fsc_idx].astype(np.float64)
            fsc_vals = np.where(np.isfinite(fsc_vals), fsc_vals, np.nan)
            low = np.nanpercentile(fsc_vals, min_percentile)
            high = np.nanpercentile(fsc_vals, max_percentile)
            mask &= np.isfinite(fsc_vals) & (fsc_vals >= low) & (fsc_vals <= high)
        
        # Trouver SSC (priorité à SSC-A)
        ssc_idx = PreGating.find_marker_index(var_names, ['SSC-A', 'SSC-H', 'SSC'])
        if ssc_idx is not None:
            ssc_vals = X[:, ssc_idx].astype(np.float64)
            ssc_vals = np.where(np.isfinite(ssc_vals), ssc_vals, np.nan)
            low = np.nanpercentile(ssc_vals, min_percentile)
            high = np.nanpercentile(ssc_vals, max_percentile)
            mask &= np.isfinite(ssc_vals) & (ssc_vals >= low) & (ssc_vals <= high)
        
        return mask
    
    @staticmethod
    def gate_singlets(X: np.ndarray, var_names: List[str],
                      ratio_min: float = 0.6, ratio_max: float = 1.5) -> np.ndarray:
        """
        Gate les singlets basé sur le ratio FSC-A/FSC-H.
        Les doublets ont typiquement un ratio > 1.3-1.5.
        
        Args:
            X: Matrice des données
            var_names: Liste des noms de marqueurs
            ratio_min: Ratio minimum acceptable
            ratio_max: Ratio maximum acceptable
        
        Returns:
            Masque booléen des singlets
        """
        n_cells = X.shape[0]
        
        fsc_a_idx = PreGating.find_marker_index(var_names, ['FSC-A'])
        fsc_h_idx = PreGating.find_marker_index(var_names, ['FSC-H'])
        
        if fsc_a_idx is None or fsc_h_idx is None:
            print("[!] FSC-A ou FSC-H non trouvé, pas de gating singlets")
            return np.ones(n_cells, dtype=bool)
        
        fsc_a = X[:, fsc_a_idx].astype(np.float64)
        fsc_h = X[:, fsc_h_idx].astype(np.float64)
        
        # Valeurs minimum pour éviter division par zéro
        min_val = 100
        valid_h = fsc_h > min_val
        
        ratio = np.full(n_cells, np.nan)
        ratio[valid_h] = fsc_a[valid_h] / fsc_h[valid_h]
        
        mask = np.isfinite(ratio) & (ratio >= ratio_min) & (ratio <= ratio_max)
        
        return mask
    
    @staticmethod
    def gate_cd45_positive(X: np.ndarray, var_names: List[str],
                           threshold_percentile: float = 10) -> np.ndarray:
        """
        Gate les cellules CD45+ (leucocytes).
        
        Returns:
            Masque booléen des cellules CD45+
        """
        n_cells = X.shape[0]
        
        cd45_idx = PreGating.find_marker_index(var_names, ['CD45', 'CD45-PECY5', 'CD45-PC5'])
        if cd45_idx is None:
            print("[!] CD45 non trouvé, pas de gating CD45+")
            return np.ones(n_cells, dtype=bool)
        
        cd45_vals = X[:, cd45_idx].astype(np.float64)
        cd45_vals = np.where(np.isfinite(cd45_vals), cd45_vals, np.nan)
        
        threshold = np.nanpercentile(cd45_vals, threshold_percentile)
        
        return np.where(np.isnan(cd45_vals), False, cd45_vals > threshold)
    
    @staticmethod
    def gate_cd34_blasts(X: np.ndarray, var_names: List[str],
                         threshold_percentile: float = 85,
                         use_ssc_filter: bool = True,
                         ssc_max_percentile: float = 70) -> np.ndarray:
        """
        Gate les blastes CD34+ (cellules souches/progénitrices).
        
        Les blastes sont typiquement:
        - CD34 bright (haute expression)
        - SSC low (faible granularité)
        
        Args:
            X: Matrice des données (n_cells, n_markers)
            var_names: Liste des noms de marqueurs
            threshold_percentile: Percentile pour définir le seuil CD34+ (ex: 85 = top 15%)
            use_ssc_filter: Appliquer aussi un filtre SSC pour enrichir en blastes
            ssc_max_percentile: Percentile max de SSC pour blastes (faible granularité)
        
        Returns:
            Masque booléen des blastes CD34+
        """
        n_cells = X.shape[0]
        
        # Chercher CD34 avec différents nommages possibles
        cd34_idx = PreGating.find_marker_index(var_names, ['CD34', 'CD34-PE', 'CD34-APC', 'CD34-PECY7'])
        if cd34_idx is None:
            print("[!] CD34 non trouvé, pas de gating blastes")
            return np.ones(n_cells, dtype=bool)
        
        cd34_vals = X[:, cd34_idx].astype(np.float64)
        cd34_vals = np.where(np.isfinite(cd34_vals), cd34_vals, np.nan)
        
        # Seuil CD34+ (prendre les cellules avec haute expression)
        threshold_cd34 = np.nanpercentile(cd34_vals, threshold_percentile)
        mask_cd34 = np.where(np.isnan(cd34_vals), False, cd34_vals >= threshold_cd34)
        
        # Optionnel: filtrer aussi par SSC low (blastes = faible granularité)
        if use_ssc_filter:
            ssc_idx = PreGating.find_marker_index(var_names, ['SSC-A', 'SSC-H', 'SSC'])
            if ssc_idx is not None:
                ssc_vals = X[:, ssc_idx].astype(np.float64)
                ssc_vals = np.where(np.isfinite(ssc_vals), ssc_vals, np.nan)
                threshold_ssc = np.nanpercentile(ssc_vals, ssc_max_percentile)
                mask_ssc = np.where(np.isnan(ssc_vals), False, ssc_vals <= threshold_ssc)
                return mask_cd34 & mask_ssc
        
        return mask_cd34
    
    @staticmethod
    def gate_debris_polygon(X: np.ndarray, var_names: List[str],
                            fsc_min: float = None, fsc_max: float = None,
                            ssc_min: float = None, ssc_max: float = None,
                            auto_percentiles: bool = True,
                            min_pct: float = 1.0, max_pct: float = 99.0) -> np.ndarray:
        """
        Gate rectangulaire/polygonal pour exclure les débris sur FSC-A vs SSC-A.
        
        Args:
            X: Matrice des données
            var_names: Liste des noms de marqueurs
            fsc_min/fsc_max: Seuils FSC manuels (si auto_percentiles=False)
            ssc_min/ssc_max: Seuils SSC manuels (si auto_percentiles=False)
            auto_percentiles: Calculer automatiquement les seuils via percentiles
            min_pct/max_pct: Percentiles pour auto-calcul
        
        Returns:
            Masque booléen des cellules (non-débris)
        """
        n_cells = X.shape[0]
        
        fsc_idx = PreGating.find_marker_index(var_names, ['FSC-A'])
        ssc_idx = PreGating.find_marker_index(var_names, ['SSC-A'])
        
        if fsc_idx is None or ssc_idx is None:
            print("[!] FSC-A ou SSC-A non trouvé pour gate débris")
            return np.ones(n_cells, dtype=bool)
        
        fsc_vals = X[:, fsc_idx].astype(np.float64)
        ssc_vals = X[:, ssc_idx].astype(np.float64)
        
        # Calculer les seuils automatiquement si demandé
        if auto_percentiles:
            fsc_min = np.nanpercentile(fsc_vals, min_pct)
            fsc_max = np.nanpercentile(fsc_vals, max_pct)
            ssc_min = np.nanpercentile(ssc_vals, min_pct)
            ssc_max = np.nanpercentile(ssc_vals, max_pct)
        
        # Appliquer le gate rectangulaire
        mask = (
            np.isfinite(fsc_vals) & np.isfinite(ssc_vals) &
            (fsc_vals >= fsc_min) & (fsc_vals <= fsc_max) &
            (ssc_vals >= ssc_min) & (ssc_vals <= ssc_max)
        )
        
        return mask


print("[OK] Classes DataTransformer et PreGating chargées!")

In [ ]:
# =============================================================================
# CLASSE AutoGating — Gating adaptatif par GMM/KDE
# =============================================================================
# Inspiré de CytoPy AutonomousGate (sans dépendance MongoDB)
# Utilise scikit-learn GaussianMixture pour trouver les "creux" réels
# entre les populations au lieu de couper à des percentiles fixes.
#
# Avantages vs PreGating (percentiles):
#   - Si un échantillon a 10% de débris → la porte s'adapte automatiquement
#   - Si un échantillon est propre → moins de perte de données
#   - Pour les doublets: modélise la diagonale FSC-A/FSC-H statistiquement
#   - Pour CD45+: trouve le creux bimodal entre CD45- et CD45+
# =============================================================================

class AutoGating:
    """
    Gating automatique adaptatif basé sur des modèles de mélange gaussien (GMM)
    et estimation de densité. Inspiré de CytoPy AutonomousGate.
    
    Chaque méthode utilise un GMM pour identifier les populations naturelles
    dans les données, au lieu de seuils fixes basés sur des percentiles.
    
    Dépendances: scikit-learn (GaussianMixture, StandardScaler)
    """
    
    @staticmethod
    def auto_gate_debris(X: np.ndarray, var_names: List[str],
                          n_components: int = 3,
                          min_cluster_fraction: float = 0.02) -> np.ndarray:
        """
        Gate débris adaptatif par GMM 2D sur FSC-A / SSC-A.
        
        L'algorithme identifie les clusters naturels dans l'espace FSC/SSC:
        - Débris: événements bas en FSC-A (petites particules)
        - Cellules: population principale (cluster dominant)  
        - Saturés: événements très hauts (optionnel, détecté par BIC)
        
        Sélection automatique du nombre de composantes par BIC (2 ou 3).
        
        Args:
            X: Matrice des données (n_cells, n_markers)
            var_names: Noms des marqueurs
            n_components: Nombre max de composantes GMM à tester
            min_cluster_fraction: Fraction min d'événements pour inclure un cluster
        
        Returns:
            Masque booléen (True = cellule viable, False = débris/saturé)
        """
        from sklearn.mixture import GaussianMixture
        from sklearn.preprocessing import StandardScaler
        
        n_cells = X.shape[0]
        fsc_idx = PreGating.find_marker_index(var_names, ['FSC-A'])
        ssc_idx = PreGating.find_marker_index(var_names, ['SSC-A'])
        
        if fsc_idx is None or ssc_idx is None:
            print("[!] FSC-A ou SSC-A non trouvé pour auto-gate débris")
            return np.ones(n_cells, dtype=bool)
        
        fsc = X[:, fsc_idx].astype(np.float64)
        ssc = X[:, ssc_idx].astype(np.float64)
        
        # Filtrer les NaN/Inf
        valid = np.isfinite(fsc) & np.isfinite(ssc)
        data_2d = np.column_stack([fsc[valid], ssc[valid]])
        
        if valid.sum() < 200:
            print("[!] Pas assez de données valides pour auto-gate débris")
            return np.ones(n_cells, dtype=bool)
        
        # Standardiser avant GMM pour meilleure convergence
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data_2d)
        
        # Sélection automatique du nombre de composantes par BIC
        best_bic = np.inf
        best_gmm = None
        for n_comp in [2, 3]:
            gmm_test = GaussianMixture(
                n_components=n_comp, random_state=42,
                covariance_type='full', n_init=3, max_iter=200
            )
            gmm_test.fit(data_scaled)
            bic = gmm_test.bic(data_scaled)
            if bic < best_bic:
                best_bic = bic
                best_gmm = gmm_test
        
        labels = best_gmm.predict(data_scaled)
        n_comp = best_gmm.n_components
        
        # Statistiques par cluster (en espace original)
        cluster_sizes = np.bincount(labels, minlength=n_comp)
        cluster_fsc_means = np.array([data_2d[labels == i, 0].mean() for i in range(n_comp)])
        
        # Population principale = plus grand cluster
        main_cluster = np.argmax(cluster_sizes)
        
        # Inclure les clusters avec assez d'événements et un FSC raisonnable
        # (exclure les débris = FSC très bas)
        mask_valid = np.zeros(valid.sum(), dtype=bool)
        fsc_threshold = cluster_fsc_means[main_cluster] * 0.25
        
        for i in range(n_comp):
            fraction = cluster_sizes[i] / len(labels)
            if fraction >= min_cluster_fraction and cluster_fsc_means[i] >= fsc_threshold:
                mask_valid |= (labels == i)
        
        # Sécurité: si aucun cluster sélectionné, garder le principal
        if not mask_valid.any():
            mask_valid = (labels == main_cluster)
        
        mask = np.zeros(n_cells, dtype=bool)
        mask[valid] = mask_valid
        
        n_kept = mask.sum()
        print(f"   [Auto-GMM] {best_gmm.n_components} composantes détectées (BIC={best_bic:.0f})")
        for i in range(n_comp):
            status = "✓" if mask_valid[labels == i].any() else "✗"
            print(f"     {status} Cluster {i}: {cluster_sizes[i]:,} evt, FSC-A moy={cluster_fsc_means[i]:.0f}")
        print(f"   [Auto-GMM] → Conservés: {n_kept:,} événements")
        
        return mask
    
    @staticmethod
    def auto_gate_singlets(X: np.ndarray, var_names: List[str]) -> np.ndarray:
        """
        Gate singlets adaptatif par GMM sur le ratio FSC-A/FSC-H.
        
        Les singlets forment une diagonale sur le plot FSC-A vs FSC-H 
        avec un ratio FSC-A/FSC-H ≈ 1.0.
        Les doublets ont un ratio plus élevé (FSC-A augmente mais pas FSC-H).
        
        Méthode:
        1. Calcul du ratio FSC-A / FSC-H pour chaque événement
        2. GMM 2 composantes sur le ratio
        3. Le composant avec μ le plus proche de 1.0 = singlets
        
        Args:
            X: Matrice des données
            var_names: Noms des marqueurs
        
        Returns:
            Masque booléen (True = singlet, False = doublet)
        """
        from sklearn.mixture import GaussianMixture
        
        n_cells = X.shape[0]
        fsc_a_idx = PreGating.find_marker_index(var_names, ['FSC-A'])
        fsc_h_idx = PreGating.find_marker_index(var_names, ['FSC-H'])
        
        if fsc_a_idx is None or fsc_h_idx is None:
            print("[!] FSC-A ou FSC-H non trouvé pour auto-gate singlets")
            return np.ones(n_cells, dtype=bool)
        
        fsc_a = X[:, fsc_a_idx].astype(np.float64)
        fsc_h = X[:, fsc_h_idx].astype(np.float64)
        
        # Filtrer: valeurs valides avec FSC-H > seuil minimal (éviter division par 0)
        valid = np.isfinite(fsc_a) & np.isfinite(fsc_h) & (fsc_h > 100) & (fsc_a > 100)
        
        if valid.sum() < 200:
            print("[!] Pas assez de données valides pour auto-gate singlets")
            return np.ones(n_cells, dtype=bool)
        
        # Calcul du ratio FSC-A / FSC-H
        ratio = fsc_a[valid] / fsc_h[valid]
        
        # GMM 2 composantes sur le ratio
        # Singlets: ratio ≈ 1.0 | Doublets: ratio > ~1.3-1.5
        gmm = GaussianMixture(n_components=2, random_state=42, n_init=3)
        gmm.fit(ratio.reshape(-1, 1))
        
        labels = gmm.predict(ratio.reshape(-1, 1))
        means = gmm.means_.flatten()
        
        # Le composant singlet = celui avec μ le plus proche de 1.0
        singlet_component = np.argmin(np.abs(means - 1.0))
        
        mask = np.zeros(n_cells, dtype=bool)
        mask[valid] = (labels == singlet_component)
        
        n_singlets = mask.sum()
        n_doublets = valid.sum() - n_singlets
        print(f"   [Auto-GMM] Ratio FSC-A/FSC-H — 2 composantes: μ={means.round(3)}")
        print(f"   [Auto-GMM] Singlets (μ≈{means[singlet_component]:.3f}): {n_singlets:,}")
        print(f"   [Auto-GMM] Doublets rejetés: {n_doublets:,}")
        
        return mask
    
    @staticmethod
    def auto_gate_cd45(X: np.ndarray, var_names: List[str],
                        n_components: int = 2) -> np.ndarray:
        """
        Gate CD45+ adaptatif par GMM 1D.
        
        Trouve automatiquement le creux bimodal entre CD45- et CD45+
        au lieu d'un percentile fixe. Le GMM modélise la distribution
        bimodale et assigne chaque événement à la population la plus probable.
        
        Args:
            X: Matrice des données
            var_names: Noms des marqueurs
            n_components: Nombre de composantes GMM (2 = CD45- / CD45+)
        
        Returns:
            Masque booléen (True = CD45+, False = CD45-)
        """
        from sklearn.mixture import GaussianMixture
        
        n_cells = X.shape[0]
        cd45_idx = PreGating.find_marker_index(var_names, ['CD45', 'CD45-PECY5', 'CD45-PC5'])
        
        if cd45_idx is None:
            print("[!] CD45 non trouvé pour auto-gate CD45+")
            return np.ones(n_cells, dtype=bool)
        
        cd45 = X[:, cd45_idx].astype(np.float64)
        valid = np.isfinite(cd45)
        
        if valid.sum() < 200:
            print("[!] Pas assez de données valides pour auto-gate CD45+")
            return np.ones(n_cells, dtype=bool)
        
        # GMM pour séparer CD45- et CD45+
        gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=3)
        gmm.fit(cd45[valid].reshape(-1, 1))
        
        labels = gmm.predict(cd45[valid].reshape(-1, 1))
        means = gmm.means_.flatten()
        
        # CD45+ = composant avec la moyenne la plus élevée
        pos_component = np.argmax(means)
        
        # Calculer le seuil approximatif (intersection des 2 gaussiennes)
        sorted_means = np.sort(means)
        stds = np.sqrt(gmm.covariances_.flatten())
        sorted_stds = stds[np.argsort(means)]
        # Point d'intersection ≈ moyenne pondérée par les écarts-types
        threshold_approx = (sorted_means[0] * sorted_stds[1] + sorted_means[1] * sorted_stds[0]) / (sorted_stds[0] + sorted_stds[1])
        
        mask = np.zeros(n_cells, dtype=bool)
        mask[valid] = (labels == pos_component)
        
        n_pos = mask.sum()
        print(f"   [Auto-GMM] CD45: {n_components} composantes, μ={means.round(0)}")
        print(f"   [Auto-GMM] Seuil adaptatif ≈ {threshold_approx:.0f} (creux entre populations)")
        print(f"   [Auto-GMM] CD45+ identifiés: {n_pos:,} ({n_pos/valid.sum()*100:.1f}%)")
        
        return mask
    
    @staticmethod
    def auto_gate_cd34(X: np.ndarray, var_names: List[str],
                        use_ssc_filter: bool = True,
                        n_components: int = 2) -> np.ndarray:
        """
        Gate CD34+ blastes adaptatif par GMM.
        
        Identifie la population CD34 bright (blastes) par GMM au lieu d'un 
        percentile fixe. Optionnel: combine avec SSC low (blastes = faible granularité).
        
        Args:
            X: Matrice des données
            var_names: Noms des marqueurs
            use_ssc_filter: Combiner avec filtre GMM SSC low
            n_components: Nombre de composantes GMM
        
        Returns:
            Masque booléen (True = blaste CD34+, False = autre)
        """
        from sklearn.mixture import GaussianMixture
        
        n_cells = X.shape[0]
        cd34_idx = PreGating.find_marker_index(var_names, ['CD34', 'CD34-PE', 'CD34-APC', 'CD34-PECY7'])
        
        if cd34_idx is None:
            print("[!] CD34 non trouvé pour auto-gate blastes")
            return np.ones(n_cells, dtype=bool)
        
        cd34 = X[:, cd34_idx].astype(np.float64)
        valid = np.isfinite(cd34)
        
        if valid.sum() < 200:
            print("[!] Pas assez de données valides pour auto-gate CD34")
            return np.ones(n_cells, dtype=bool)
        
        # GMM pour séparer CD34- et CD34+
        gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=3)
        gmm.fit(cd34[valid].reshape(-1, 1))
        
        labels = gmm.predict(cd34[valid].reshape(-1, 1))
        means = gmm.means_.flatten()
        pos_component = np.argmax(means)
        
        mask_cd34 = np.zeros(n_cells, dtype=bool)
        mask_cd34[valid] = (labels == pos_component)
        
        n_cd34_pos = mask_cd34.sum()
        print(f"   [Auto-GMM] CD34: μ={means.round(0)}, CD34+ cluster = μ={means[pos_component]:.0f}")
        
        # Filtre SSC low optionnel (blastes = faible granularité)
        if use_ssc_filter:
            ssc_idx = PreGating.find_marker_index(var_names, ['SSC-A', 'SSC-H', 'SSC'])
            if ssc_idx is not None:
                ssc = X[:, ssc_idx].astype(np.float64)
                valid_ssc = np.isfinite(ssc)
                
                if valid_ssc.sum() >= 200:
                    gmm_ssc = GaussianMixture(n_components=2, random_state=42, n_init=3)
                    gmm_ssc.fit(ssc[valid_ssc].reshape(-1, 1))
                    
                    labels_ssc = gmm_ssc.predict(ssc[valid_ssc].reshape(-1, 1))
                    ssc_means = gmm_ssc.means_.flatten()
                    low_ssc_component = np.argmin(ssc_means)
                    
                    mask_ssc = np.zeros(n_cells, dtype=bool)
                    mask_ssc[valid_ssc] = (labels_ssc == low_ssc_component)
                    
                    combined = mask_cd34 & mask_ssc
                    print(f"   [Auto-GMM] + Filtre SSC low (μ={ssc_means[low_ssc_component]:.0f}): {combined.sum():,} blastes purs")
                    return combined
        
        print(f"   [Auto-GMM] CD34+ blastes: {n_cd34_pos:,}")
        return mask_cd34


print("[OK] Classe AutoGating chargée (gating adaptatif GMM/KDE)")
print("     Méthodes: auto_gate_debris, auto_gate_singlets, auto_gate_cd45, auto_gate_cd34")

## 2. Chargement des Fichiers FCS

Chargement des fichiers FCS depuis les dossiers spécifiés. 
- **Sain (NBM)**: Moelle osseuse normale (référence)
- **Pathologique**: Échantillons patients à analyser

In [ ]:
# CONFIGURATION DES CHEMINS
# Bien modifier ces chemins selon votre environnement actuel pour le bon chargement des données

# Dossier des fichiers sains (référence NBM)
HEALTHY_FOLDER = Path(r"C:\Users\Florian Travail\Documents\FlowSom\Data\BLAST110\BLAST110\Petit NBM")

# Dossier des fichiers pathologiques (patients)
PATHOLOGICAL_FOLDER = Path(r"Data/Patho")

# Mode d'analyse: 
# - True: Comparer Sain vs Pathologique
# - False: Analyser uniquement les fichiers pathologiques
COMPARE_MODE = True

print(f"Dossier Sain: {HEALTHY_FOLDER}")
print(f"Dossier Pathologique: {PATHOLOGICAL_FOLDER}")
print(f"Mode comparaison: {'Activé' if COMPARE_MODE else 'Patient seul'}")

In [ ]:
# FONCTIONS DE CHARGEMENT FCS

def get_fcs_files(folder: Path) -> List[str]:
    """Récupère la liste des fichiers FCS dans un dossier. Et renvoie une chaine de caractère"""
    if not folder.exists():
        print(f"[!] Dossier non trouvé: {folder}")
        return []
    
    files = set()
    for f in folder.glob("*.fcs"):
        files.add(str(f))
    for f in folder.glob("*.FCS"):
        files.add(str(f))
    
    return sorted(list(files))


def load_fcs_files(files: List[str], condition: str = "Unknown") -> List[ad.AnnData]:
    """
    Charge plusieurs fichiers FCS et retourne une liste d'AnnData.
    
    Args:
        files: Liste des chemins de fichiers FCS
        condition: Label de condition ("Sain" ou "Pathologique")
    
    Returns:
        Liste d'objets AnnData
    """
    # La ligne suivante crée la liste vide pour stocker les AnnData puis boucle sur chaque fichier (éviter le plantage complet)
    adatas = []
    
    for fpath in files:
        try:
            print(f"    Chargement: {Path(fpath).name}...", end=" ")
            
            # Lecture avec la fonction de base de flowsom
            adata = fs.io.read_FCS(fpath)
            
            # Ajouter les métadonnées avec un nombre de cellules qui sera égale a la forme de l'objet adata 
            n_cells = adata.shape[0]
            adata.obs['condition'] = condition # Rajoute la condition du fichier : "Sain" ou "Pathologique"
            adata.obs['file_origin'] = Path(fpath).name # Rajoute une observation avec Nom du fichier source (obs = One-dimensional annotation of observations)
            
            adatas.append(adata) # Ajoute à la liste des AnnData
            print(f"{n_cells:,} cellules")
            
        except Exception as e:
            print(f"Erreur: {e}")
    
    return adatas

# Logs sur le cahrgement des fichiers
print("="*60)
print("CHARGEMENT DES FICHIERS FCS")
print("="*60)

# Fichiers sains en fonction du mode défini
healthy_files = get_fcs_files(HEALTHY_FOLDER) if COMPARE_MODE else []
print(f"\nFichiers Sains (NBM): {len(healthy_files)}")

healthy_adatas = []
if healthy_files:
    healthy_adatas = load_fcs_files(healthy_files, condition="Sain")

# Fichiers sains en fonction du mode défini
patho_files = get_fcs_files(PATHOLOGICAL_FOLDER)
print(f"\nFichiers Pathologiques: {len(patho_files)}")

patho_adatas = []
if patho_files:
    patho_adatas = load_fcs_files(patho_files, condition="Pathologique")

# Résumé
print("\n" + "="*60)
print(f"RÉSUMÉ DU CHARGEMENT")
print(f"   Fichiers Sains chargés: {len(healthy_adatas)}")
print(f"   Fichiers Pathologiques chargés: {len(patho_adatas)}")
# Résumé a.shape = pour chaque AnnData, prend le nombre de cellules (lignes) et concatène si nécessaire
total_cells = sum([a.shape[0] for a in healthy_adatas + patho_adatas])
print(f"   Total cellules: {total_cells:,}")
print("="*60)

## 3. Exploration de la Structure des Données Brutes

Avant toute transformation, examinons la structure des données:
- Dimensions (cellules x marqueurs)
- Noms des colonnes (marqueurs)
- Types de données et plages de valeurs

In [ ]:
# CONCATÉNATION DES DONNÉES

# Combiner tous les AnnData défini dans la cellule précédente
all_adatas = healthy_adatas + patho_adatas

# Vérification
if len(all_adatas) == 0:
    raise ValueError("[X] Aucun fichier FCS chargé! Vérifiez les chemins.")

# Concaténer avec intersection des colonnes (communes à tous les fichiers) ligne par ligne
if len(all_adatas) > 1:
    combined_data = ad.concat(all_adatas, join='inner') # join='inner' pour ne garder que les marqueurs communs à changer par outer si on veut garder tous les marqueurs
else:
    combined_data = all_adatas[0].copy() # Si un seul fichier, juste copier pour éviter de mofifier l'original

print(f"Données combinées: {combined_data.shape}")
print(f"   → {combined_data.shape[0]:,} cellules")
print(f"   → {combined_data.shape[1]} marqueurs")

In [ ]:
# EXPLORATION DE LA STRUCTURE
print("="*70)
print("STRUCTURE DES DONNÉES")
print("="*70)

# Liste des marqueurs enregistré dans la varaible var_names = canaux (ici c'est bien un nom de variable)
var_names = list(combined_data.var_names)
print(f"\nMarqueurs ({len(var_names)}):")
for i, name in enumerate(var_names):
    print(f"   [{i:2d}] {name}")

# Identification des types de marqueurs car les recos indiquent d'enelever le scatter pour les analyses de clustering
print("\nClassification des marqueurs:")

#Ici le code n for n in var pose la question : "Est-ce qu'au moins UN des motifs de la liste scatter_patterns se trouve dans le nom actuel n ?"
scatter_patterns = ['FSC', 'SSC', 'TIME', 'EVENT']
scatter_markers = [n for n in var_names if any(p in n.upper() for p in scatter_patterns)]
fluor_markers = [n for n in var_names if n not in scatter_markers]

print(f"   Scatter/Time: {scatter_markers}")
print(f"   Fluorescence: {fluor_markers}")

# Statistiques de base
print("\nObservations (métadonnées):")
print(combined_data.obs.head(10))

In [ ]:
# CONVERSION EN DATAFRAME POUR EXPLORATION
HEADER = True
# Extraire la matrice de données
X = combined_data.X # Matrice des données (n_cells, n_markers)
if hasattr(X, 'toarray'): # Si sparse matrix, convertir en dense pour pandas
    X = X.toarray()

# Créer un DataFrame pandas pour faciliter l'exploration avec df comme commande pandas classique
df_raw = pd.DataFrame(X, columns=var_names) # Crée le DataFrame avec les noms de colonnes 
df_raw['condition'] = combined_data.obs['condition'].values # Ajoute une colonne condition
df_raw['file_origin'] = combined_data.obs['file_origin'].values # Ajoute une colonne file_origin

print("DataFrame créé pour exploration")
print(f"   Shape: {df_raw.shape}")
print("\nAperçu des données brutes:")
df_raw.head(50)

In [ ]:
# Stats descriptives marqueurs de fluorescence et scatter
print("Statistiques descriptives fluorescence")
display(df_raw[fluor_markers].describe())

print("\nStatistiques descriptives scatter")
display(df_raw[scatter_markers].describe())

## 4. Contrôle Qualité des données- Analyse des Distributions

Visualisation des distributions brutes pour identifier:
- Outliers et valeurs aberrantes
- Valeurs négatives (problème de compensation)
- NaN/Inf dans les données

In [ ]:
# Vérif des varaibles problématiques suite de l'exploration du dataset

print("ANALYSE DES DONNÉES BRUTES")
print("="*60)

# ========== MARQUEURS DE FLUORESCENCE ==========
print("\nMARQUEURS DE FLUORESCENCE")
print("-"*60)

# Vérifier NaN
nan_count = df_raw[fluor_markers].isna().sum()
print(f"\nValeurs NaN par marqueur:")
for marker, count in nan_count.items():
    if count > 0:
        print(f"   {marker}: {count:,} ({count/len(df_raw)*100:.2f}%)")
    
if nan_count.sum() == 0:
    print("   [OK] Aucun NaN détecté!")

# Vérifier Inf (valeur infinie) ex sur un post log 
inf_count = np.isinf(df_raw[fluor_markers]).sum()
print(f"\nValeurs Inf par marqueur:")
if inf_count.sum() == 0:
    print("   [OK] Aucun Inf détecté!")
else:
    for marker, count in inf_count.items():
        if count > 0:
            print(f"   {marker}: {count:,}")

# Vérifier valeurs négatives
neg_count = (df_raw[fluor_markers] < 0).sum()
print(f"\n➖ Valeurs négatives par marqueur:")
has_negatives = False
for marker, count in neg_count.items():
    if count > 0:
        has_negatives = True
        # Compter le nombre total de cellules valides (non-NaN) pour ce marqueur
        total_valid = df_raw[marker].notna().sum()
        print(f"   {marker}: {count:,} / {total_valid:,} ({count/total_valid*100:.2f}%)")
        
if not has_negatives:
    print("   [OK] Aucune valeur négative!")
else:
    print("\n   [!] Les valeurs négatives peuvent indiquer un problème de compensation")
    print("   → La transformation Arcsinh ou Logicle peut les gérer")

# ========== MARQUEURS SCATTER/TIME ==========
print("\n\nMARQUEURS SCATTER/TIME")
print("-"*60)

# Vérifier NaN
nan_count_scatter = df_raw[scatter_markers].isna().sum()
print(f"\nValeurs NaN par marqueur:")
for marker, count in nan_count_scatter.items():
    if count > 0:
        print(f"   {marker}: {count:,} ({count/len(df_raw)*100:.2f}%)")
    
if nan_count_scatter.sum() == 0:
    print("   [OK] Aucun NaN détecté!")

# Vérifier Inf
inf_count_scatter = np.isinf(df_raw[scatter_markers]).sum()
print(f"\nValeurs Inf par marqueur:")
if inf_count_scatter.sum() == 0:
    print("   [OK] Aucun Inf détecté!")
else:
    for marker, count in inf_count_scatter.items():
        if count > 0:
            print(f"   {marker}: {count:,}")

# Vérifier valeurs négatives
neg_count_scatter = (df_raw[scatter_markers] < 0).sum()
print(f"\n➖ Valeurs négatives par marqueur:")
has_negatives_scatter = False
for marker, count in neg_count_scatter.items():
    if count > 0:
        has_negatives_scatter = True
        total_valid = df_raw[marker].notna().sum()
        print(f"   {marker}: {count:,} / {total_valid:,} ({count/total_valid*100:.2f}%)")
        
if not has_negatives_scatter:
    print("   [OK] Aucune valeur négative!")
else:
    print("\n   ℹ️ Les valeurs négatives dans scatter sont rares mais possibles")

print("\n" + "="*60)

In [ ]:
# Histogrammes des distributions brutes pour explorer visuellement les données

# Sélectionner les marqueurs à visualiser (max 12 pour lisibilité)
markers_to_plot = fluor_markers[:12] if len(fluor_markers) > 12 else fluor_markers  # Opérateur ternaire: prendre 12 premiers si > 12, sinon tous

n_markers = len(markers_to_plot)  # Nombre de marqueurs à afficher
n_cols = 4  # 4 colonnes par ligne
n_rows = (n_markers + n_cols - 1) // n_cols  # Calcul nb lignes (division entière arrondie vers le haut)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))  # Créer grille n_rows × n_cols (largeur 20, hauteur 5 par ligne)
axes = axes.flatten() if n_markers > 1 else [axes]  # Aplatir tableau 2D en liste 1D pour itération facile

for i, marker in enumerate(markers_to_plot):  # Boucle sur chaque marqueur avec index i
    ax = axes[i]  # Récupérer le sous-graphique i
    data = df_raw[marker].dropna()  # Extraire données du marqueur et supprimer NaN
    
    ax.hist(data, bins=100, color='#89b4fa', alpha=0.7, edgecolor='none')  # Histogramme 100 barres, bleu, 70% opacité
    ax.set_title(marker, fontsize=11, fontweight='bold')  # Titre = nom du marqueur
    ax.set_xlabel('Valeur brute')  # Label axe X
    ax.set_ylabel('Count')  # Label axe Y = nombre de cellules
    ax.axvline(0, color='#f38ba8', linestyle='--', alpha=0.5, label='Zéro')  # Ligne verticale rouge à x=0
    
    # Statistiques min/max dans une boîte en haut à droite
    ax.text(0.98, 0.95, f'min: {data.min():.0f}\nmax: {data.max():.0f}',  # Texte avec stats
            transform=ax.transAxes, ha='right', va='top', fontsize=8,  # Coordonnées relatives (0-1), alignement
            bbox=dict(boxstyle='round', facecolor="#FFFFFF", alpha=0.8))  # Boîte grise arrondie semi-transparente

# Cacher les axes vides (si 10 marqueurs sur grille 3×4, cacher les 2 dernières cases)
for i in range(n_markers, len(axes)):
    axes[i].set_visible(False)

plt.suptitle('Distributions Brutes des Marqueurs (avant transformation)',  # Titre général
             fontsize=14, fontweight='bold', y=1.02)  # Décalé vers le haut
plt.tight_layout()  # Ajuster espacement auto
plt.show()  # Afficher

In [ ]:
# Histogrammes des marqueurs SCATTER/TIME pour exploration visuelle

# Sélectionner tous les marqueurs scatter (généralement peu nombreux)
scatter_to_plot = scatter_markers  # FSC, SSC, TIME

n_scatter = len(scatter_to_plot)  # Nombre de marqueurs scatter
n_cols_scatter = min(3, n_scatter)  # Max 3 colonnes pour les scatter
n_rows_scatter = (n_scatter + n_cols_scatter - 1) // n_cols_scatter  # Calcul nb lignes

fig, axes = plt.subplots(n_rows_scatter, n_cols_scatter, figsize=(18, 6*n_rows_scatter))  # Grille pour scatter (largeur 18, hauteur 6 par ligne)
axes = axes.flatten() if n_scatter > 1 else [axes]  # Aplatir en liste 1D

for i, marker in enumerate(scatter_to_plot):  # Boucle sur chaque marqueur scatter
    ax = axes[i]  # Sous-graphique i
    data = df_raw[marker].dropna()  # Données sans NaN
    
    ax.hist(data, bins=100, color='#a6e3a1', alpha=0.7, edgecolor='none')  # Vert pour différencier
    ax.set_title(marker, fontsize=12, fontweight='bold')  # Titre
    ax.set_xlabel('Valeur brute')  # Axe X
    ax.set_ylabel('Count')  # Axe Y
    
    # Statistiques complètes
    ax.text(0.02, 0.95, f'min: {data.min():.0f}\nmax: {data.max():.0f}\nmean: {data.mean():.0f}\nmedian: {data.median():.0f}',
            transform=ax.transAxes, ha='left', va='top', fontsize=8,
            bbox=dict(boxstyle='round', facecolor="#FFFFFF", alpha=0.8))

# Cacher axes vides
for i in range(n_scatter, len(axes)):
    axes[i].set_visible(False)

plt.suptitle('Distributions Scatter/Time (FSC, SSC, TIME)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Visualisation Interactive avec FlowKit + Bokeh

Utilisation native de FlowKit pour visualisations interactives :
- **Histogrammes** avec bins/ranges personnalisables
- **Scatter plots** interactifs avec zoom/pan
- **Contour plots** avec densité
- Rendu Bokeh pour l'interactivité (zoom, pan, hover)

📚 Documentation : https://flowkit.readthedocs.io/en/latest/index.html
https://www.frontiersin.org/journals/immunology/articles/10.3389/fimmu.2021.768541/full

---

### [!] IMPORTANT : Nomenclature FCS

**FlowKit utilise les PnN labels** (noms techniques), pas les PnS (descriptions).

- **PnN** = Nom technique (ex: `'Horizon V500-A'`) ← À utiliser
- **PnS** = Description bio (ex: `'CD45 KO'`) ← Non utilisable

**Exemple :** Pour CD45, utiliser `'Horizon V500-A'` (pas `'CD45 KO'`).

Exécutez la cellule suivante pour voir la correspondance PnN ↔ PnS.


In [ ]:
# Initialisation FlowKit et Bokeh + Création du Sample

if not FLOWKIT_AVAILABLE:
    fk_sample = None
else:
    try:
        from bokeh.plotting import show, output_notebook
        from bokeh.io import export_png
        output_notebook()
    except ImportError:
        pass
    
    # Utiliser les fichiers FCS déjà identifiés dans le notebook
    all_fcs_files = healthy_files + patho_files
    
    if all_fcs_files:
        example_fcs = str(all_fcs_files[0])
        fk_sample = fk.Sample(example_fcs)
    else:
        fk_sample = None

In [ ]:
# AFFICHER LES NOMS DE CANAUX EXACTS DU FICHIER FCS

if fk_sample is not None:
    print("="*80)
    print(f"CANAUX DU FICHIER FCS: {Path(example_fcs).name}")
    print("="*80)
    
    print(f"\nPnN Labels ({len(fk_sample.pnn_labels)} canaux) - NOMS À UTILISER DANS FLOWKIT:")
    print("-"*80)
    for i, label in enumerate(fk_sample.pnn_labels, 1):
        print(f"   [{i:2d}] '{label}'")
    
    # Afficher aussi les PnS labels (descriptions) si disponibles
    print(f"\n\nPnS Labels (descriptions):")
    print("-"*80)
    for i, label in enumerate(fk_sample.pns_labels, 1):
        print(f"   [{i:2d}] {label}")
    
else:
    print("[!] FlowKit Sample non chargé")

In [ ]:
# Définir les colonnes FSC et SSC pour les visualisations ultérieures
fsc_col = next((c for c in var_names if 'FSC-A' in c.upper() or 'FSC' in c.upper()), None)
ssc_col = next((c for c in var_names if 'SSC-A' in c.upper() or 'SSC' in c.upper()), None)

if fsc_col:
    print(f"[OK] FSC détecté: {fsc_col}")
if ssc_col:
    print(f"[OK] SSC détecté: {ssc_col}")

In [ ]:
# Histogramme basique FlowKit (échelle linéaire)
# source='raw' = données LINÉAIRES (non transformées)
# source='xform' = données transformées (Logicle)

if fk_sample is not None:
    CHANNEL = 'Horizon V500-A'  # Changer ici le channel à afficher
    
    # Histogramme en échelle LINÉAIRE (source='raw')
    p = fk_sample.plot_histogram(CHANNEL, source='raw', bins=256)
    
    # Forcer l'échelle linéaire sur les axes (pas log)
    p.xaxis.formatter.use_scientific = False  # Désactiver notation scientifique
    p.yaxis.formatter.use_scientific = False
    
    show(p)
    print(f" Histogramme {CHANNEL} - Échelle LINÉAIRE (source='raw')")

In [ ]:
# Scatter plot 2D interactif (échelle linéaire)
# source='raw' = données LINÉAIRES (non transformées)

if fk_sample is not None:
    x_channel = 'Horizon V500-A'
    y_channel = 'PE-A'
    
    # Scatter en échelle LINÉAIRE (source='raw')
    p = fk_sample.plot_scatter(x_channel, y_channel, source='raw', color_density=True)
    
    # Forcer l'échelle linéaire sur les axes (pas log)
    p.xaxis.formatter.use_scientific = False  # Désactiver notation scientifique
    p.yaxis.formatter.use_scientific = False
    
    show(p)
    print(f" Scatter {x_channel} vs {y_channel} - Échelle LINÉAIRE (source='raw')")

In [ ]:
# Histogramme 1D interactif avec Plotly (zoom, pan, hover)
# Sélectionner un marqueur à visualiser (modifiable)
MARKER_TO_PLOT = 'CD45 KO'  # Changer ici le nom exact du marqueur à visualiser

print(f"Visualisation: {MARKER_TO_PLOT}")

# Extraire les données
marker_data = df_raw[MARKER_TO_PLOT].dropna().values

# Importer plotly pour l'interactivité
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.io as pio
    
    # Configurer le renderer pour Jupyter (évite l'erreur nbformat)
    try:
        pio.renderers.default = 'notebook'
    except:
        try:
            pio.renderers.default = 'jupyterlab'
        except:
            pio.renderers.default = 'browser'
    
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("[!] Plotly non installé - pip install plotly")

if PLOTLY_AVAILABLE:
    # Créer une figure avec 4 subplots (2x2)
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f'{MARKER_TO_PLOT} - Brut (Linéaire)',
            f'{MARKER_TO_PLOT} - Arcsinh (cofactor=5)',
            f'{MARKER_TO_PLOT} - Logicle/Arcsinh',
            f'{MARKER_TO_PLOT} - Log10'
        ),
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # 1. Données brutes
    fig.add_trace(
        go.Histogram(x=marker_data, nbinsx=200, name='Brut',
                     marker_color='#89b4fa', opacity=0.7,
                     hovertemplate='Intensité: %{x:.1f}<br>Count: %{y}<extra></extra>'),
        row=1, col=1
    )
    
    # 2. Arcsinh cofactor=5
    marker_arcsinh = DataTransformer.arcsinh_transform(marker_data, cofactor=5)
    fig.add_trace(
        go.Histogram(x=marker_arcsinh, nbinsx=200, name='Arcsinh (5)',
                     marker_color='#a6e3a1', opacity=0.7,
                     hovertemplate='Intensité: %{x:.2f}<br>Count: %{y}<extra></extra>'),
        row=1, col=2
    )
    
    # 3. Logicle ou Arcsinh cofactor=150
    if FLOWKIT_AVAILABLE:
        marker_logicle = DataTransformer.logicle_transform(marker_data)
        transform_name = 'Logicle'
    else:
        marker_logicle = DataTransformer.arcsinh_transform(marker_data, cofactor=150)
        transform_name = 'Arcsinh (150)'
    
    fig.add_trace(
        go.Histogram(x=marker_logicle, nbinsx=200, name=transform_name,
                     marker_color='#f9e2af', opacity=0.7,
                     hovertemplate='Intensité: %{x:.2f}<br>Count: %{y}<extra></extra>'),
        row=2, col=1
    )
    
    # 4. Log10
    marker_log = DataTransformer.log_transform(marker_data, base=10, min_val=1)
    fig.add_trace(
        go.Histogram(x=marker_log, nbinsx=200, name='Log10',
                     marker_color='#cba6f7', opacity=0.7,
                     hovertemplate='Intensité: %{x:.2f}<br>Count: %{y}<extra></extra>'),
        row=2, col=2
    )
    
    # Mise en page
    fig.update_xaxes(title_text="Intensité brute", row=1, col=1)
    fig.update_xaxes(title_text="Intensité transformée", row=1, col=2)
    fig.update_xaxes(title_text="Intensité transformée", row=2, col=1)
    fig.update_xaxes(title_text="Intensité log10", row=2, col=2)
    
    fig.update_yaxes(title_text="Fréquence", row=1, col=1)
    fig.update_yaxes(title_text="Fréquence", row=1, col=2)
    fig.update_yaxes(title_text="Fréquence", row=2, col=1)
    fig.update_yaxes(title_text="Fréquence", row=2, col=2)
    
    # Thème sombre et configuration
    fig.update_layout(
        title_text=f'Comparaison Transformations - {MARKER_TO_PLOT} ({len(marker_data):,} cellules)',
        title_font_size=16,
        height=900,
        showlegend=False,
        template='plotly_dark',
        hovermode='x unified'
    )
    
    # Afficher avec gestion d'erreur
    try:
        fig.show()
        print(f"\n[OK] Visualisation interactive générée")
        print(f"    Utilisez les outils Plotly: Zoom (box select), Pan, Reset, Download")
    except Exception as e:
        print(f"\n[!] Erreur affichage Plotly: {e}")
        print("   → Affichage en HTML dans le notebook...")
        
        # Alternative: Afficher le HTML directement dans le notebook
        try:
            from IPython.display import HTML, display
            html_str = fig.to_html(include_plotlyjs='cdn', include_mathjax='cdn')
            display(HTML(html_str))
            print(f"   [OK] Graphique affiché en HTML (pleinement interactif)")
        except Exception as e2:
            print(f"   [X] Erreur HTML: {e2}")
            # Dernier recours: sauvegarder en fichier
            html_file = 'plotly_visualization.html'
            fig.write_html(html_file)
            print(f"   → Fichier sauvegardé: {html_file}")
            print(f"   → Ouvrez ce fichier dans votre navigateur pour l'interactivité complète")
    
    print(f"   Cellules: {len(marker_data):,}")
    print(f"   Min brut: {marker_data.min():.2f} | Max brut: {marker_data.max():.2f}")
else:
    # Fallback matplotlib si Plotly non disponible
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    ax = axes[0]
    ax.hist(marker_data, bins=200, color='#89b4fa', alpha=0.7, edgecolor='none')
    ax.set_title(f'{MARKER_TO_PLOT} - Brut (Linéaire)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité brute')
    ax.set_ylabel('Fréquence')
    
    ax = axes[1]
    marker_arcsinh = DataTransformer.arcsinh_transform(marker_data, cofactor=5)
    ax.hist(marker_arcsinh, bins=200, color='#a6e3a1', alpha=0.7, edgecolor='none')
    ax.set_title(f'{MARKER_TO_PLOT} - Arcsinh (cofactor=5)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité transformée')
    ax.set_ylabel('Fréquence')
    
    ax = axes[2]
    if FLOWKIT_AVAILABLE:
        marker_logicle = DataTransformer.logicle_transform(marker_data)
        ax.hist(marker_logicle, bins=200, color='#f9e2af', alpha=0.7, edgecolor='none')
        ax.set_title(f'{MARKER_TO_PLOT} - Logicle', fontsize=12, fontweight='bold')
    else:
        marker_logicle = DataTransformer.arcsinh_transform(marker_data, cofactor=150)
        ax.hist(marker_logicle, bins=200, color='#f9e2af', alpha=0.7, edgecolor='none')
        ax.set_title(f'{MARKER_TO_PLOT} - Arcsinh (cofactor=150)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité transformée')
    ax.set_ylabel('Fréquence')
    
    ax = axes[3]
    marker_log = DataTransformer.log_transform(marker_data, base=10, min_val=1)
    ax.hist(marker_log, bins=200, color='#cba6f7', alpha=0.7, edgecolor='none')
    ax.set_title(f'{MARKER_TO_PLOT} - Log10', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité transformée (log10)')
    ax.set_ylabel('Fréquence')
    
    plt.suptitle(f'Comparaison Transformations - {MARKER_TO_PLOT} ({len(marker_data):,} cellules)', 
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

## 5. Pre-Gating: Élimination des Débris, Doublets et Sélection des Populations

Application du pre-gating séquentiel en 4 étapes:

1. **Gate 1 - Débris (SSC-A vs FSC-A)**: Exclusion des débris et événements hors-limites
2. **Gate 2 - Doublets (FSC-H vs FSC-A)**: Exclusion des agrégats cellulaires
3. **Gate 3 - Leucocytes CD45+ (CD45 vs SSC-A)**: Sélection des leucocytes — **Gate principal**
4. **Gate 4 - Blastes CD34+ (optionnel)**: Sous-sélection CD34 bright + SSC low

### Choix du mode de gating : `GATING_MODE`

| Mode | Méthode | Avantage | Inconvénient |
|------|---------|----------|-------------|
| `"manual"` | Percentiles fixes (PreGating) | Reproductible, rapide | Seuils arbitraires, perte si échantillon propre |
| `"auto"` | GMM adaptatif (AutoGating) | Trouve le creux réel entre populations, s'adapte | Dépend de la forme des distributions |

### Stratégie de gating classique en cytométrie:
```
Événements totaux
    └── Gate Débris (SSC-A vs FSC-A) → Cellules viables
            └── Gate Singlets (FSC-H vs FSC-A) → Cellules individuelles
                    └── Gate CD45+ (CD45 vs SSC-A) → Leucocytes
                            └── [optionnel] Gate CD34+ (CD34 vs SSC-A) → Blastes
```

### Paramètres:
- **`GATING_MODE`**: `"manual"` (percentiles) ou `"auto"` (GMM adaptatif)
- **`FILTER_BLASTS`**: True = sous-filtrage CD34+ | False = tous les CD45+ conservés

In [ ]:
# =============================================================================
# APPLICATION DU PRE-GATING SÉQUENTIEL (4 ÉTAPES)
# =============================================================================
# Stratégie de gating hiérarchique:
# 1. SSC-A vs FSC-A → Exclure débris
# 2. FSC-H vs FSC-A → Exclure doublets (singlets line)
# 3. CD45 vs SSC-A  → Sélectionner leucocytes (GATE PRINCIPAL)
# 4. CD34 vs SSC-A  → Sélectionner blastes (optionnel, si FILTER_BLASTS=True)
# =============================================================================

# ===================== OPTIONS DE PRE-GATING =====================
APPLY_PREGATING = True  # Activer/désactiver le pre-gating complet

# =================================================================
# MODE DE GATING: MANUEL (percentiles) ou AUTOMATIQUE (GMM adaptatif)
# =================================================================
# "manual" : Seuils fixes basés sur les percentiles (classe PreGating)
#            → Coupe toujours aux mêmes percentiles, peu importe la distribution
#            → Rapide et reproductible, mais perd des données si échantillon propre
#
# "auto"   : Gating adaptatif par modèles de mélange gaussien (classe AutoGating)
#            → Trouve le "creux" réel entre les populations (débris vs cellules)
#            → S'adapte automatiquement: 10% de débris → porte à 10%, pas 2%
#            → Pour les doublets: modélise la diagonale FSC-A/FSC-H statistiquement
#            → Inspiré de CytoPy AutonomousGate (GMM/KDE)
# =================================================================
GATING_MODE = "auto"  # ← CHANGER ICI: "manual" ou "auto"

# =================================================================
# MODE SPÉCIAL: BLASTES CD34+ (PATHO) vs MOELLE NORMALE (SAIN)
# =================================================================
# Si activé:
#   - Fichier PATHOLOGIQUE: Gate complet (débris + doublets + CD45+ + éventuellement CD34+)
#   - Fichier SAIN: Gate partiel (débris + doublets + CD45+ UNIQUEMENT, pas de CD34+)
# Résultat: Population sélectionnée du patient + Leucocytes normaux de la moelle saine
# =================================================================
MODE_BLASTES_VS_NORMAL = True  # [!] ACTIVER POUR CE MODE SPÉCIAL

# Gate 1: Débris (SSC-A vs FSC-A) - True/False
GATE_DEBRIS = True
DEBRIS_MIN_PERCENTILE = 1.0   # [Manual] Exclure les 1% les plus bas (débris/bruit)
DEBRIS_MAX_PERCENTILE = 99.0  # [Manual] Exclure les 1% les plus hauts (saturés)

# Gate 2: Doublets (FSC-H vs FSC-A) - True/False
GATE_DOUBLETS = True
RATIO_MIN = 0.6   # [Manual] Ratio FSC-A/FSC-H minimum (cellules trop petites)
RATIO_MAX = 1.4   # [Manual] Ratio FSC-A/FSC-H maximum (doublets/agrégats)

# Gate 3: Leucocytes CD45+ (CD45 vs SSC-A) - GATE PRINCIPAL
GATE_CD45 = True  # [!] Sélectionner les leucocytes CD45+
CD45_THRESHOLD_PERCENTILE = 10  # [Manual] Seuil CD45+ (exclure les 10% les plus bas)

# Gate 4: Blastes CD34+ (CD34 vs SSC-A) - OPTIONNEL (sous-population des CD45+)
# =================================================================
# FILTER_BLASTS: Contrôle le sous-filtrage CD34+ après le gate CD45+
#   - True:  Après CD45+, applique un gate CD34+ → Blastes uniquement
#   - False: Conserve TOUS les leucocytes CD45+ pour FlowSOM
# =================================================================
FILTER_BLASTS = False  # [!] Mettre False pour garder tous les leucocytes CD45+

# Paramètres CD34 (utilisés uniquement si FILTER_BLASTS=True)
CD34_THRESHOLD_PERCENTILE = 85  # [Manual] Seuil CD34+ (top 15% = percentile 85)
USE_SSC_FILTER_FOR_BLASTS = True  # Combiner avec SSC low pour blastes purs
SSC_MAX_PERCENTILE_BLASTS = 60  # [Manual] SSC maximum pour blastes (faible granularité)

# =================================================================
# VALIDATION DES PARAMÈTRES
# =================================================================
assert GATING_MODE in ("manual", "auto"), f"GATING_MODE doit être 'manual' ou 'auto', reçu: '{GATING_MODE}'"

if GATING_MODE == "auto" and not SKLEARN_AVAILABLE:
    print("[!] ATTENTION: scikit-learn requis pour GATING_MODE='auto'")
    print("    → Fallback automatique vers mode 'manual'")
    GATING_MODE = "manual"

# Vérification de cohérence pour MODE_BLASTES_VS_NORMAL
if MODE_BLASTES_VS_NORMAL and not COMPARE_MODE:
    print("[!] ATTENTION: MODE_BLASTES_VS_NORMAL nécessite COMPARE_MODE=True")
    print("    → Le mode a besoin de fichiers Sain + Patho pour fonctionner")
    print("    → Désactivation automatique du mode différentiel")
    MODE_BLASTES_VS_NORMAL = False

# Données avant gating
X_raw = combined_data.X
if hasattr(X_raw, 'toarray'):
    X_raw = X_raw.toarray()
n_before = X_raw.shape[0]

# Récupérer le vecteur de conditions pour le mode différentiel
conditions = combined_data.obs['condition'].values

print("="*70)
print(" PRE-GATING SÉQUENTIEL - STRATÉGIE EN 4 ÉTAPES")
print("="*70)
print(f"\n Événements initiaux: {n_before:,}")

# Affichage du mode de gating
mode_label = "AUTOMATIQUE (GMM adaptatif)" if GATING_MODE == "auto" else "MANUEL (percentiles fixes)"
print(f"\n Mode de gating: {mode_label}")
if GATING_MODE == "auto":
    print("    → Les seuils sont calculés automatiquement par modèle de mélange gaussien")
    print("    → Les paramètres [Manual] ci-dessus sont IGNORÉS")
else:
    print("    → Seuils basés sur les percentiles configurés ci-dessus")

# Affichage mode spécial
if MODE_BLASTES_VS_NORMAL:
    print("\n [!] MODE BLASTES vs MOELLE NORMALE ACTIVÉ (GATING ASYMÉTRIQUE)")
    if FILTER_BLASTS:
        print("     - Patho: Gate complet (débris + doublets + CD45+ + CD34+) → Blastes seuls")
    else:
        print("     - Patho: Gate (débris + doublets + CD45+) → Leucocytes CD45+ stricts")
    print("     - Sain:  Gate (débris + doublets UNIQUEMENT) → Toutes les cellules conservées (pas de gate CD45)")
    n_patho = (conditions == "Pathologique").sum()
    n_sain = (conditions == "Sain").sum()
    print(f"     - Cellules Patho: {n_patho:,}")
    print(f"     - Cellules Sain: {n_sain:,}")

print(f"\n Configuration:")
print(f"   [Gate 1] Débris (SSC-A/FSC-A):     {'[OK] ACTIVÉ' if GATE_DEBRIS else '[X] DÉSACTIVÉ'}")
print(f"   [Gate 2] Doublets (FSC-H/FSC-A):   {'[OK] ACTIVÉ' if GATE_DOUBLETS else '[X] DÉSACTIVÉ'}")
if MODE_BLASTES_VS_NORMAL and GATE_CD45:
    print(f"   [Gate 3] Leucocytes CD45+:         [OK] PATHO UNIQUEMENT (gating asymétrique — Sain: pas de gate CD45)")
else:
    print(f"   [Gate 3] Leucocytes CD45+:         {'[OK] ACTIVÉ' if GATE_CD45 else '[X] DÉSACTIVÉ'}")
if FILTER_BLASTS:
    if MODE_BLASTES_VS_NORMAL:
        print(f"   [Gate 4] Blastes CD34+:            [OK] PATHO UNIQUEMENT (mode différentiel)")
    else:
        print(f"   [Gate 4] Blastes CD34+:            [OK] ACTIVÉ (FILTER_BLASTS=True)")
else:
    print(f"   [Gate 4] Blastes CD34+:            [X] DÉSACTIVÉ (FILTER_BLASTS=False → tous les CD45+ conservés)")

if APPLY_PREGATING:
    # Initialisation des masques
    mask_debris = np.ones(n_before, dtype=bool)
    mask_singlets = np.ones(n_before, dtype=bool)
    mask_cd45 = np.ones(n_before, dtype=bool)
    mask_cd34 = np.ones(n_before, dtype=bool)
    
    print("\n" + "-"*70)
    
    # ========== GATE 1: DÉBRIS (SSC-A vs FSC-A) ==========
    if GATE_DEBRIS:
        print(f"\n GATE 1: Exclusion des débris (SSC-A vs FSC-A) [{GATING_MODE.upper()}]")
        if GATING_MODE == "auto":
            mask_debris = AutoGating.auto_gate_debris(X_raw, var_names)
        else:
            mask_debris = PreGating.gate_debris_polygon(
                X_raw, var_names,
                auto_percentiles=True,
                min_pct=DEBRIS_MIN_PERCENTILE,
                max_pct=DEBRIS_MAX_PERCENTILE
            )
        n_after_debris = mask_debris.sum()
        n_excluded_debris = n_before - n_after_debris
        if GATING_MODE == "manual":
            print(f"   Percentiles: [{DEBRIS_MIN_PERCENTILE}%, {DEBRIS_MAX_PERCENTILE}%]")
        print(f"   → Conservés: {n_after_debris:,} ({n_after_debris/n_before*100:.1f}%)")
        print(f"   → Exclus (débris): {n_excluded_debris:,} ({n_excluded_debris/n_before*100:.1f}%)")
    else:
        print("\n GATE 1: Débris - SKIP")
    
    # ========== GATE 2: DOUBLETS (FSC-H vs FSC-A) ==========
    if GATE_DOUBLETS:
        print(f"\n GATE 2: Exclusion des doublets (FSC-H vs FSC-A) [{GATING_MODE.upper()}]")
        if GATING_MODE == "auto":
            mask_singlets = AutoGating.auto_gate_singlets(X_raw, var_names)
        else:
            mask_singlets = PreGating.gate_singlets(
                X_raw, var_names,
                ratio_min=RATIO_MIN,
                ratio_max=RATIO_MAX
            )
        # Appliquer sur les cellules déjà filtrées par gate 1
        mask_after_g1_g2 = mask_debris & mask_singlets
        n_after_singlets = mask_after_g1_g2.sum()
        n_doublets = mask_debris.sum() - n_after_singlets
        if GATING_MODE == "manual":
            print(f"   Ratio FSC-A/FSC-H: [{RATIO_MIN}, {RATIO_MAX}]")
        print(f"   → Conservés (singlets): {n_after_singlets:,}")
        print(f"   → Exclus (doublets): {n_doublets:,}")
    else:
        print("\n GATE 2: Doublets - SKIP")
    
    # ========== GATE 3: LEUCOCYTES CD45+ (CD45 vs SSC-A) — GATE PRINCIPAL ==========
    # LOGIQUE ASYMÉTRIQUE: Pathologique → CD45 strict | Sain → Pas de gate CD45
    if GATE_CD45:
        if MODE_BLASTES_VS_NORMAL:
            print(f"\n GATE 3: Sélection ASYMÉTRIQUE CD45+ [{GATING_MODE.upper()}]")
            print("   → Patho: Gate CD45+ STRICT appliqué (élimination CD45-)")
            print("   → Sain:  Gate CD45+ IGNORÉ (toutes cellules conservées — progéniteurs, CD45 low/neg inclus)")
            
            # Calculer le masque CD45+ sur TOUTES les données
            if GATING_MODE == "auto":
                mask_cd45_full = AutoGating.auto_gate_cd45(X_raw, var_names)
            else:
                mask_cd45_full = PreGating.gate_cd45_positive(
                    X_raw, var_names,
                    threshold_percentile=CD45_THRESHOLD_PERCENTILE
                )
            
            # Appliquer le gate CD45 UNIQUEMENT aux cellules pathologiques
            mask_patho_cd45 = (conditions == "Pathologique")
            mask_sain_cd45 = (conditions == "Sain")
            
            # Masque CD45: True pour Sain (on garde tout), mask_cd45_full pour Patho
            mask_cd45 = np.ones(n_before, dtype=bool)
            mask_cd45[mask_patho_cd45] = mask_cd45_full[mask_patho_cd45]
            # Sain: mask_cd45 reste True → aucun filtrage CD45
            
            # Stats par condition
            n_patho_g12 = (mask_patho_cd45 & mask_debris & mask_singlets).sum()
            n_patho_cd45_kept = (mask_patho_cd45 & mask_debris & mask_singlets & mask_cd45).sum()
            n_patho_cd45_excl = n_patho_g12 - n_patho_cd45_kept
            n_sain_g12 = (mask_sain_cd45 & mask_debris & mask_singlets).sum()
            
            if GATING_MODE == "manual":
                print(f"   Seuil CD45+: percentile {CD45_THRESHOLD_PERCENTILE} (appliqué PATHO uniquement)")
            print(f"   → Patho CD45+ conservés: {n_patho_cd45_kept:,} / {n_patho_g12:,} ({n_patho_cd45_kept/max(n_patho_g12,1)*100:.1f}%)")
            print(f"   → Patho CD45- exclus:    {n_patho_cd45_excl:,}")
            print(f"   → Sain conservés (100%):  {n_sain_g12:,} / {n_sain_g12:,} (aucun gate CD45)")
            
            mask_after_g1_g2_g3 = mask_debris & mask_singlets & mask_cd45
            n_after_cd45 = mask_after_g1_g2_g3.sum()
            n_cd45_excluded = (mask_debris & mask_singlets).sum() - n_after_cd45
            print(f"   → Total après Gate 3: {n_after_cd45:,} (exclus CD45: {n_cd45_excluded:,} — Patho uniquement)")
        else:
            print(f"\n GATE 3: Sélection des leucocytes CD45+ (GATE PRINCIPAL) [{GATING_MODE.upper()}]")
            if GATING_MODE == "auto":
                mask_cd45 = AutoGating.auto_gate_cd45(X_raw, var_names)
            else:
                mask_cd45 = PreGating.gate_cd45_positive(
                    X_raw, var_names,
                    threshold_percentile=CD45_THRESHOLD_PERCENTILE
                )
            mask_after_g1_g2_g3 = mask_debris & mask_singlets & mask_cd45
            n_after_cd45 = mask_after_g1_g2_g3.sum()
            n_cd45_excluded = (mask_debris & mask_singlets).sum() - n_after_cd45
            if GATING_MODE == "manual":
                print(f"   Seuil CD45+: percentile {CD45_THRESHOLD_PERCENTILE} (exclure les {CD45_THRESHOLD_PERCENTILE}% les plus bas)")
            print(f"   → Leucocytes CD45+ conservés: {n_after_cd45:,}")
            print(f"   → Exclus (CD45-): {n_cd45_excluded:,}")
    else:
        print("\n GATE 3: CD45+ - SKIP")
    
    # ========== GATE 4: BLASTES CD34+ (optionnel, conditionné par FILTER_BLASTS) ==========
    if FILTER_BLASTS:
        # Mode différentiel: appliquer CD34+ gate UNIQUEMENT sur les cellules pathologiques
        if MODE_BLASTES_VS_NORMAL:
            print(f"\n GATE 4: Sélection DIFFÉRENTIELLE des blastes CD34+ [{GATING_MODE.upper()}]")
            print("   → Patho: Gate CD34+ appliqué (blastes uniquement)")
            print("   → Sain: Gate CD34+ IGNORÉ (tous les leucocytes CD45+ conservés)")
            
            # Calculer le masque CD34+ sur TOUTES les données
            if GATING_MODE == "auto":
                mask_cd34_full = AutoGating.auto_gate_cd34(
                    X_raw, var_names,
                    use_ssc_filter=USE_SSC_FILTER_FOR_BLASTS
                )
            else:
                mask_cd34_full = PreGating.gate_cd34_blasts(
                    X_raw, var_names,
                    threshold_percentile=CD34_THRESHOLD_PERCENTILE,
                    use_ssc_filter=USE_SSC_FILTER_FOR_BLASTS,
                    ssc_max_percentile=SSC_MAX_PERCENTILE_BLASTS
                )
            
            # Appliquer le gate CD34+ UNIQUEMENT aux cellules pathologiques
            mask_patho = (conditions == "Pathologique")
            mask_sain = (conditions == "Sain")
            
            # Masque CD34: True pour sain (on garde tout), mask_cd34_full pour patho
            mask_cd34 = np.ones(n_before, dtype=bool)
            mask_cd34[mask_patho] = mask_cd34_full[mask_patho]
            
            # Stats
            n_patho_before = mask_patho.sum()
            n_patho_cd34 = (mask_patho & mask_cd34_full).sum()
            n_sain_kept = mask_sain.sum()
            
            if GATING_MODE == "manual":
                print(f"   Seuil CD34+: top {100-CD34_THRESHOLD_PERCENTILE:.0f}% (percentile {CD34_THRESHOLD_PERCENTILE})")
                if USE_SSC_FILTER_FOR_BLASTS:
                    print(f"   Filtre SSC low: ≤ percentile {SSC_MAX_PERCENTILE_BLASTS}")
            print(f"   → Patho: {n_patho_cd34:,} blastes / {n_patho_before:,} ({n_patho_cd34/n_patho_before*100:.1f}%)")
            print(f"   → Sain: {n_sain_kept:,} leucocytes conservés (100%)")
            
        else:
            print(f"\n GATE 4: Sélection des blastes CD34+ (toutes conditions) [{GATING_MODE.upper()}]")
            if GATING_MODE == "auto":
                mask_cd34 = AutoGating.auto_gate_cd34(
                    X_raw, var_names,
                    use_ssc_filter=USE_SSC_FILTER_FOR_BLASTS
                )
            else:
                mask_cd34 = PreGating.gate_cd34_blasts(
                    X_raw, var_names,
                    threshold_percentile=CD34_THRESHOLD_PERCENTILE,
                    use_ssc_filter=USE_SSC_FILTER_FOR_BLASTS,
                    ssc_max_percentile=SSC_MAX_PERCENTILE_BLASTS
                )
            if GATING_MODE == "manual":
                print(f"   Seuil CD34+: top {100-CD34_THRESHOLD_PERCENTILE:.0f}% (percentile {CD34_THRESHOLD_PERCENTILE})")
                if USE_SSC_FILTER_FOR_BLASTS:
                    print(f"   Filtre SSC low: ≤ percentile {SSC_MAX_PERCENTILE_BLASTS}")
    else:
        print("\n GATE 4: Blastes CD34+ - SKIP (FILTER_BLASTS=False)")
        print("   → Tous les leucocytes CD45+ seront conservés pour FlowSOM")
    
    # ========== MASQUE FINAL COMBINÉ ==========
    mask_final = mask_debris & mask_singlets & mask_cd45 & mask_cd34
    n_final = mask_final.sum()
    n_excluded = n_before - n_final
    
    print("\n" + "="*70)
    print(f" RÉSUMÉ DU PRE-GATING [{mode_label}]")
    print("="*70)
    print(f"   Événements initiaux:       {n_before:,}")
    print(f"   Après Gate 1 (débris):     {mask_debris.sum():,}")
    print(f"   Après Gate 2 (doublets):   {(mask_debris & mask_singlets).sum():,}")
    print(f"   Après Gate 3 (CD45+):      {(mask_debris & mask_singlets & mask_cd45).sum():,}")
    
    # --- Détails par condition si MODE_BLASTES_VS_NORMAL ---
    if MODE_BLASTES_VS_NORMAL:
        mask_patho = (conditions == "Pathologique")
        mask_sain = (conditions == "Sain")
        n_patho_total = mask_patho.sum()
        n_sain_total = mask_sain.sum()
        n_patho_final = (mask_final & mask_patho).sum()
        n_sain_final = (mask_final & mask_sain).sum()
        
        print(f"\n   {'─'*55}")
        print(f"   DÉTAIL PAR CONDITION (GATING ASYMÉTRIQUE)")
        print(f"   {'─'*55}")
        print(f"   PATHOLOGIQUE (CD45 strict):")
        print(f"     Initial:                 {n_patho_total:,}")
        print(f"     Après débris+doublets:   {(mask_patho & mask_debris & mask_singlets).sum():,}")
        print(f"     Après CD45+ (strict):    {(mask_patho & mask_debris & mask_singlets & mask_cd45).sum():,}")
        print(f"     Final conservé:          {n_patho_final:,} ({n_patho_final/max(n_patho_total,1)*100:.1f}%)")
        print(f"   SAIN / NBM (pas de gate CD45):")
        print(f"     Initial:                 {n_sain_total:,}")
        print(f"     Après débris+doublets:   {(mask_sain & mask_debris & mask_singlets).sum():,}")
        print(f"     CD45 gate:               NON APPLIQUÉ (toutes cellules conservées)")
        print(f"     Final conservé:          {n_sain_final:,} ({n_sain_final/max(n_sain_total,1)*100:.1f}%)")
        print(f"   {'─'*55}")
    
    if FILTER_BLASTS:
        if MODE_BLASTES_VS_NORMAL:
            print(f"   Après Gate 4 (CD34+ patho): {n_final:,}")
            print(f"\n   [MODE BLASTES vs MOELLE NORMALE]")
            print(f"   Blastes CD34+ (patho):     {n_patho_final:,}")
            print(f"   Cellules normales (sain):  {n_sain_final:,}")
            print(f"   ─────────────────────────────────────")
            print(f"   [OK] TOTAL CONSERVÉ:       {n_final:,} ({n_final/n_before*100:.1f}%)")
            print(f"   [X] TOTAL EXCLUS:          {n_excluded:,} ({n_excluded/n_before*100:.1f}%)")
            print(f"\n   → Prêt pour FlowSOM: Blastes purs + Cellules normales (moelle saine complète)")
        else:
            print(f"   Après Gate 4 (CD34+):      {n_final:,}")
            print(f"\n   [OK] ÉVÉNEMENTS CONSERVÉS: {n_final:,} ({n_final/n_before*100:.1f}%)")
            print(f"   [X] ÉVÉNEMENTS EXCLUS: {n_excluded:,} ({n_excluded/n_before*100:.1f}%)")
    else:
        if MODE_BLASTES_VS_NORMAL:
            print(f"\n   [MODE ASYMÉTRIQUE — LEUCOCYTES vs MOELLE NORMALE]")
            print(f"   Patho (CD45+ stricts):     {n_patho_final:,} ({n_patho_final/max(n_patho_total,1)*100:.1f}% du fichier patient)")
            print(f"   Sain (toutes cellules):    {n_sain_final:,} ({n_sain_final/max(n_sain_total,1)*100:.1f}% du fichier NBM)")
            print(f"   ─────────────────────────────────────")
            print(f"   [OK] TOTAL CONSERVÉ:       {n_final:,} ({n_final/n_before*100:.1f}%)")
            print(f"   [X] TOTAL EXCLUS:          {n_excluded:,} ({n_excluded/n_before*100:.1f}%)")
            print(f"\n   → Prêt pour FlowSOM: Leucocytes CD45+ (patho) + Moelle normale complète (sain)")
        else:
            population_type = "Leucocytes CD45+" if GATE_CD45 else "Cellules"
            print(f"\n   [OK] {population_type} CONSERVÉS: {n_final:,} ({n_final/n_before*100:.1f}%)")
            print(f"   [X] ÉVÉNEMENTS EXCLUS: {n_excluded:,} ({n_excluded/n_before*100:.1f}%)")
            print(f"\n   → Prêt pour FlowSOM: Population CD45+ complète (pas de sous-sélection CD34+)")
    
else:
    print("\n PRE-GATING COMPLÈTEMENT DÉSACTIVÉ")
    print("="*70)
    print(f"   → Toutes les {n_before:,} cellules seront conservées")
    mask_final = np.ones(n_before, dtype=bool)
    n_final = n_before
    mask_debris = np.ones(n_before, dtype=bool)
    mask_singlets = np.ones(n_before, dtype=bool)
    mask_cd45 = np.ones(n_before, dtype=bool)
    mask_cd34 = np.ones(n_before, dtype=bool)

In [ ]:
# =============================================================================
# VISUALISATION PROFESSIONNELLE DES ÉTAPES DE GATING
# =============================================================================
# Graphiques SÉPARÉS et BIEN DÉFINIS pour chaque étape
# Style professionnel type FlowJo/Kaluza
# =============================================================================

from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects

# =============================================================================
# FONCTIONS DE VISUALISATION
# =============================================================================

def format_axis(value, pos):
    """Format intelligent des axes (K pour milliers, M pour millions)"""
    if abs(value) >= 1e6:
        return f'{value/1e6:.1f}M'
    elif abs(value) >= 1e3:
        return f'{value/1e3:.0f}K'
    return f'{value:.0f}'


def plot_density(ax, x, y, title, xlabel, ylabel, n_bins=120):
    """Scatter plot avec densité 2D (style FlowJo)"""
    # Nettoyer
    valid = np.isfinite(x) & np.isfinite(y)
    x, y = x[valid], y[valid]
    
    if len(x) < 100:
        ax.text(0.5, 0.5, 'Données insuffisantes', ha='center', va='center', 
                transform=ax.transAxes, fontsize=14, color='white')
        ax.set_facecolor('#1e1e2e')
        return
    
    # Limites
    x_lo, x_hi = np.percentile(x, [0.5, 99.5])
    y_lo, y_hi = np.percentile(y, [0.5, 99.5])
    
    # Colormap densité
    cmap = LinearSegmentedColormap.from_list('density', 
        ['#0d0d0d', '#1a1a2e', '#0077b6', '#00b4d8', '#90e0ef', '#f9e2af', '#ffffff'])
    
    # Histogramme 2D
    h = ax.hist2d(x, y, bins=n_bins, 
                  range=[[x_lo, x_hi], [y_lo, y_hi]],
                  cmap=cmap, norm=plt.matplotlib.colors.LogNorm(vmin=1))
    
    # Style
    ax.set_xlabel(xlabel, fontsize=13, fontweight='bold', color='white')
    ax.set_ylabel(ylabel, fontsize=13, fontweight='bold', color='white')
    ax.set_title(title, fontsize=14, fontweight='bold', color='white', pad=12)
    ax.set_facecolor('#1e1e2e')
    ax.tick_params(colors='white', labelsize=11)
    ax.xaxis.set_major_formatter(FuncFormatter(format_axis))
    ax.yaxis.set_major_formatter(FuncFormatter(format_axis))
    
    for spine in ax.spines.values():
        spine.set_color('#45475a')
        spine.set_linewidth(1.5)
    
    # Colorbar
    cbar = plt.colorbar(h[3], ax=ax, shrink=0.85)
    cbar.ax.tick_params(colors='white', labelsize=9)
    cbar.set_label('Densité', color='white', fontsize=11)
    
    return h


def plot_gating(ax, x, y, mask, title, xlabel, ylabel, 
                label_in='Conservés', label_out='Exclus', max_pts=100000):
    """Scatter plot avec overlay gating (vert=conservés, rouge=exclus)"""
    # Nettoyer
    valid = np.isfinite(x) & np.isfinite(y)
    x, y, mask = x[valid], y[valid], mask[valid]
    
    if len(x) < 100:
        ax.text(0.5, 0.5, 'Données insuffisantes', ha='center', va='center',
                transform=ax.transAxes, fontsize=14, color='white')
        ax.set_facecolor('#1e1e2e')
        return
    
    # Sous-échantillonner
    if len(x) > max_pts:
        idx = np.random.choice(len(x), max_pts, replace=False)
        x, y, mask = x[idx], y[idx], mask[idx]
    
    # Couleurs
    c_out = '#f38ba8'  # Rouge pastel
    c_in = '#a6e3a1'   # Vert pastel
    
    # Tracer exclus (fond)
    ax.scatter(x[~mask], y[~mask], s=4, c=c_out, alpha=0.3, 
               label=label_out, edgecolors='none', rasterized=True)
    # Tracer conservés (avant-plan)
    ax.scatter(x[mask], y[mask], s=5, c=c_in, alpha=0.5, 
               label=label_in, edgecolors='none', rasterized=True)
    
    # Stats
    n_tot = len(x)
    n_in = mask.sum()
    pct = n_in / n_tot * 100 if n_tot > 0 else 0
    
    # Style
    ax.set_xlabel(xlabel, fontsize=13, fontweight='bold', color='white')
    ax.set_ylabel(ylabel, fontsize=13, fontweight='bold', color='white')
    ax.set_title(f'{title}\n{n_in:,} / {n_tot:,} ({pct:.1f}%)', 
                fontsize=14, fontweight='bold', color='white', pad=12)
    ax.set_facecolor('#1e1e2e')
    ax.tick_params(colors='white', labelsize=11)
    ax.xaxis.set_major_formatter(FuncFormatter(format_axis))
    ax.yaxis.set_major_formatter(FuncFormatter(format_axis))
    
    for spine in ax.spines.values():
        spine.set_color('#45475a')
        spine.set_linewidth(1.5)
    
    ax.legend(loc='upper right', fontsize=10, markerscale=3,
              facecolor='#313244', labelcolor='white', edgecolor='#45475a')
    
    # Limites
    x_lo, x_hi = np.percentile(x, [0.5, 99.5])
    y_lo, y_hi = np.percentile(y, [0.5, 99.5])
    ax.set_xlim(x_lo - (x_hi-x_lo)*0.05, x_hi + (x_hi-x_lo)*0.05)
    ax.set_ylim(y_lo - (y_hi-y_lo)*0.05, y_hi + (y_hi-y_lo)*0.05)


# =============================================================================
# GÉNÉRATION DES GRAPHIQUES (UN PAR UN)
# =============================================================================

if APPLY_PREGATING:
    
    print("="*70)
    print(" VISUALISATION DU PRE-GATING - GRAPHIQUES SÉPARÉS")
    print("="*70)
    
    # Indices des canaux
    fsc_a_idx = PreGating.find_marker_index(var_names, ['FSC-A'])
    fsc_h_idx = PreGating.find_marker_index(var_names, ['FSC-H'])
    ssc_a_idx = PreGating.find_marker_index(var_names, ['SSC-A', 'SSC-H', 'SSC'])
    cd45_idx = PreGating.find_marker_index(var_names, ['CD45', 'CD45-PECY5', 'CD45-PC5'])
    cd34_idx = PreGating.find_marker_index(var_names, ['CD34', 'CD34-PE', 'CD34-APC'])
    
    # Sous-échantillonner
    n_sample = min(60000, n_before)
    np.random.seed(42)
    idx_s = np.random.choice(n_before, n_sample, replace=False)
    
    # =========================================================================
    # GRAPHIQUE 1 : VUE D'ENSEMBLE (FSC-A vs SSC-A)
    # =========================================================================
    print("\n" + "─"*50)
    print(" GRAPHIQUE 1 : VUE D'ENSEMBLE")
    print("─"*50)
    
    if fsc_a_idx is not None and ssc_a_idx is not None:
        fig1, ax1 = plt.subplots(figsize=(10, 8), facecolor='#1e1e2e')
        
        plot_density(ax1, 
                     X_raw[idx_s, fsc_a_idx], 
                     X_raw[idx_s, ssc_a_idx],
                     f'VUE D\'ENSEMBLE\n{n_before:,} événements totaux',
                     'FSC-A (Forward Scatter - Taille)',
                     'SSC-A (Side Scatter - Granularité)')
        
        plt.tight_layout()
        plt.savefig('gating_01_overview.png', dpi=150, facecolor='#1e1e2e', bbox_inches='tight')
        plt.show()
        print("   [OK] Sauvegardé: gating_01_overview.png")
    else:
        print("   [!] FSC-A ou SSC-A non trouvé dans les données")
    
    # =========================================================================
    # GRAPHIQUE 2 : GATE DÉBRIS (FSC-A vs SSC-A avec overlay)
    # =========================================================================
    if GATE_DEBRIS:
        print("\n" + "─"*50)
        print(" GRAPHIQUE 2 : GATE DÉBRIS")
        print("─"*50)
        
        if fsc_a_idx is not None and ssc_a_idx is not None:
            fig2, ax2 = plt.subplots(figsize=(10, 8), facecolor='#1e1e2e')
            
            plot_gating(ax2,
                        X_raw[idx_s, fsc_a_idx],
                        X_raw[idx_s, ssc_a_idx],
                        mask_debris[idx_s],
                        'GATE 1 : Exclusion des Débris',
                        'FSC-A (Taille)',
                        'SSC-A (Granularité)',
                        'Cellules viables', 'Débris/Bruit')
            
            # Rectangle de gate
            fsc_lo = np.nanpercentile(X_raw[:, fsc_a_idx], DEBRIS_MIN_PERCENTILE)
            fsc_hi = np.nanpercentile(X_raw[:, fsc_a_idx], DEBRIS_MAX_PERCENTILE)
            ssc_lo = np.nanpercentile(X_raw[:, ssc_a_idx], DEBRIS_MIN_PERCENTILE)
            ssc_hi = np.nanpercentile(X_raw[:, ssc_a_idx], DEBRIS_MAX_PERCENTILE)
            
            rect = Rectangle((fsc_lo, ssc_lo), fsc_hi-fsc_lo, ssc_hi-ssc_lo,
                             fill=False, edgecolor='#f9e2af', linewidth=3, linestyle='--')
            ax2.add_patch(rect)
            ax2.text(fsc_lo + (fsc_hi-fsc_lo)/2, ssc_hi, ' Zone de sélection',
                    ha='center', va='bottom', fontsize=11, color='#f9e2af', fontweight='bold')
            
            # Stats
            n_kept = mask_debris.sum()
            print(f"   → Événements conservés: {n_kept:,} ({n_kept/n_before*100:.1f}%)")
            print(f"   → Débris exclus: {n_before - n_kept:,}")
            
            plt.tight_layout()
            plt.savefig('gating_02_debris.png', dpi=150, facecolor='#1e1e2e', bbox_inches='tight')
            plt.show()
            print("   [OK] Sauvegardé: gating_02_debris.png")
    
    # =========================================================================
    # GRAPHIQUE 3 : GATE SINGLETS (FSC-H vs FSC-A)
    # =========================================================================
    if GATE_DOUBLETS:
        print("\n" + "─"*50)
        print(" GRAPHIQUE 3 : GATE SINGLETS (Doublets)")
        print("─"*50)
        
        if fsc_a_idx is not None and fsc_h_idx is not None:
            fig3, ax3 = plt.subplots(figsize=(10, 8), facecolor='#1e1e2e')
            
            # Après gate 1
            m_g1 = mask_debris[idx_s]
            x3 = X_raw[idx_s, fsc_a_idx][m_g1]
            y3 = X_raw[idx_s, fsc_h_idx][m_g1]
            m3 = mask_singlets[idx_s][m_g1]
            
            if len(x3) > 100:
                plot_gating(ax3, x3, y3, m3,
                            'GATE 2 : Exclusion des Doublets',
                            'FSC-A (Area)',
                            'FSC-H (Height)',
                            'Singlets', 'Doublets/Agrégats')
                
                # Lignes de ratio
                x_range = np.linspace(np.nanpercentile(x3, 1), np.nanpercentile(x3, 99), 100)
                ax3.plot(x_range, x_range, 'w-', lw=2, alpha=0.7, label='Ratio 1:1')
                ax3.plot(x_range, x_range * RATIO_MIN, '--', color='#f9e2af', lw=2)
                ax3.plot(x_range, x_range * RATIO_MAX, '--', color='#f9e2af', lw=2)
                ax3.fill_between(x_range, x_range * RATIO_MIN, x_range * RATIO_MAX,
                                alpha=0.1, color='#f9e2af')
                
                # Stats
                n_after_g2 = (mask_debris & mask_singlets).sum()
                n_doublets = mask_debris.sum() - n_after_g2
                print(f"   → Singlets conservés: {n_after_g2:,}")
                print(f"   → Doublets exclus: {n_doublets:,}")
                
                plt.tight_layout()
                plt.savefig('gating_03_singlets.png', dpi=150, facecolor='#1e1e2e', bbox_inches='tight')
                plt.show()
                print("   [OK] Sauvegardé: gating_03_singlets.png")
    
    # =========================================================================
    # GRAPHIQUE 4 : GATE CD45+ (CD45 vs SSC-A) — GATE PRINCIPAL
    # =========================================================================
    if GATE_CD45:
        print("\n" + "─"*50)
        print(" GRAPHIQUE 4 : GATE CD45+ (Leucocytes) — GATE PRINCIPAL")
        print("─"*50)
        
        if cd45_idx is not None and ssc_a_idx is not None:
            fig4, ax4 = plt.subplots(figsize=(10, 8), facecolor='#1e1e2e')
            
            # Après gates 1+2
            m_g12 = (mask_debris & mask_singlets)[idx_s]
            x4 = X_raw[idx_s, cd45_idx][m_g12]
            y4 = X_raw[idx_s, ssc_a_idx][m_g12]
            m4 = mask_cd45[idx_s][m_g12]
            
            if len(x4) > 100:
                plot_gating(ax4, x4, y4, m4,
                            'GATE 3 : Sélection des Leucocytes CD45+',
                            'CD45 (Intensité)',
                            'SSC-A (Granularité)',
                            'Leucocytes CD45+', 'Cellules CD45-')
                
                # Seuil CD45
                cd45_th = np.nanpercentile(X_raw[:, cd45_idx], CD45_THRESHOLD_PERCENTILE)
                ax4.axvline(x=cd45_th, color='#89b4fa', lw=3, ls='--')
                ax4.text(cd45_th, ax4.get_ylim()[1], f' Seuil CD45+\n (P{CD45_THRESHOLD_PERCENTILE})',
                        va='top', ha='left', fontsize=10, color='#89b4fa', fontweight='bold')
                
                # Stats
                n_cd45_kept = (mask_debris & mask_singlets & mask_cd45).sum()
                print(f"   → Leucocytes CD45+ conservés: {n_cd45_kept:,}")
                print(f"   → CD45- exclus: {(mask_debris & mask_singlets).sum() - n_cd45_kept:,}")
                
                plt.tight_layout()
                plt.savefig('gating_04_cd45.png', dpi=150, facecolor='#1e1e2e', bbox_inches='tight')
                plt.show()
                print("   [OK] Sauvegardé: gating_04_cd45.png")
        else:
            print("   [!] CD45 non trouvé - Gate CD45+ ignoré")
    
    # =========================================================================
    # GRAPHIQUE 5 : GATE CD34+ (CD34 vs SSC-A) — OPTIONNEL
    # =========================================================================
    if FILTER_BLASTS:
        print("\n" + "─"*50)
        print(" GRAPHIQUE 5 : GATE CD34+ (Blastes) — Sous-population des CD45+")
        print("─"*50)
        
        if cd34_idx is not None and ssc_a_idx is not None:
            fig_cd34, ax_cd34 = plt.subplots(figsize=(10, 8), facecolor='#1e1e2e')
            
            # Après gates 1+2+3 (CD45+)
            m_g123 = (mask_debris & mask_singlets & mask_cd45)[idx_s]
            x5 = X_raw[idx_s, cd34_idx][m_g123]
            y5 = X_raw[idx_s, ssc_a_idx][m_g123]
            m5 = mask_cd34[idx_s][m_g123]
            
            if len(x5) > 100:
                plot_gating(ax_cd34, x5, y5, m5,
                            'GATE 4 : Sélection des Blastes CD34+ (parmi CD45+)',
                            'CD34 (Intensité)',
                            'SSC-A (Granularité)',
                            'Blastes CD34+', 'Autres leucocytes')
                
                # Seuils
                cd34_th = np.nanpercentile(X_raw[:, cd34_idx], CD34_THRESHOLD_PERCENTILE)
                ax_cd34.axvline(x=cd34_th, color='#f9e2af', lw=3, ls='--')
                ax_cd34.text(cd34_th, ax_cd34.get_ylim()[1], f' Seuil CD34\n (P{CD34_THRESHOLD_PERCENTILE})',
                        va='top', ha='left', fontsize=10, color='#f9e2af', fontweight='bold')
                
                if USE_SSC_FILTER_FOR_BLASTS:
                    ssc_th = np.nanpercentile(X_raw[:, ssc_a_idx], SSC_MAX_PERCENTILE_BLASTS)
                    ax_cd34.axhline(y=ssc_th, color='#fab387', lw=3, ls='--')
                    ax_cd34.text(ax_cd34.get_xlim()[1], ssc_th, f' SSC max (P{SSC_MAX_PERCENTILE_BLASTS}) ',
                            va='bottom', ha='right', fontsize=10, color='#fab387', fontweight='bold')
                
                # Stats
                print(f"   → Blastes CD34+ sélectionnés: {n_final:,}")
                print(f"   → Autres leucocytes exclus: {(mask_debris & mask_singlets & mask_cd45).sum() - n_final:,}")
                
                plt.tight_layout()
                plt.savefig('gating_05_cd34.png', dpi=150, facecolor='#1e1e2e', bbox_inches='tight')
                plt.show()
                print("   [OK] Sauvegardé: gating_05_cd34.png")
        else:
            print("   [!] CD34 non trouvé - Gate CD34+ ignoré")
    
    # =========================================================================
    # GRAPHIQUE 6 : COMPARAISON AVANT / APRÈS
    # =========================================================================
    print("\n" + "─"*50)
    print(" GRAPHIQUE 6 : COMPARAISON AVANT / APRÈS")
    print("─"*50)
    
    if fsc_a_idx is not None and ssc_a_idx is not None:
        fig5, (ax5a, ax5b) = plt.subplots(1, 2, figsize=(16, 7), facecolor='#1e1e2e')
        
        # AVANT
        cmap_red = LinearSegmentedColormap.from_list('reds', 
            ['#1a1a2e', '#7f1d1d', '#dc2626', '#fca5a5', '#ffffff'])
        
        valid = np.isfinite(X_raw[idx_s, fsc_a_idx]) & np.isfinite(X_raw[idx_s, ssc_a_idx])
        x_bef = X_raw[idx_s, fsc_a_idx][valid]
        y_bef = X_raw[idx_s, ssc_a_idx][valid]
        
        x_lo, x_hi = np.percentile(x_bef, [0.5, 99.5])
        y_lo, y_hi = np.percentile(y_bef, [0.5, 99.5])
        
        h1 = ax5a.hist2d(x_bef, y_bef, bins=100, 
                         range=[[x_lo, x_hi], [y_lo, y_hi]],
                         cmap=cmap_red, norm=plt.matplotlib.colors.LogNorm(vmin=1))
        ax5a.set_title(f'AVANT Gating\n{n_before:,} événements', fontsize=14, fontweight='bold', color='white')
        ax5a.set_xlabel('FSC-A', fontsize=12, fontweight='bold', color='white')
        ax5a.set_ylabel('SSC-A', fontsize=12, fontweight='bold', color='white')
        ax5a.set_facecolor('#1e1e2e')
        ax5a.tick_params(colors='white')
        ax5a.xaxis.set_major_formatter(FuncFormatter(format_axis))
        ax5a.yaxis.set_major_formatter(FuncFormatter(format_axis))
        for spine in ax5a.spines.values():
            spine.set_color('#45475a')
        
        # APRÈS
        cmap_green = LinearSegmentedColormap.from_list('greens', 
            ['#1a1a2e', '#14532d', '#22c55e', '#86efac', '#ffffff'])
        
        m_final = mask_final[idx_s]
        x_aft = X_raw[idx_s, fsc_a_idx][m_final]
        y_aft = X_raw[idx_s, ssc_a_idx][m_final]
        
        if len(x_aft) > 100:
            h2 = ax5b.hist2d(x_aft, y_aft, bins=100,
                             range=[[x_lo, x_hi], [y_lo, y_hi]],
                             cmap=cmap_green, norm=plt.matplotlib.colors.LogNorm(vmin=1))
        
        pct_final = n_final / n_before * 100
        population_label = "Blastes CD34+" if FILTER_BLASTS else "Leucocytes CD45+"
        ax5b.set_title(f'APRÈS Gating ({population_label})\n{n_final:,} événements ({pct_final:.1f}%)', 
                      fontsize=14, fontweight='bold', color='white')
        ax5b.set_xlabel('FSC-A', fontsize=12, fontweight='bold', color='white')
        ax5b.set_ylabel('SSC-A', fontsize=12, fontweight='bold', color='white')
        ax5b.set_facecolor('#1e1e2e')
        ax5b.tick_params(colors='white')
        ax5b.xaxis.set_major_formatter(FuncFormatter(format_axis))
        ax5b.yaxis.set_major_formatter(FuncFormatter(format_axis))
        for spine in ax5b.spines.values():
            spine.set_color('#45475a')
        
        plt.tight_layout()
        plt.savefig('gating_06_comparison.png', dpi=150, facecolor='#1e1e2e', bbox_inches='tight')
        plt.show()
        print("   [OK] Sauvegardé: gating_06_comparison.png")
    
    # =========================================================================
    # RÉSUMÉ FINAL
    # =========================================================================
    print("\n" + "="*70)
    print(" RÉSUMÉ DU PRE-GATING")
    print("="*70)
    
    ret_g1 = mask_debris.sum() / n_before * 100
    ret_g2 = (mask_debris & mask_singlets).sum() / n_before * 100
    ret_g3 = (mask_debris & mask_singlets & mask_cd45).sum() / n_before * 100
    ret_final = n_final / n_before * 100
    
    print(f"""
    ┌─────────────────────────────────────────────────────────────┐
    │                    STATISTIQUES DE RÉTENTION                │
    ├─────────────────────────────────────────────────────────────┤
    │  Étape                        Cellules        Rétention     │
    ├─────────────────────────────────────────────────────────────┤
    │   Initial                   {n_before:>10,}        100.0%     │
    │  - Gate 1 (Débris)           {mask_debris.sum():>10,}        {ret_g1:>5.1f}%     │
    │  - Gate 2 (Doublets)         {(mask_debris & mask_singlets).sum():>10,}        {ret_g2:>5.1f}%     │
    │  - Gate 3 (CD45+)            {(mask_debris & mask_singlets & mask_cd45).sum():>10,}        {ret_g3:>5.1f}%     │""")
    if FILTER_BLASTS:
        print(f"    │  - Gate 4 (CD34+)            {n_final:>10,}        {ret_final:>5.1f}%     │")
    print(f"""    ├─────────────────────────────────────────────────────────────┤
    │  [OK] CELLULES CONSERVÉES       {n_final:>10,}        {ret_final:>5.1f}%     │
    │  [X] CELLULES EXCLUES          {n_before - n_final:>10,}        {100-ret_final:>5.1f}%     │
    └─────────────────────────────────────────────────────────────┘
    """)
    
    # --- Détail par condition si gating asymétrique ---
    if MODE_BLASTES_VS_NORMAL:
        _mask_patho = (conditions == "Pathologique")
        _mask_sain = (conditions == "Sain")
        _n_patho_tot = _mask_patho.sum()
        _n_sain_tot = _mask_sain.sum()
        _n_patho_fin = (mask_final & _mask_patho).sum()
        _n_sain_fin = (mask_final & _mask_sain).sum()
        _ret_patho = _n_patho_fin / max(_n_patho_tot, 1) * 100
        _ret_sain = _n_sain_fin / max(_n_sain_tot, 1) * 100
        print(f"""
    ┌─────────────────────────────────────────────────────────────┐
    │          DÉTAIL GATING ASYMÉTRIQUE PAR CONDITION            │
    ├─────────────────────────────────────────────────────────────┤
    │  PATHOLOGIQUE (CD45 strict appliqué):                       │
    │    Initial:          {_n_patho_tot:>10,}                              │
    │    Conservé:         {_n_patho_fin:>10,}   ({_ret_patho:>5.1f}%)                   │
    │    Gates: Débris + Doublets + CD45+ strict                  │
    ├─────────────────────────────────────────────────────────────┤
    │  SAIN / NBM (PAS de gate CD45):                             │
    │    Initial:          {_n_sain_tot:>10,}                              │
    │    Conservé:         {_n_sain_fin:>10,}   ({_ret_sain:>5.1f}%)                   │
    │    Gates: Débris + Doublets UNIQUEMENT                      │
    └─────────────────────────────────────────────────────────────┘
        """)
    
    population_desc = "Blastes CD34+ (parmi CD45+)" if FILTER_BLASTS else ("Leucocytes CD45+ (patho) + Moelle normale complète (sain)" if MODE_BLASTES_VS_NORMAL else "Leucocytes CD45+ (population complète)")
    print(f" Population finale: {population_desc}")
    
    print("\n Fichiers générés:")
    print("   - gating_01_overview.png    - Vue d'ensemble")
    print("   - gating_02_debris.png      - Gate débris")
    print("   - gating_03_singlets.png    - Gate singlets")
    print("   - gating_04_cd45.png        - Gate CD45+ (leucocytes)")
    if FILTER_BLASTS:
        print("   - gating_05_cd34.png        - Gate CD34+ (blastes)")
    print("   - gating_06_comparison.png  - Avant/Après")

else:
    print("\n Pre-gating désactivé - Aucun graphique généré")
    print("   → Activez APPLY_PREGATING = True pour visualiser")

## Visualisation Interactive du Pre-Gating — Style CytoPy

Dashboard interactif Plotly pour inspecter chaque étape du gating séquentiel :

1. **Sankey Diagram** : Flux des événements à travers les 4 gates (rétention vs exclusion)
2. **Density Plots Interactifs** : Scatter 2D avec contours de densité pour chaque gate
3. **Histogrammes 1D** : Distribution des marqueurs clés avec seuils GMM annotés
4. **Comparaison Patho / Sain** (si `MODE_BLASTES_VS_NORMAL`)

> *Inspiré de CytoPy AutonomousGate — tous les graphiques sont interactifs (zoom, hover, export)*

In [ ]:
# =============================================================================
# VISUALISATION INTERACTIVE STYLE CYTOPY — DASHBOARD DE GATING
# =============================================================================
# Utilise Plotly pour des graphiques interactifs (zoom, hover, export)
# Inspiré de CytoPy AutonomousGate et FlowJo hierarchical gating
# =============================================================================

if APPLY_PREGATING and PLOTLY_AVAILABLE:

    print("=" * 70)
    print(" CYTOPY-STYLE GATING DASHBOARD — VISUALISATION INTERACTIVE")
    print("=" * 70)

    # =====================================================================
    # 0. PRÉPARATION DES DONNÉES
    # =====================================================================
    _fsc_a_idx = PreGating.find_marker_index(var_names, ['FSC-A'])
    _fsc_h_idx = PreGating.find_marker_index(var_names, ['FSC-H'])
    _ssc_a_idx = PreGating.find_marker_index(var_names, ['SSC-A', 'SSC-H', 'SSC'])
    _cd45_idx  = PreGating.find_marker_index(var_names, ['CD45', 'CD45-PECY5', 'CD45-PC5'])
    _cd34_idx  = PreGating.find_marker_index(var_names, ['CD34', 'CD34-PE', 'CD34-APC'])

    # Sous-échantillonner pour fluidité Plotly
    _n_pts = min(40_000, n_before)
    np.random.seed(42)
    _idx = np.random.choice(n_before, _n_pts, replace=False)

    # Masques cumulatifs appliqués séquentiellement
    _m_g1     = mask_debris[_idx]
    _m_g12    = (mask_debris & mask_singlets)[_idx]
    _m_g123   = (mask_debris & mask_singlets & mask_cd45)[_idx]
    _m_final  = mask_final[_idx]

    # Vecteur de conditions pour le sous-échantillon
    _cond_sub = conditions[_idx]
    _is_sain_sub = (_cond_sub == "Sain")

    # Labels de gate pour chaque événement (gating asymétrique)
    # ─── LÉGENDE DÉTAILLÉE : sépare Patho conservés / Sain NBM conservés ───
    def _gate_label(i):
        if not _m_g1[i]:
            return "Débris (exclu G1)"
        if not _m_g12[i]:
            return "Doublet (exclu G2)"
        if not _m_g123[i]:
            # En mode asymétrique, seuls les Patho peuvent être exclus par CD45
            return "CD45- Patho (exclu G3)"
        if FILTER_BLASTS and not _m_final[i]:
            return "Non-blaste (exclu G4)"
        # Cellule conservée → distinguer Patho CD45+ vs Sain NBM
        if _cond_sub[i] == "Pathologique":
            return "CD45+ Patho conservés ✓"
        elif _cond_sub[i] == "Sain":
            return "Conservés sains NBM ✓"
        return "Conservé ✓"

    _labels = np.array([_gate_label(i) for i in range(_n_pts)])

    # Palette CytoPy-style (avec catégories détaillées pour conservés)
    _color_map = {
        "Débris (exclu G1)":        "#636363",
        "Doublet (exclu G2)":       "#e6550d",
        "CD45- Patho (exclu G3)":   "#fd8d3c",
        "Non-blaste (exclu G4)":    "#fdae6b",
        "CD45+ Patho conservés ✓":  "#d62728",   # rouge – patho conservés
        "Conservés sains NBM ✓":    "#2ca02c",   # vert  – sains NBM conservés
        "Conservé ✓":               "#31a354",   # fallback
    }

    # =====================================================================
    # 1. SANKEY DIAGRAM — FLUX DES ÉVÉNEMENTS
    # =====================================================================
    print("\n [1/5] Sankey Diagram — Flux du gating hiérarchique")

    _n_total   = n_before
    _n_g1_pass = int(mask_debris.sum())
    _n_g1_fail = _n_total - _n_g1_pass
    _n_g2_pass = int((mask_debris & mask_singlets).sum())
    _n_g2_fail = _n_g1_pass - _n_g2_pass
    _n_g3_pass = int((mask_debris & mask_singlets & mask_cd45).sum())
    _n_g3_fail = _n_g2_pass - _n_g3_pass
    _n_g4_pass = int(n_final)
    _n_g4_fail = _n_g3_pass - _n_g4_pass

    _sankey_labels = [
        f"Événements<br>totaux<br>{_n_total:,}",              # 0
        f"Gate 1<br>Débris<br>{_n_g1_pass:,}",                # 1
        f"Débris<br>exclus<br>{_n_g1_fail:,}",                # 2
        f"Gate 2<br>Singlets<br>{_n_g2_pass:,}",              # 3
        f"Doublets<br>exclus<br>{_n_g2_fail:,}",              # 4
        f"Gate 3<br>CD45+ (Patho)<br>{_n_g3_pass:,}",         # 5
        f"CD45- Patho<br>exclus<br>{_n_g3_fail:,}",           # 6
    ]

    _src  = [0, 0, 1, 1, 3, 3]
    _tgt  = [1, 2, 3, 4, 5, 6]
    _vals = [_n_g1_pass, _n_g1_fail, _n_g2_pass, _n_g2_fail, _n_g3_pass, _n_g3_fail]
    _link_colors = [
        "rgba(49,163,84,0.4)", "rgba(99,99,99,0.3)",
        "rgba(49,163,84,0.4)", "rgba(230,85,13,0.3)",
        "rgba(49,163,84,0.4)", "rgba(253,141,60,0.3)",
    ]

    if FILTER_BLASTS:
        _sankey_labels.append(f"Gate 4<br>CD34+<br>{_n_g4_pass:,}")   # 7
        _sankey_labels.append(f"Non-blastes<br>exclus<br>{_n_g4_fail:,}")  # 8
        _src  += [5, 5]
        _tgt  += [7, 8]
        _vals += [_n_g4_pass, _n_g4_fail]
        _link_colors += ["rgba(49,163,84,0.4)", "rgba(253,174,107,0.3)"]
        _final_label = f"Population<br>finale<br>{_n_g4_pass:,}"
        _sankey_labels.append(_final_label)  # 9
        _src.append(7)
        _tgt.append(9)
        _vals.append(_n_g4_pass)
        _link_colors.append("rgba(49,163,84,0.6)")
    else:
        _final_label = f"Population<br>finale<br>{_n_g3_pass:,}"
        _sankey_labels.append(_final_label)  # 7
        _src.append(5)
        _tgt.append(7)
        _vals.append(_n_g3_pass)
        _link_colors.append("rgba(49,163,84,0.6)")

    _node_colors = ["#4a90d9"] + ["#31a354", "#636363"] * 1 + \
                   ["#31a354", "#e6550d", "#31a354", "#fd8d3c"]
    if FILTER_BLASTS:
        _node_colors += ["#31a354", "#fdae6b", "#2ca02c"]
    else:
        _node_colors += ["#2ca02c"]

    fig_sankey = go.Figure(go.Sankey(
        arrangement="snap",
        node=dict(
            pad=20,
            thickness=25,
            line=dict(color="#333", width=1),
            label=_sankey_labels,
            color=_node_colors[:len(_sankey_labels)],
        ),
        link=dict(
            source=_src,
            target=_tgt,
            value=_vals,
            color=_link_colors,
        ),
    ))
    fig_sankey.update_layout(
        title=dict(
            text="<b>Gating Hierarchy — Flux des Événements</b>",
            font=dict(size=18),
        ),
        font=dict(size=13, color="#222"),
        paper_bgcolor="#fafafa",
        height=450,
        margin=dict(l=20, r=20, t=60, b=20),
    )
    fig_sankey.show()

    # =====================================================================
    # 2. DENSITY SCATTER PLOTS — CHAQUE GATE
    # =====================================================================
    print(" [2/5] Density Scatter Plots — Gating séquentiel")

    _gate_plots = []

    # --- Gate 1 : FSC-A vs SSC-A (Débris) ---
    if GATE_DEBRIS and _fsc_a_idx is not None and _ssc_a_idx is not None:
        _gate_plots.append({
            "title": "Gate 1 — Débris (SSC-A vs FSC-A)",
            "x": X_raw[_idx, _fsc_a_idx],
            "y": X_raw[_idx, _ssc_a_idx],
            "mask": _m_g1,
            "xlabel": "FSC-A (Taille)",
            "ylabel": "SSC-A (Granularité)",
            "label_in": "Cellules viables",
            "label_out": "Débris",
        })

    # --- Gate 2 : FSC-H vs FSC-A (Doublets) — sur les survivants de G1 ---
    if GATE_DOUBLETS and _fsc_a_idx is not None and _fsc_h_idx is not None:
        _g1_ok = _m_g1
        _gate_plots.append({
            "title": "Gate 2 — Doublets (FSC-H vs FSC-A) [après G1]",
            "x": X_raw[_idx, _fsc_a_idx][_g1_ok],
            "y": X_raw[_idx, _fsc_h_idx][_g1_ok],
            "mask": mask_singlets[_idx][_g1_ok],
            "xlabel": "FSC-A (Area)",
            "ylabel": "FSC-H (Height)",
            "label_in": "Singlets",
            "label_out": "Doublets",
        })

    # --- Gate 3 : CD45 vs SSC-A (Leucocytes) — sur les survivants de G1+G2 ---
    # En mode asymétrique: affiche UNIQUEMENT les cellules Patho (les Sain ne sont pas gatées CD45)
    if GATE_CD45 and _cd45_idx is not None and _ssc_a_idx is not None:
        _g12_ok = _m_g12
        if MODE_BLASTES_VS_NORMAL:
            # Filtre: montrer seulement les cellules Patho après G1+G2
            _patho_g12 = _g12_ok & (_cond_sub == "Pathologique")
            _gate_plots.append({
                "title": "Gate 3 — CD45+ PATHO seul (Sain: pas de gate CD45)",
                "x": X_raw[_idx, _cd45_idx][_patho_g12],
                "y": X_raw[_idx, _ssc_a_idx][_patho_g12],
                "mask": mask_cd45[_idx][_patho_g12],
                "xlabel": "CD45 (Intensité)",
                "ylabel": "SSC-A (Granularité)",
                "label_in": "CD45+ Patho (conservés)",
                "label_out": "CD45− Patho (exclus)",
            })
        else:
            _gate_plots.append({
                "title": "Gate 3 — CD45+ Leucocytes (CD45 vs SSC-A) [après G1+G2]",
                "x": X_raw[_idx, _cd45_idx][_g12_ok],
                "y": X_raw[_idx, _ssc_a_idx][_g12_ok],
                "mask": mask_cd45[_idx][_g12_ok],
                "xlabel": "CD45 (Intensité)",
                "ylabel": "SSC-A (Granularité)",
                "label_in": "Leucocytes CD45+",
                "label_out": "CD45−",
            })

    # --- Gate 4 : CD34 vs SSC-A (Blastes) — si activé ---
    if FILTER_BLASTS and _cd34_idx is not None and _ssc_a_idx is not None:
        _g123_ok = _m_g123
        _gate_plots.append({
            "title": "Gate 4 — CD34+ Blastes (CD34 vs SSC-A) [après G1+G2+G3]",
            "x": X_raw[_idx, _cd34_idx][_g123_ok],
            "y": X_raw[_idx, _ssc_a_idx][_g123_ok],
            "mask": mask_cd34[_idx][_g123_ok],
            "xlabel": "CD34 (Intensité)",
            "ylabel": "SSC-A (Granularité)",
            "label_in": "Blastes CD34+",
            "label_out": "Autres leucocytes",
        })

    # Générer chaque subplot avec Plotly
    n_gates = len(_gate_plots)
    if n_gates > 0:
        fig_gates = make_subplots(
            rows=1, cols=n_gates,
            subplot_titles=[g["title"] for g in _gate_plots],
            horizontal_spacing=0.06,
        )

        for col_i, gp in enumerate(_gate_plots, 1):
            _x, _y, _mk = gp["x"], gp["y"], gp["mask"]
            _valid = np.isfinite(_x) & np.isfinite(_y)
            _x, _y, _mk = _x[_valid], _y[_valid], _mk[_valid]

            # Exclus (fond, semi-transparent)
            fig_gates.add_trace(go.Scattergl(
                x=_x[~_mk], y=_y[~_mk],
                mode="markers",
                marker=dict(size=2, color="#d62728", opacity=0.25),
                name=gp["label_out"],
                legendgroup=f"g{col_i}_out",
                showlegend=(col_i == 1),
                hovertemplate=f"{gp['xlabel']}: %{{x:.0f}}<br>{gp['ylabel']}: %{{y:.0f}}<br>{gp['label_out']}<extra></extra>",
            ), row=1, col=col_i)

            # Conservés (avant-plan)
            fig_gates.add_trace(go.Scattergl(
                x=_x[_mk], y=_y[_mk],
                mode="markers",
                marker=dict(size=2, color="#2ca02c", opacity=0.4),
                name=gp["label_in"],
                legendgroup=f"g{col_i}_in",
                showlegend=(col_i == 1),
                hovertemplate=f"{gp['xlabel']}: %{{x:.0f}}<br>{gp['ylabel']}: %{{y:.0f}}<br>{gp['label_in']}<extra></extra>",
            ), row=1, col=col_i)

            # Axes labels
            fig_gates.update_xaxes(title_text=gp["xlabel"], row=1, col=col_i)
            fig_gates.update_yaxes(title_text=gp["ylabel"], row=1, col=col_i)

        fig_gates.update_layout(
            title="<b>Gating Séquentiel — Density Scatter (Plotly interactif)</b>",
            height=500,
            width=min(500 * n_gates, 2000),
            paper_bgcolor="#fafafa",
            plot_bgcolor="#f0f0f0",
            font=dict(size=11),
            legend=dict(
                orientation="h", yanchor="bottom", y=-0.22,
                xanchor="center", x=0.5,
                font=dict(size=12),
            ),
            margin=dict(t=80, b=100),
        )
        fig_gates.show()

    # =====================================================================
    # 3. HISTOGRAMMES 1D — DISTRIBUTIONS DES MARQUEURS CLÉS
    # =====================================================================
    print(" [3/5] Histogrammes 1D — Distributions avec seuils GMM")

    _hist_data = []
    if _fsc_a_idx is not None:
        _hist_data.append(("FSC-A", X_raw[:, _fsc_a_idx], None))
    if _cd45_idx is not None:
        _hist_data.append(("CD45", X_raw[:, _cd45_idx], "cd45"))
    if FILTER_BLASTS and _cd34_idx is not None:
        _hist_data.append(("CD34", X_raw[:, _cd34_idx], "cd34"))

    if _hist_data:
        fig_hist = make_subplots(
            rows=1, cols=len(_hist_data),
            subplot_titles=[h[0] for h in _hist_data],
            horizontal_spacing=0.08,
        )

        for hi, (name, vals, marker_type) in enumerate(_hist_data, 1):
            _v = vals[np.isfinite(vals)]

            # Avant gating (toutes cellules)
            fig_hist.add_trace(go.Histogram(
                x=_v, nbinsx=200, name=f"{name} (tous)",
                marker_color="rgba(100,100,100,0.4)",
                showlegend=(hi == 1),
                legendgroup="all",
            ), row=1, col=hi)

            # Après gating (conservés)
            _v_kept = vals[mask_final & np.isfinite(vals)]
            fig_hist.add_trace(go.Histogram(
                x=_v_kept, nbinsx=200, name=f"{name} (conservés)",
                marker_color="rgba(44,160,44,0.6)",
                showlegend=(hi == 1),
                legendgroup="kept",
            ), row=1, col=hi)

            # Annoter le seuil GMM si pertinent
            if marker_type == "cd45" and GATE_CD45:
                _cd45_vals = X_raw[:, _cd45_idx]
                _cd45_clean = _cd45_vals[np.isfinite(_cd45_vals)]
                _th = np.percentile(_cd45_clean, CD45_THRESHOLD_PERCENTILE)
                fig_hist.add_vline(
                    x=_th, line_dash="dash", line_color="#d62728", line_width=2,
                    annotation_text=f"Seuil CD45+", annotation_position="top right",
                    row=1, col=hi,
                )
            elif marker_type == "cd34" and FILTER_BLASTS:
                _cd34_vals = X_raw[:, _cd34_idx]
                _cd34_clean = _cd34_vals[np.isfinite(_cd34_vals)]
                _th34 = np.percentile(_cd34_clean, CD34_THRESHOLD_PERCENTILE)
                fig_hist.add_vline(
                    x=_th34, line_dash="dash", line_color="#ff7f0e", line_width=2,
                    annotation_text=f"Seuil CD34+", annotation_position="top right",
                    row=1, col=hi,
                )

            fig_hist.update_xaxes(title_text=name, row=1, col=hi)
            fig_hist.update_yaxes(title_text="Nombre d'événements", row=1, col=hi)

        fig_hist.update_layout(
            title="<b>Distributions 1D — Avant / Après Gating (seuils annotés)</b>",
            barmode="overlay",
            height=400,
            width=min(550 * len(_hist_data), 1800),
            paper_bgcolor="#fafafa",
            plot_bgcolor="#f5f5f5",
            font=dict(size=11),
            legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.5),
            margin=dict(t=70, b=90),
        )
        fig_hist.show()

    # =====================================================================
    # 4. COMPARAISON PATHO vs SAIN (si MODE_BLASTES_VS_NORMAL)
    # =====================================================================
    if MODE_BLASTES_VS_NORMAL and _cd45_idx is not None and _ssc_a_idx is not None:
        print(" [4/5] Comparaison Patho vs Sain — CD45 vs SSC-A (GATING ASYMÉTRIQUE)")

        _cond = conditions[_idx]

        fig_comp = make_subplots(
            rows=1, cols=2,
            subplot_titles=[
                "Pathologique (CD45 strict appliqué)",
                "Sain / NBM (PAS de gate CD45)"
            ],
            horizontal_spacing=0.08,
        )

        for ci, (cond_label, color_kept, color_all) in enumerate([
            ("Pathologique", "#d62728", "#ffcccc"),
            ("Sain", "#2ca02c", "#ccffcc"),
        ], 1):
            _sel = (_cond == cond_label)
            _sel_final = _sel & _m_final

            _xc = X_raw[_idx, _cd45_idx]
            _yc = X_raw[_idx, _ssc_a_idx]

            # Toutes les cellules de cette condition (fond)
            fig_comp.add_trace(go.Scattergl(
                x=_xc[_sel], y=_yc[_sel],
                mode="markers",
                marker=dict(size=2, color=color_all, opacity=0.15),
                name=f"{cond_label} (tous)",
                showlegend=True,
            ), row=1, col=ci)

            # Cellules conservées
            fig_comp.add_trace(go.Scattergl(
                x=_xc[_sel_final], y=_yc[_sel_final],
                mode="markers",
                marker=dict(size=2.5, color=color_kept, opacity=0.5),
                name=f"{cond_label} (conservés)",
                showlegend=True,
            ), row=1, col=ci)

            _n_all = _sel.sum()
            _n_kept = _sel_final.sum()
            _gate_info = "CD45 strict" if cond_label == "Pathologique" else "Pas de gate CD45"
            fig_comp.update_xaxes(title_text="CD45", row=1, col=ci)
            fig_comp.update_yaxes(title_text="SSC-A", row=1, col=ci)

            _xax = "x domain" if ci == 1 else f"x{ci} domain"
            _yax = "y domain" if ci == 1 else f"y{ci} domain"
            fig_comp.add_annotation(
                text=f"<b>{_n_kept:,} / {_n_all:,} ({_n_kept/_n_all*100:.1f}%)<br>{_gate_info}</b>" if _n_all > 0 else "N/A",
                xref=_xax, yref=_yax,
                x=0.5, y=0.02, showarrow=False,
                font=dict(size=13, color=color_kept),
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor=color_kept, borderwidth=1, borderpad=4,
            )

        fig_comp.update_layout(
            title="<b>Gating Asymétrique — Patho (CD45 strict) vs Sain (pas de CD45)</b>",
            height=500,
            width=1100,
            paper_bgcolor="#fafafa",
            plot_bgcolor="#f0f0f0",
            font=dict(size=11),
            legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5),
            margin=dict(t=70, b=90),
        )
        fig_comp.show()
    else:
        print(" [4/5] Comparaison Patho vs Sain — SKIP (MODE_BLASTES_VS_NORMAL désactivé)")

    # =====================================================================
    # 5. OVERVIEW FINAL — FSC-A vs SSC-A coloré par gate d'exclusion
    # =====================================================================
    if _fsc_a_idx is not None and _ssc_a_idx is not None:
        print(" [5/5] Overview Final — Coloré par étape d'exclusion")

        _xo = X_raw[_idx, _fsc_a_idx]
        _yo = X_raw[_idx, _ssc_a_idx]

        fig_overview = go.Figure()

        # Tracer chaque catégorie d'exclusion dans l'ordre
        # ─── Inclut désormais les conservés Patho et Sain séparément ───
        _order = [
            "Débris (exclu G1)",
            "Doublet (exclu G2)",
            "CD45- Patho (exclu G3)",
            "Non-blaste (exclu G4)",
            "CD45+ Patho conservés ✓",
            "Conservés sains NBM ✓",
            "Conservé ✓",          # fallback si condition inconnue
        ]
        for cat in _order:
            _sel_cat = (_labels == cat)
            if _sel_cat.sum() == 0:
                continue
            _is_kept = cat.endswith("✓")
            fig_overview.add_trace(go.Scattergl(
                x=_xo[_sel_cat], y=_yo[_sel_cat],
                mode="markers",
                marker=dict(
                    size=2.5,
                    color=_color_map.get(cat, "#999"),
                    opacity=0.55 if _is_kept else 0.25,
                ),
                name=f"{cat} ({_sel_cat.sum():,})",
                hovertemplate=f"FSC-A: %{{x:.0f}}<br>SSC-A: %{{y:.0f}}<br>{cat}<extra></extra>",
            ))

        fig_overview.update_layout(
            title="<b>Overview — Événements colorés par étape d'exclusion</b>",
            xaxis_title="FSC-A (Taille)",
            yaxis_title="SSC-A (Granularité)",
            height=600,
            width=900,
            paper_bgcolor="#fafafa",
            plot_bgcolor="#f0f0f0",
            font=dict(size=12),
            legend=dict(
                title="Catégorie",
                font=dict(size=12),
                bgcolor="rgba(255,255,255,0.9)",
                bordercolor="#ccc", borderwidth=1,
            ),
            margin=dict(t=70, b=50),
        )
        fig_overview.show()

    # =====================================================================
    # RÉSUMÉ TABULAIRE
    # =====================================================================
    print("\n" + "=" * 70)
    print(" RÉSUMÉ GATING — TABLEAU INTERACTIF")
    print("=" * 70)

    _summary_df = pd.DataFrame({
        "Étape": ["Initial", "Gate 1 (Débris)", "Gate 2 (Doublets)",
                  "Gate 3 (CD45+ Patho only)" if MODE_BLASTES_VS_NORMAL else "Gate 3 (CD45+)"] + (["Gate 4 (CD34+)"] if FILTER_BLASTS else []) + ["Population finale"],
        "Événements": [
            n_before,
            int(mask_debris.sum()),
            int((mask_debris & mask_singlets).sum()),
            int((mask_debris & mask_singlets & mask_cd45).sum()),
        ] + ([int(n_final)] if FILTER_BLASTS else []) + [int(n_final)],
        "Rétention (%)": [
            100.0,
            mask_debris.sum() / n_before * 100,
            (mask_debris & mask_singlets).sum() / n_before * 100,
            (mask_debris & mask_singlets & mask_cd45).sum() / n_before * 100,
        ] + ([n_final / n_before * 100] if FILTER_BLASTS else []) + [n_final / n_before * 100],
        "Exclus": [
            0,
            n_before - int(mask_debris.sum()),
            int(mask_debris.sum()) - int((mask_debris & mask_singlets).sum()),
            int((mask_debris & mask_singlets).sum()) - int((mask_debris & mask_singlets & mask_cd45).sum()),
        ] + ([int((mask_debris & mask_singlets & mask_cd45).sum()) - int(n_final)] if FILTER_BLASTS else []) + [n_before - int(n_final)],
    })
    _summary_df["Rétention (%)"] = _summary_df["Rétention (%)"].round(1)

    fig_table = go.Figure(go.Table(
        header=dict(
            values=[f"<b>{c}</b>" for c in _summary_df.columns],
            fill_color="#4a90d9",
            font=dict(color="white", size=13),
            align="center", height=35,
        ),
        cells=dict(
            values=[_summary_df[c] for c in _summary_df.columns],
            fill_color=[
                ["#f9f9f9", "#fff", "#f9f9f9", "#fff"] +
                (["#f9f9f9"] if FILTER_BLASTS else []) + ["#d4edda"]
            ] * 4,
            font=dict(size=12),
            align="center", height=30,
            format=[None, ",", ".1f", ","],
        ),
    ))
    fig_table.update_layout(
        title="<b>Résumé du Pre-Gating — Statistiques par étape</b>",
        height=50 + 35 * (len(_summary_df) + 1),
        width=800,
        margin=dict(l=20, r=20, t=50, b=10),
    )
    fig_table.show()

    # --- Tableau par condition si gating asymétrique ---
    if MODE_BLASTES_VS_NORMAL:
        _mp = (conditions == "Pathologique")
        _ms = (conditions == "Sain")
        _cond_df = pd.DataFrame({
            "Condition": ["Pathologique", "Sain / NBM"],
            "Initial": [int(_mp.sum()), int(_ms.sum())],
            "Après Débris+Doublets": [
                int((_mp & mask_debris & mask_singlets).sum()),
                int((_ms & mask_debris & mask_singlets).sum()),
            ],
            "Après Gate CD45": [
                int((_mp & mask_debris & mask_singlets & mask_cd45).sum()),
                f"{int((_ms & mask_debris & mask_singlets).sum())} (non appliqué)",
            ],
            "Final": [int((mask_final & _mp).sum()), int((mask_final & _ms).sum())],
            "Rétention (%)": [
                round((mask_final & _mp).sum() / max(_mp.sum(), 1) * 100, 1),
                round((mask_final & _ms).sum() / max(_ms.sum(), 1) * 100, 1),
            ],
            "Logique CD45": ["CD45 STRICT", "AUCUN gate CD45"],
        })

        fig_table_cond = go.Figure(go.Table(
            header=dict(
                values=[f"<b>{c}</b>" for c in _cond_df.columns],
                fill_color="#6a0dad",
                font=dict(color="white", size=13),
                align="center", height=35,
            ),
            cells=dict(
                values=[_cond_df[c] for c in _cond_df.columns],
                fill_color=[["#ffe6e6", "#e6ffe6"]] * len(_cond_df.columns),
                font=dict(size=12),
                align="center", height=30,
            ),
        ))
        fig_table_cond.update_layout(
            title="<b>Gating Asymétrique — Détail par Condition (Patho: CD45 strict / Sain: pas de CD45)</b>",
            height=150,
            width=1100,
            margin=dict(l=20, r=20, t=50, b=10),
        )
        fig_table_cond.show()

    print("\n [OK] Dashboard CytoPy-style généré avec succès !")
    print("     → Utilisez la souris pour zoomer, survoler, et exporter (icône 📷)")

elif not PLOTLY_AVAILABLE:
    print("[!] Plotly requis pour le dashboard interactif.")
    print("    → pip install plotly")
else:
    print("[!] Pre-gating désactivé — Aucun dashboard généré.")

In [ ]:
# =============================================================================
# CRÉATION DU SECOND ANNDATA (avec ou sans gating)
# =============================================================================

# Créer l'AnnData filtré (ou copie complète si pas de gating)
combined_gated = combined_data[mask_final].copy()

if APPLY_PREGATING:
    print(f"[OK] AnnData après gating: {combined_gated.shape}")
    print(f"   → {combined_gated.shape[0]:,} cellules conservées")
    print(f"   → {combined_gated.shape[1]} marqueurs")
else:
    print(f"[OK] AnnData créé (sans pre-gating): {combined_gated.shape}")
    print(f"   → {combined_gated.shape[0]:,} cellules (toutes conservées)")
    print(f"   → {combined_gated.shape[1]} marqueurs")

## 6. Transformation des Données (Arcsinh / Logicle)

Les données brutes de cytométrie nécessitent une transformation pour:
- Gérer les valeurs négatives (compensation)
- Compresser la plage dynamique
- Améliorer la visualisation des populations faiblement exprimées

### Transformations disponibles:
- **Arcsinh (cofactor=5)**: Recommandé pour flow cytometry
- **Logicle**: Transformation biexponentielle (standard ISAC)
- **Log10**: Transformation logarithmique simple

In [ ]:
# CONFIGURATION DE LA TRANSFORMATION

# Choix de la transformation
TRANSFORM_TYPE = "none"  # Options: "arcsinh", "logicle", "log10", "none"
COFACTOR = 5  # Pour arcsinh: 5 (flow)

# Appliquer uniquement aux marqueurs de fluorescence (pas FSC/SSC/Time)
APPLY_TO_SCATTER = False

print("TRANSFORMATION DES DONNÉES")
print("="*60)
print(f"   Type: {TRANSFORM_TYPE.upper()}")
if TRANSFORM_TYPE == "arcsinh":
    print(f"   Cofacteur: {COFACTOR}")
print(f"   Appliquer au scatter: {'Oui' if APPLY_TO_SCATTER else 'Non'}")

In [ ]:
# APPLICATION DE LA TRANSFORMATION

# Extraire les données
X_gated = combined_gated.X
if hasattr(X_gated, 'toarray'):
    X_gated = X_gated.toarray()

# Copie pour transformation
X_transformed = X_gated.copy()

# Déterminer les indices des colonnes à transformer
if APPLY_TO_SCATTER:
    cols_to_transform = list(range(len(var_names)))
else:
    # Exclure FSC, SSC, Time
    scatter_patterns = ['FSC', 'SSC', 'TIME', 'EVENT']
    cols_to_transform = [i for i, name in enumerate(var_names) 
                         if not any(p in name.upper() for p in scatter_patterns)]

print(f"\nColonnes à transformer: {len(cols_to_transform)}/{len(var_names)}")

# Appliquer la transformation
if TRANSFORM_TYPE == "arcsinh":
    print(f"\n Application Arcsinh (cofactor={COFACTOR})...")
    X_transformed[:, cols_to_transform] = DataTransformer.arcsinh_transform(
        X_gated[:, cols_to_transform], cofactor=COFACTOR
    )
    
elif TRANSFORM_TYPE == "logicle":
    print("\n Application Logicle...")
    X_transformed[:, cols_to_transform] = DataTransformer.logicle_transform(
        X_gated[:, cols_to_transform]
    )
    
elif TRANSFORM_TYPE == "log10":
    print("\n Application Log10...")
    X_transformed[:, cols_to_transform] = DataTransformer.log_transform(
        X_gated[:, cols_to_transform]
    )
    
else:
    print("\n[!] Pas de transformation appliquée")

# Vérifier les résultats
print(f"\n[OK] Transformation terminée!")
print(f"   Plage avant: [{X_gated[:, cols_to_transform].min():.2f}, {X_gated[:, cols_to_transform].max():.2f}]")
print(f"   Plage après: [{X_transformed[:, cols_to_transform].min():.2f}, {X_transformed[:, cols_to_transform].max():.2f}]")

## 7. Comparaison Avant/Après Transformation

Visualisation côte à côte des distributions pour valider l'effet de la transformation.

In [ ]:
# COMPARAISON DISTRIBUTIONS AVANT/APRÈS

# Sélectionner quelques marqueurs représentatifs
markers_compare = [var_names[i] for i in cols_to_transform[:6]]

fig, axes = plt.subplots(2, len(markers_compare), figsize=(4*len(markers_compare), 8))

for i, marker in enumerate(markers_compare):
    col_idx = var_names.index(marker)
    
    # Avant transformation
    ax = axes[0, i]
    data_before = X_gated[:, col_idx]
    ax.hist(data_before, bins=80, color='#f38ba8', alpha=0.7, edgecolor='none')
    ax.set_title(f'{marker}\n(Brut)', fontsize=10, fontweight='bold')
    ax.axvline(0, color='white', linestyle='--', alpha=0.5)
    if i == 0:
        ax.set_ylabel('AVANT\nCount', fontsize=11, fontweight='bold')
    
    # Après transformation
    ax = axes[1, i]
    data_after = X_transformed[:, col_idx]
    ax.hist(data_after, bins=80, color='#a6e3a1', alpha=0.7, edgecolor='none')
    ax.set_title(f'{TRANSFORM_TYPE.upper()}', fontsize=10)
    ax.axvline(0, color='white', linestyle='--', alpha=0.5)
    if i == 0:
        ax.set_ylabel('APRÈS\nCount', fontsize=11, fontweight='bold')

plt.suptitle(f'Comparaison des Distributions: Brut vs {TRANSFORM_TYPE.upper()} (cofactor={COFACTOR})', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# TEST DE DIFFÉRENTS COFACTEURS (pour tuning)

print("COMPARAISON DES COFACTEURS ARCSINH")
print("="*60)

# Sélectionner un marqueur représentatif
test_marker = markers_compare[0]
test_idx = var_names.index(test_marker)
test_data = X_gated[:, test_idx]

# Tester différents cofacteurs
cofactors = [1, 5, 50, 150, 500]

fig, axes = plt.subplots(1, len(cofactors)+1, figsize=(4*(len(cofactors)+1), 4))

# Données brutes
ax = axes[0]
ax.hist(test_data, bins=80, color='#89b4fa', alpha=0.7, edgecolor='none')
ax.set_title('Brut\n(pas de transfo)', fontsize=10, fontweight='bold')
ax.set_xlabel(test_marker)

# Transformations avec différents cofacteurs
for i, cof in enumerate(cofactors):
    ax = axes[i+1]
    transformed = DataTransformer.arcsinh_transform(test_data, cofactor=cof)
    ax.hist(transformed, bins=80, color='#cba6f7', alpha=0.7, edgecolor='none')
    ax.set_title(f'Arcsinh\ncofactor={cof}', fontsize=10, fontweight='bold')
    ax.set_xlabel(test_marker)

plt.suptitle(f'Impact du Cofacteur sur la Distribution ({test_marker})', 
             fontsize=13, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

## 8. Préparation des Données pour FlowSOM

Sélection des colonnes pour le clustering:
- Exclusion des paramètres scatter (FSC, SSC) et Time
- Conservation uniquement des marqueurs de fluorescence
- Nettoyage final (NaN/Inf)

In [ ]:
# SÉLECTION DES COLONNES POUR FLOWSOM

# Option: exclure FSC/SSC/Time
EXCLUDE_SCATTER = True

# Identifier les colonnes à utiliser
scatter_patterns = ['FSC', 'SSC', 'TIME', 'EVENT']

if EXCLUDE_SCATTER:
    cols_to_use = [i for i, name in enumerate(var_names) 
                   if not any(p in name.upper() for p in scatter_patterns)]
else:
    cols_to_use = list(range(len(var_names)))

used_markers = [var_names[i] for i in cols_to_use]

print("COLONNES POUR FLOWSOM")
print("="*60)
print(f"   Exclure scatter: {'Oui' if EXCLUDE_SCATTER else 'Non'}")
print(f"   Colonnes sélectionnées: {len(cols_to_use)}/{len(var_names)}")
print(f"\nMarqueurs utilisés:")
for i, marker in enumerate(used_markers):
    print(f"   [{i:2d}] {marker}")

In [ ]:
# =============================================================================
# FILTRAGE PAR TYPE D'ÉCHANTILLON (-A ou -H) - OPTIONNEL
# =============================================================================
# Ce filtre permet de sélectionner uniquement les cellules provenant de fichiers
# dont le nom contient "-A" (ex: échantillons de type A) ou "-H" (ex: échantillons de type H)
# 
# [!] IMPORTANT: Si tes fichiers n'ont pas ces suffixes, désactive le filtrage!
# =============================================================================

# ===================== ACTIVATION DU FILTRAGE =====================
APPLY_FILE_FILTERING = False  # [!] Mettre True pour activer, False pour DÉSACTIVER

# OPTIONS DE FILTRAGE (utilisées seulement si APPLY_FILE_FILTERING = True)
INCLUDE_A = True   # Inclure les échantillons avec "-A" dans le nom
INCLUDE_H = False  # Inclure les échantillons avec "-H" dans le nom

print("="*70)
print("FILTRAGE PAR TYPE D'ÉCHANTILLON (-A / -H)")
print("="*70)
print(f"   Filtrage activé: {'[OK] OUI' if APPLY_FILE_FILTERING else '[X] NON (toutes les données conservées)'}")

if APPLY_FILE_FILTERING:
    print(f"   Inclure -A: {'[OK] Oui' if INCLUDE_A else '[X] Non'}")
    print(f"   Inclure -H: {'[OK] Oui' if INCLUDE_H else '[X] Non'}")

# =============================================================================
# AFFICHAGE DES COLONNES/MARQUEURS AVEC -A ET -H
# =============================================================================
print("\n" + "="*70)
print(" ANALYSE DES COLONNES (MARQUEURS) PAR TYPE -A / -H")
print("="*70)

# Récupérer les noms de colonnes
all_columns = list(combined_gated.var_names)

# Séparer les colonnes avec -A, -H, ou autres
cols_with_A = [col for col in all_columns if "-A" in col.upper()]
cols_with_H = [col for col in all_columns if "-H" in col.upper()]
cols_other = [col for col in all_columns if "-A" not in col.upper() and "-H" not in col.upper()]

print(f"\n🔵 COLONNES avec '-A' ({len(cols_with_A)}):")
for col in cols_with_A:
    print(f"   • {col}")

print(f"\n[+] COLONNES avec '-H' ({len(cols_with_H)}):")
for col in cols_with_H:
    print(f"   • {col}")

print(f"\n⚪ COLONNES sans -A/-H ({len(cols_other)}):")
for col in cols_other:
    print(f"   • {col}")

# =============================================================================
# FILTRAGE DES FICHIERS (si activé)
# =============================================================================
if APPLY_FILE_FILTERING:
    print("\n" + "="*70)
    print(" FILTRAGE DES FICHIERS PAR TYPE -A / -H")
    print("="*70)

    # Afficher les fichiers disponibles avant filtrage
    print(f"\n Fichiers disponibles AVANT filtrage:")
    file_counts_before = combined_gated.obs['file_origin'].value_counts()

    files_A = []
    files_H = []
    files_other = []

    for fname, count in file_counts_before.items():
        has_A = "-A" in fname.upper()
        has_H = "-H" in fname.upper()
        type_label = []
        if has_A:
            type_label.append("-A")
            files_A.append((fname, count))
        if has_H:
            type_label.append("-H")
            files_H.append((fname, count))
        if not has_A and not has_H:
            files_other.append((fname, count))
        type_str = ", ".join(type_label) if type_label else "(autre)"
        print(f"   {fname}: {count:,} cellules [{type_str}]")

    print(f"\n Résumé des fichiers:")
    print(f"   Fichiers -A: {len(files_A)} ({sum(c for _, c in files_A):,} cellules)")
    print(f"   Fichiers -H: {len(files_H)} ({sum(c for _, c in files_H):,} cellules)")
    print(f"   Fichiers autres: {len(files_other)} ({sum(c for _, c in files_other):,} cellules)")

    # Créer le masque de filtrage
    file_names = combined_gated.obs['file_origin'].values.astype(str)

    # Initialiser le masque à False (aucun échantillon inclus par défaut)
    mask_A = np.zeros(len(file_names), dtype=bool)
    mask_H = np.zeros(len(file_names), dtype=bool)

    # Appliquer les filtres
    for i, fname in enumerate(file_names):
        fname_upper = fname.upper()
        if "-A" in fname_upper:
            mask_A[i] = True
        if "-H" in fname_upper:
            mask_H[i] = True

    # Combiner selon les options
    if INCLUDE_A and INCLUDE_H:
        filter_mask = mask_A | mask_H
        filter_description = "échantillons -A ET -H"
    elif INCLUDE_A and not INCLUDE_H:
        filter_mask = mask_A
        filter_description = "échantillons -A uniquement"
    elif not INCLUDE_A and INCLUDE_H:
        filter_mask = mask_H
        filter_description = "échantillons -H uniquement"
    else:
        print("\n[!] ATTENTION: INCLUDE_A et INCLUDE_H sont tous les deux False!")
        print("   → Garde tous les échantillons par défaut")
        filter_mask = np.ones(len(file_names), dtype=bool)
        filter_description = "tous les échantillons (aucun filtre)"

    # [!] VÉRIFICATION: Si aucune cellule ne correspond, désactiver le filtrage
    n_matching = filter_mask.sum()
    if n_matching == 0:
        print("\n" + "!"*70)
        print("[!] ATTENTION: AUCUN FICHIER NE CORRESPOND AU FILTRE!")
        print("!"*70)
        print(f"   → Tes fichiers n'ont pas de suffixe -A ou -H")
        print(f"   → FILTRAGE DÉSACTIVÉ AUTOMATIQUEMENT")
        print(f"   → Toutes les cellules sont conservées")
        filter_mask = np.ones(len(file_names), dtype=bool)
        filter_description = "AUCUN (auto-désactivé - pas de fichiers correspondants)"

    # Appliquer le filtre
    n_before = combined_gated.shape[0]
    combined_gated_filtered = combined_gated[filter_mask].copy()
    n_after = combined_gated_filtered.shape[0]

    print(f"\n" + "="*70)
    print(f" RÉSULTAT DU FILTRAGE ({filter_description})")
    print("="*70)
    print(f"   Cellules avant filtrage: {n_before:,}")
    print(f"   Cellules après filtrage: {n_after:,}")
    if n_before > 0:
        print(f"   Cellules exclues: {n_before - n_after:,} ({100*(n_before-n_after)/n_before:.1f}%)")

    # Afficher les fichiers restants après filtrage
    if n_after > 0:
        print(f"\n Fichiers CONSERVÉS après filtrage:")
        file_counts_after = combined_gated_filtered.obs['file_origin'].value_counts()
        for fname, count in file_counts_after.items():
            print(f"   [OK] {fname}: {count:,} cellules")

    # Mettre à jour X_transformed
    X_transformed = X_transformed[filter_mask]
    combined_gated = combined_gated_filtered

else:
    # FILTRAGE DÉSACTIVÉ - garder toutes les données
    print("\n" + "="*70)
    print(" FILTRAGE DES FICHIERS DÉSACTIVÉ")
    print("="*70)
    print(f"   → Toutes les {combined_gated.shape[0]:,} cellules sont conservées")
    
    # Afficher quand même les fichiers disponibles
    print(f"\n Fichiers disponibles:")
    file_counts = combined_gated.obs['file_origin'].value_counts()
    for fname, count in file_counts.items():
        has_A = "-A" in fname.upper()
        has_H = "-H" in fname.upper()
        type_label = []
        if has_A:
            type_label.append("-A")
        if has_H:
            type_label.append("-H")
        type_str = ", ".join(type_label) if type_label else "(pas de suffixe -A/-H)"
        print(f"   {fname}: {count:,} cellules [{type_str}]")

# =============================================================================
# AFFICHAGE DES COLONNES UTILISÉES POUR FLOWSOM
# =============================================================================
print(f"\n" + "="*70)
print(" COLONNES DISPONIBLES POUR FLOWSOM")
print("="*70)
print(f"\nToutes les colonnes ({len(list(combined_gated.var_names))}):")
for i, col in enumerate(combined_gated.var_names):
    marker_type = ""
    if "-A" in col.upper():
        marker_type = " [Type -A]"
    elif "-H" in col.upper():
        marker_type = " [Type -H]"
    print(f"   [{i:2d}] {col}{marker_type}")

print(f"\n[OK] Données prêtes pour la suite du pipeline")
print(f"   Shape combined_gated: {combined_gated.shape}")
print(f"   Shape X_transformed: {X_transformed.shape}")

In [ ]:
# CRÉATION DE L'ANNDATA TRANSFORMÉ ET EXPLORATION POST-ARCSINH

# Créer un nouvel AnnData avec les données transformées (X_transformed)
import anndata as ad

# Créer adata_flowsom - le nouvel AnnData pour FlowSOM avec données transformées
adata_flowsom = ad.AnnData(
    X=X_transformed,  # Données POST-transformation arcsinh
    obs=combined_gated.obs.copy(),  # Copie des métadonnées
    var=combined_gated.var.copy() if combined_gated.var is not None else None
)

# Ajouter les noms de variables
adata_flowsom.var_names = var_names

print("="*70)
print("CRÉATION ANNDATA POUR FLOWSOM (DONNÉES POST-ARCSINH)")
print("="*70)
print(f"\n[OK] Nouvel AnnData 'adata_flowsom' créé avec données transformées")
print(f"   Shape: {adata_flowsom.shape}")
print(f"   Observations (cellules): {adata_flowsom.n_obs:,}")
print(f"   Variables (marqueurs): {adata_flowsom.n_vars}")

# ============================================================================
# EXPLORATION DU DATAFRAME POST-TRANSFORMATION
# ============================================================================

# Extraire la matrice transformée depuis le NOUVEL AnnData
X_trans = adata_flowsom.X
if hasattr(X_trans, 'toarray'):
    X_trans = X_trans.toarray()

# Créer un DataFrame pour exploration
df_transformed = pd.DataFrame(X_trans, columns=var_names)
df_transformed['condition'] = adata_flowsom.obs['condition'].values
df_transformed['file_origin'] = adata_flowsom.obs['file_origin'].values

print("\n" + "="*70)
print("APERÇU DES DONNÉES TRANSFORMÉES (premières 10 lignes)")
print("="*70)
print(f"Shape du DataFrame: {df_transformed.shape}")
display(df_transformed.head(10))

# VÉRIFICATION DES NaN ET Inf POST-ARCSINH

print("\n" + "="*70)
print("VÉRIFICATION DES VALEURS NaN ET Inf POST-ARCSINH")
print("="*70)

# Colonnes numériques uniquement
numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns.tolist()

# Comptage des NaN
nan_counts = df_transformed[numeric_cols].isna().sum()
total_nan = nan_counts.sum()

# Comptage des Inf (positifs et négatifs)
inf_pos_counts = (df_transformed[numeric_cols] == np.inf).sum()
inf_neg_counts = (df_transformed[numeric_cols] == -np.inf).sum()
total_inf_pos = inf_pos_counts.sum()
total_inf_neg = inf_neg_counts.sum()
total_inf = total_inf_pos + total_inf_neg

total_cells = df_transformed.shape[0] * len(numeric_cols)

print(f"\nRÉSUMÉ GLOBAL:")
print(f"   Total valeurs analysées: {total_cells:,}")
print(f"   Total NaN:    {total_nan:,} ({100*total_nan/total_cells:.4f}%)")
print(f"   Total +Inf:   {total_inf_pos:,} ({100*total_inf_pos/total_cells:.4f}%)")
print(f"   Total -Inf:   {total_inf_neg:,} ({100*total_inf_neg/total_cells:.4f}%)")

# Détail par colonne si problèmes détectés
if total_nan > 0 or total_inf > 0:
    print(f"\nDÉTAIL PAR COLONNE AVEC PROBLÈMES:")
    print("-"*60)
    for col in numeric_cols:
        n_nan = df_transformed[col].isna().sum()
        n_inf_pos = (df_transformed[col] == np.inf).sum()
        n_inf_neg = (df_transformed[col] == -np.inf).sum()
        if n_nan > 0 or n_inf_pos > 0 or n_inf_neg > 0:
            print(f"   {col:30s}: NaN={n_nan:,}, +Inf={n_inf_pos:,}, -Inf={n_inf_neg:,}")
else:
    print(f"\nAucune valeur NaN ou Inf détectée - Données propres!")

# ============================================================================
# STATISTIQUES DESCRIPTIVES POST-ARCSINH
# ============================================================================
print("\n" + "="*70)
print("STATISTIQUES DESCRIPTIVES POST-ARCSINH")
print("="*70)
display(df_transformed[numeric_cols].describe())

# ============================================================================
# VÉRIFICATION DES RANGES POST-TRANSFORMATION
# ============================================================================
print("\n" + "="*70)
print("VÉRIFICATION DES RANGES POST-TRANSFORMATION")
print("="*70)
print("(arcsinh avec cofactor=150 donne typiquement des valeurs entre -5 et 10)\n")

for col in used_markers[:10]:  # Premiers 10 marqueurs utilisés
    col_min = df_transformed[col].min()
    col_max = df_transformed[col].max()
    col_mean = df_transformed[col].mean()
    print(f"   {col:30s}: min={col_min:8.3f}, max={col_max:8.3f}, mean={col_mean:8.3f}")

In [ ]:
# NETTOYAGE FINAL ET VALIDATION DE L'ANNDATA POUR FLOWSOM

# Nettoyage final: remplacer NaN/Inf par 0 dans adata_flowsom
X_final = adata_flowsom.X
if hasattr(X_final, 'toarray'):
    X_final = X_final.toarray()

# Vérifier et nettoyer
nan_mask = ~np.isfinite(X_final)
n_nan = nan_mask.sum()
if n_nan > 0:
    print(f"[!] {n_nan} valeurs NaN/Inf détectées et remplacées par 0")
    X_final = np.nan_to_num(X_final, nan=0.0, posinf=0.0, neginf=0.0)
    adata_flowsom.X = X_final
else:
    print("[OK] Aucune valeur problématique - pas de nettoyage nécessaire")

print(f"\n[OK] AnnData 'adata_flowsom' prêt pour FlowSOM:")
print(f"   Shape: {adata_flowsom.shape}")
print(f"   Colonnes pour clustering: {len(cols_to_use)}")

# Résumé par condition
print(f"\n Distribution par condition:")
for condition in adata_flowsom.obs['condition'].unique():
    n = (adata_flowsom.obs['condition'] == condition).sum()
    print(f"   {condition}: {n:,} cellules")

## 9. Exécution du Clustering FlowSOM

Configuration et lancement de l'analyse FlowSOM avec:
- Grille SOM (xdim × ydim)
- Nombre de métaclusters
- Seed pour reproductibilité

In [ ]:
# PARAMÈTRES FLOWSOM

# Dimensions de la grille SOM
XDIM = 10
YDIM = 10

# Nombre de métaclusters
N_CLUSTERS = 7

# Seed pour reproductibilité
SEED = 42

# Auto-clustering avec silhouette score?
AUTO_CLUSTER = False
MAX_CLUSTERS_AUTO = 20

print("PARAMÈTRES FLOWSOM")
print("="*60)
print(f"   Grille SOM: {XDIM} × {YDIM} = {XDIM*YDIM} nodes")
print(f"   Métaclusters: {N_CLUSTERS}")
print(f"   Seed: {SEED}")
print(f"   Auto-clustering: {'Oui' if AUTO_CLUSTER else 'Non'}")

In [ ]:
# FONCTION POUR TROUVER LE NOMBRE OPTIMAL DE CLUSTERS (optionnel)
# [!] Le silhouette score nécessite une matrice N×N → impossible avec 1M cellules
# Solution: Sous-échantillonner pour l'évaluation silhouette uniquement

SAMPLE_SIZE_SILHOUETTE = 10000  # Taille de l'échantillon pour silhouette

def find_optimal_clusters(data, cols_to_use, seed, max_clusters=20, sample_size=10000):
    """
    Trouve le nombre optimal de métaclusters via silhouette score.
    Utilise un échantillon représentatif pour éviter l'explosion mémoire.
    """
    print("Recherche du nombre optimal de clusters...")
    
    np.random.seed(seed)
    
    X = data.X
    if hasattr(X, 'toarray'):
        X = X.toarray()
    
    X_full = X[:, cols_to_use]
    X_full = np.nan_to_num(X_full, nan=0.0)
    
    n_total = X_full.shape[0]
    
    # Sous-échantillonner pour silhouette (sinon O(N²) mémoire)
    if n_total > sample_size:
        print(f"   [!] {n_total:,} cellules → échantillon de {sample_size:,} pour silhouette")
        idx = np.random.choice(n_total, sample_size, replace=False)
        X_sample = X_full[idx]
    else:
        print(f"   Utilisation de {n_total:,} cellules")
        X_sample = X_full
    
    scores = []
    cluster_range = range(2, min(max_clusters + 1, len(X_sample) // 10))
    
    for k in cluster_range:
        try:
            clustering = AgglomerativeClustering(n_clusters=k)
            labels = clustering.fit_predict(X_sample)
            score = silhouette_score(X_sample, labels)
            scores.append((k, score))
            print(f"   k={k}: silhouette={score:.4f}")
        except Exception as e:
            print(f"   k={k}: erreur - {e}")
    
    if scores:
        best_k, best_score = max(scores, key=lambda x: x[1])
        print(f"\n[OK] Nombre optimal: {best_k} (silhouette={best_score:.4f})")
        return best_k
    
    return 10  # Valeur par défaut

# Exécuter si AUTO_CLUSTER est activé
if AUTO_CLUSTER:
    N_CLUSTERS = find_optimal_clusters(
        combined_gated, cols_to_use, SEED, 
        MAX_CLUSTERS_AUTO, SAMPLE_SIZE_SILHOUETTE
    )
    print(f"\n Utilisation de {N_CLUSTERS} métaclusters")

In [ ]:
# =============================================================================
# EXÉCUTION FLOWSOM
# =============================================================================

import time
start_time = time.time()

# Exécuter FlowSOM avec adata_flowsom (données transformées arcsinh)
fsom = fs.FlowSOM(
    adata_flowsom,  # ← IMPORTANT: utilise les données POST-transformation
    cols_to_use=cols_to_use,
    xdim=XDIM,
    ydim=YDIM,
    n_clusters=N_CLUSTERS,
    seed=SEED
)

elapsed = time.time() - start_time
print(f"\nTemps d'exécution: {elapsed:.2f} secondes")

# Récupérer les données de clustering
cell_data = fsom.get_cell_data()
cluster_data = fsom.get_cluster_data()

# Ajouter les métadonnées originales
cell_data.obs['condition'] = adata_flowsom.obs['condition'].values
cell_data.obs['file_origin'] = adata_flowsom.obs['file_origin'].values

print(f"\n[OK] FlowSOM terminé!")
print(f"   Cellules analysées: {cell_data.shape[0]:,}")
print(f"   Nodes SOM: {cluster_data.shape[0]}")
print(f"   Métaclusters: {N_CLUSTERS}")

## 10. Visualisation des Résultats FlowSOM

Génération des visualisations standards:
- Heatmap d'expression par métacluster
- Arbre MST (Minimum Spanning Tree)
- Star Charts
- Distribution par condition

In [ ]:
# =============================================================================
# HEATMAP D'EXPRESSION PAR MÉTACLUSTER
# =============================================================================

print(" Génération de la Heatmap d'expression...")

# Récupérer les données
X = cell_data.X
if hasattr(X, 'toarray'):
    X = X.toarray()

metaclustering = cell_data.obs['metaclustering'].values

# Calculer la MFI (Mean Fluorescence Intensity) par métacluster
mfi_matrix = np.zeros((N_CLUSTERS, len(cols_to_use)))

for i in range(N_CLUSTERS):
    mask = metaclustering == i
    if mask.sum() > 0:
        mfi_matrix[i, :] = np.nanmean(X[mask][:, cols_to_use], axis=0)

# Normalisation Z-score pour la heatmap
mfi_normalized = (mfi_matrix - np.nanmean(mfi_matrix, axis=0)) / (np.nanstd(mfi_matrix, axis=0) + 1e-10)

# Créer la heatmap
fig, ax = plt.subplots(figsize=(14, 8))

im = ax.imshow(mfi_normalized.T, aspect='auto', cmap='RdBu_r', vmin=-2, vmax=2)

# Labels
ax.set_yticks(range(len(used_markers)))
ax.set_yticklabels(used_markers, fontsize=9)
ax.set_xticks(range(N_CLUSTERS))
ax.set_xticklabels([f'MC{i}' for i in range(N_CLUSTERS)], fontsize=10)

ax.set_title('Heatmap - Expression par Métacluster (Z-score)', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Métacluster', fontsize=12)
ax.set_ylabel('Marqueur', fontsize=12)

# Colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8, label='Z-score')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# STAR CHART FLOWSOM (MST View)
# =============================================================================

print("Génération du Star Chart MST...")

try:
    # Utiliser l'API FlowSOM pour le Star Chart
    fig_stars = fs.pl.plot_stars(
        fsom,
        background_values=fsom.get_cluster_data().obs.metaclustering,
        view="MST"
    )
    plt.suptitle('FlowSOM Star Chart (MST View)', fontsize=14, fontweight='bold')
    plt.show()
except Exception as e:
    print(f"Erreur Star Chart: {e}")
    print("   Utilisation de la visualisation alternative...")

In [ ]:
# =============================================================================
# VISUALISATION GRILLE SOM (xGrid, yGrid) - Style FlowSOM R exact
# =============================================================================
# PARTIE 1 : MATPLOTLIB (statique, haute résolution pour PDF/export)
# =============================================================================

print(" VISUALISATION GRILLE SOM (style FlowSOM R avec cercles)")
print("="*70)

# =====================================================================
# FONCTION JITTER CIRCULAIRE (style FlowSOM R)
# =====================================================================
def circular_jitter_viz(n_points, cluster_ids, node_sizes, max_radius=0.45, min_radius=0.1):
    """
    Génère un jitter circulaire style FlowSOM R.
    Le rayon des cercles dépend du nombre de cellules dans le node.
    """
    theta = np.random.uniform(0, 2 * np.pi, n_points)
    u = np.random.uniform(0, 1, n_points)
    
    max_size_val = node_sizes.max()
    
    radii = np.zeros(n_points, dtype=np.float32)
    for i in range(n_points):
        node_id = int(cluster_ids[i])
        node_size = node_sizes[node_id]
        size_ratio = np.sqrt(node_size / max_size_val)
        node_radius = min_radius + (max_radius - min_radius) * size_ratio
        radii[i] = node_radius
    
    r = np.sqrt(u) * radii
    
    jitter_x = r * np.cos(theta)
    jitter_y = r * np.sin(theta)
    
    return jitter_x.astype(np.float32), jitter_y.astype(np.float32)

try:
    # Récupérer les coordonnées de grille
    grid_coords = cluster_data.obsm.get('grid', None)
    
    if grid_coords is not None:
        # Récupérer les infos de clustering
        clustering = cell_data.obs['clustering'].values
        metaclustering_nodes = cluster_data.obs['metaclustering'].values
        conditions = cell_data.obs['condition'].values
        
        # Calculer les coordonnées de grille pour chaque cellule
        xGrid_base = np.array([grid_coords[int(c), 0] for c in clustering], dtype=np.float32)
        yGrid_base = np.array([grid_coords[int(c), 1] for c in clustering], dtype=np.float32)
        
        # Décaler pour commencer à 1
        xGrid_shifted = xGrid_base - xGrid_base.min() + 1
        yGrid_shifted = yGrid_base - yGrid_base.min() + 1
        
        # Métacluster pour chaque cellule
        metaclustering_cells = np.array([metaclustering_nodes[int(c)] for c in clustering])
        
        # Calculer la taille de chaque node
        n_nodes = len(cluster_data)
        node_sizes = np.zeros(n_nodes, dtype=np.float32)
        for i in range(n_nodes):
            node_sizes[i] = (clustering == i).sum()
        
        # JITTER CIRCULAIRE style FlowSOM R
        MAX_NODE_SIZE = 0.45
        MIN_NODE_SIZE = 0.1
        np.random.seed(SEED)
        jitter_x, jitter_y = circular_jitter_viz(len(clustering), clustering, node_sizes, 
                                                  max_radius=MAX_NODE_SIZE, 
                                                  min_radius=MIN_NODE_SIZE)
        
        print(f" Jitter circulaire appliqué (rayon proportionnel à la taille du node)")
        print(f"   Rayon min: {MIN_NODE_SIZE}, Rayon max: {MAX_NODE_SIZE}")
        
        # Créer la figure avec 2 sous-plots
        fig, axes = plt.subplots(1, 2, figsize=(16, 7))
        
        # =====================================================================
        # Plot 1: Grille SOM colorée par Métacluster
        # =====================================================================
        ax1 = axes[0]
        
        n_meta = len(np.unique(metaclustering_nodes))
        cmap = plt.cm.tab20 if n_meta <= 20 else plt.cm.turbo
        
        scatter1 = ax1.scatter(
            xGrid_shifted + jitter_x, 
            yGrid_shifted + jitter_y,
            c=metaclustering_cells,
            cmap=cmap,
            s=5,
            alpha=0.5,
            edgecolors='none'
        )
        
        # Ajouter les labels des métaclusters au centre de chaque node
        for node_id in range(n_nodes):
            if node_sizes[node_id] > 0:
                x_pos = grid_coords[node_id, 0] - xGrid_base.min() + 1
                y_pos = grid_coords[node_id, 1] - yGrid_base.min() + 1
                meta_id = metaclustering_nodes[node_id]
                ax1.annotate(
                    str(int(meta_id + 1)),
                    (x_pos, y_pos),
                    ha='center', va='center',
                    fontsize=8, fontweight='bold',
                    color='white',
                    bbox=dict(boxstyle='circle,pad=0.2', facecolor=cmap(meta_id / max(n_meta - 1, 1)), edgecolor='white', alpha=0.9)
                )
        
        ax1.set_xlabel('xGrid', fontsize=12, fontweight='bold')
        ax1.set_ylabel('yGrid', fontsize=12, fontweight='bold')
        ax1.set_title(f'Grille FlowSOM - {XDIM}x{YDIM} nodes\nColoré par Métacluster (style FlowSOM R)', 
                     fontsize=12, fontweight='bold')
        ax1.set_xlim(0.5, XDIM + 1.5)
        ax1.set_ylim(0.5, YDIM + 1.5)
        ax1.set_aspect('equal')
        ax1.grid(True, alpha=0.3, linestyle='--')
        
        cbar1 = plt.colorbar(scatter1, ax=ax1, label='Métacluster')
        
        # =====================================================================
        # Plot 2: Grille SOM colorée par Condition
        # =====================================================================
        ax2 = axes[1]
        
        condition_num = np.array([0 if c == 'Sain' else 1 for c in conditions])
        
        from matplotlib.colors import ListedColormap
        cmap_cond = ListedColormap(['#a6e3a1', '#f38ba8'])
        
        scatter2 = ax2.scatter(
            xGrid_shifted + jitter_x, 
            yGrid_shifted + jitter_y,
            c=condition_num,
            cmap=cmap_cond,
            s=5,
            alpha=0.5,
            edgecolors='none'
        )
        
        ax2.set_xlabel('xGrid', fontsize=12, fontweight='bold')
        ax2.set_ylabel('yGrid', fontsize=12, fontweight='bold')
        ax2.set_title(f'Grille FlowSOM - {XDIM}x{YDIM} nodes\nColoré par Condition (style FlowSOM R)', 
                     fontsize=12, fontweight='bold')
        ax2.set_xlim(0.5, XDIM + 1.5)
        ax2.set_ylim(0.5, YDIM + 1.5)
        ax2.set_aspect('equal')
        ax2.grid(True, alpha=0.3, linestyle='--')
        
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='#a6e3a1', edgecolor='white', label='Sain (NBM)'),
            Patch(facecolor='#f38ba8', edgecolor='white', label='Pathologique')
        ]
        ax2.legend(handles=legend_elements, loc='upper right')
        
        plt.tight_layout()
        plt.show()
        
        # Afficher les statistiques
        print(f"\n STATISTIQUES DE LA GRILLE SOM:")
        print(f"   Dimensions: {XDIM} x {YDIM} = {XDIM * YDIM} nodes")
        print(f"   Nodes utilisés: {(node_sizes > 0).sum()} / {n_nodes}")
        print(f"   xGrid range: [{xGrid_shifted.min():.1f}, {xGrid_shifted.max():.1f}]")
        print(f"   yGrid range: [{yGrid_shifted.min():.1f}, {yGrid_shifted.max():.1f}]")
        
        # Afficher la taille des nodes
        print(f"\n Distribution des tailles de nodes:")
        print(f"   Min: {node_sizes.min():.0f} cellules")
        print(f"   Max: {node_sizes.max():.0f} cellules")
        print(f"   Moyenne: {node_sizes.mean():.0f} cellules")
        
        # =================================================================
        # PARTIE 2 : PLOTLY INTERACTIF (zoom, hover, export)
        # =================================================================
        print("\n" + "="*70)
        print(" GRILLE SOM — VERSION PLOTLY INTERACTIVE")
        print("="*70)
        
        import plotly.graph_objects as go
        import plotly.colors as pc_grid
        
        _xj = xGrid_shifted + jitter_x
        _yj = yGrid_shifted + jitter_y
        
        # Sous-échantillonner si trop de points pour la fluidité Plotly
        _max_pts = 50_000
        if len(clustering) > _max_pts:
            np.random.seed(SEED)
            _sample_idx = np.random.choice(len(clustering), _max_pts, replace=False)
        else:
            _sample_idx = np.arange(len(clustering))
        
        if n_meta <= 20:
            _mc_palette = pc_grid.qualitative.Alphabet[:n_meta]
        else:
            _mc_palette = [f"hsl({int(i*360/n_meta)},70%,55%)" for i in range(n_meta)]
        
        # --- Plot 1 Plotly : Métacluster ---
        fig_grid_mc = go.Figure()
        
        for mc_id in range(n_meta):
            _mask_mc = metaclustering_cells[_sample_idx] == mc_id
            if _mask_mc.sum() == 0:
                continue
            _si = _sample_idx[_mask_mc]
            fig_grid_mc.add_trace(go.Scattergl(
                x=_xj[_si], y=_yj[_si],
                mode='markers',
                marker=dict(size=3, color=_mc_palette[mc_id % len(_mc_palette)], opacity=0.5),
                name=f"MC{mc_id} ({_mask_mc.sum():,})",
                hovertemplate=f"MC{mc_id}<br>xGrid: %{{x:.2f}}<br>yGrid: %{{y:.2f}}<extra></extra>",
            ))
        
        _node_x = [grid_coords[i, 0] - xGrid_base.min() + 1 for i in range(n_nodes) if node_sizes[i] > 0]
        _node_y = [grid_coords[i, 1] - yGrid_base.min() + 1 for i in range(n_nodes) if node_sizes[i] > 0]
        _node_txt = [str(int(metaclustering_nodes[i]+1)) for i in range(n_nodes) if node_sizes[i] > 0]
        _node_sz = [node_sizes[i] for i in range(n_nodes) if node_sizes[i] > 0]
        
        fig_grid_mc.add_trace(go.Scatter(
            x=_node_x, y=_node_y,
            mode='text',
            text=_node_txt,
            textfont=dict(size=9, color='black', family='Arial Black'),
            hovertemplate=[f"Node — MC{t}<br>{int(s):,} cellules<extra></extra>" for t, s in zip(_node_txt, _node_sz)],
            showlegend=False,
        ))
        
        fig_grid_mc.update_layout(
            title=dict(
                text=f"<b>Grille FlowSOM — {XDIM}×{YDIM} nodes — Coloré par Métacluster</b><br>"
                     "<sup>Style FlowSOM R (jitter circulaire proportionnel) — Interactif</sup>",
                font=dict(size=14),
            ),
            xaxis=dict(title="xGrid", range=[0.3, XDIM+1.7], scaleanchor="y", scaleratio=1,
                       gridcolor="rgba(0,0,0,0.08)", gridwidth=1),
            yaxis=dict(title="yGrid", range=[0.3, YDIM+1.7],
                       gridcolor="rgba(0,0,0,0.08)", gridwidth=1),
            height=700, width=800,
            paper_bgcolor="#fafafa", plot_bgcolor="#f5f5f5",
            legend=dict(title="Métacluster", font=dict(size=10),
                        bgcolor="rgba(255,255,255,0.85)",
                        bordercolor="#ccc", borderwidth=1,
                        yanchor="top", y=1, xanchor="left", x=1.02),
            margin=dict(t=80, b=50, l=60, r=180),
        )
        fig_grid_mc.show()
        
        # --- Plot 2 Plotly : Condition ---
        fig_grid_cond = go.Figure()
        
        _cond_colors = {"Sain": "#2ca02c", "Pathologique": "#d62728"}
        for cond_label, cond_color in _cond_colors.items():
            _mask_c = conditions[_sample_idx] == cond_label
            if _mask_c.sum() == 0:
                continue
            _si = _sample_idx[_mask_c]
            fig_grid_cond.add_trace(go.Scattergl(
                x=_xj[_si], y=_yj[_si],
                mode='markers',
                marker=dict(size=3, color=cond_color, opacity=0.45),
                name=f"{cond_label} ({_mask_c.sum():,})",
                hovertemplate=f"{cond_label}<br>xGrid: %{{x:.2f}}<br>yGrid: %{{y:.2f}}<extra></extra>",
            ))
        
        fig_grid_cond.update_layout(
            title=dict(
                text=f"<b>Grille FlowSOM — {XDIM}×{YDIM} nodes — Coloré par Condition</b><br>"
                     "<sup>Style FlowSOM R (jitter circulaire proportionnel) — Interactif</sup>",
                font=dict(size=14),
            ),
            xaxis=dict(title="xGrid", range=[0.3, XDIM+1.7], scaleanchor="y", scaleratio=1,
                       gridcolor="rgba(0,0,0,0.08)", gridwidth=1),
            yaxis=dict(title="yGrid", range=[0.3, YDIM+1.7],
                       gridcolor="rgba(0,0,0,0.08)", gridwidth=1),
            height=700, width=800,
            paper_bgcolor="#fafafa", plot_bgcolor="#f5f5f5",
            legend=dict(title="Condition", font=dict(size=12),
                        bgcolor="rgba(255,255,255,0.85)",
                        bordercolor="#ccc", borderwidth=1),
            margin=dict(t=80, b=50, l=60, r=60),
        )
        fig_grid_cond.show()
        
    else:
        print("[!] Coordonnées de grille non disponibles dans cluster_data.obsm['grid']")
        
except Exception as e:
    import traceback
    print(f"[!] Erreur visualisation grille: {e}")
    traceback.print_exc()

In [ ]:
# =============================================================================
# ARBRE MST — MATPLOTLIB (statique) + PLOTLY (interactif)
# =============================================================================

print("Génération de l'arbre MST...")

# =====================================================================
# PARTIE 1 : MATPLOTLIB (statique, haute résolution pour PDF/export)
# =====================================================================
try:
    layout = cluster_data.obsm.get('layout', None)
    
    if layout is not None:
        clustering = cell_data.obs['clustering'].values
        metaclustering_nodes = cluster_data.obs['metaclustering'].values
        
        n_nodes = len(cluster_data)
        node_sizes = np.zeros(n_nodes)
        for i in range(n_nodes):
            node_sizes[i] = (clustering == i).sum()
        
        max_size = node_sizes.max() if node_sizes.max() > 0 else 1
        sizes = 100 + (node_sizes / max_size) * 800
        
        n_meta = len(np.unique(metaclustering_nodes))
        cmap = plt.cm.tab20 if n_meta <= 20 else plt.cm.turbo
        colors = [cmap(int(m) / max(n_meta - 1, 1)) for m in metaclustering_nodes]
        
        fig, ax = plt.subplots(figsize=(12, 10))
        
        scatter = ax.scatter(layout[:, 0], layout[:, 1], 
                           s=sizes, c=colors, edgecolors='white', 
                           linewidths=1.5, alpha=0.9, zorder=2)
        
        for i in range(n_nodes):
            ax.annotate(str(int(metaclustering_nodes[i])), 
                       (layout[i, 0], layout[i, 1]),
                       ha='center', va='center', fontsize=8, 
                       color='white', fontweight='bold')
        
        ax.set_xlabel('xNodes', fontsize=12, fontweight='bold')
        ax.set_ylabel('yNodes', fontsize=12, fontweight='bold')
        ax.set_title(f'Arbre MST - {n_nodes} nodes, {n_meta} métaclusters', 
                    fontsize=14, fontweight='bold', pad=15)
        ax.grid(True, alpha=0.15, linestyle='--')
        
        from matplotlib.patches import Patch
        if n_meta <= 15:
            legend_elements = [Patch(facecolor=cmap(i/max(n_meta-1, 1)), 
                                    label=f'MC {i}') for i in range(n_meta)]
            ax.legend(handles=legend_elements, loc='center left', 
                     bbox_to_anchor=(1.02, 0.5), fontsize=9)
        
        plt.tight_layout()
        plt.show()
        
        # =================================================================
        # PARTIE 2 : PLOTLY INTERACTIF (zoom, hover, export)
        # =================================================================
        print("\n" + "="*70)
        print(" ARBRE MST — VERSION PLOTLY INTERACTIVE")
        print("="*70)
        
        import plotly.graph_objects as go
        import plotly.colors as pc_mst
        
        _bubble_sizes = 10 + (node_sizes / max_size) * 40
        
        if n_meta <= 20:
            _mc_palette = pc_mst.qualitative.Alphabet[:n_meta]
        else:
            _mc_palette = [f"hsl({int(i*360/n_meta)},70%,55%)" for i in range(n_meta)]
        
        # Récupérer les arêtes du MST si disponibles
        _edge_x, _edge_y = [], []
        _mst_graph = None
        try:
            if hasattr(cluster_data, 'uns') and 'mst' in cluster_data.uns:
                _mst_graph = cluster_data.uns['mst']
        except Exception:
            pass
        
        if _mst_graph is not None:
            try:
                import igraph
                if isinstance(_mst_graph, igraph.Graph):
                    for edge in _mst_graph.es:
                        s, t = edge.source, edge.target
                        if s < n_nodes and t < n_nodes:
                            _edge_x += [layout[s, 0], layout[t, 0], None]
                            _edge_y += [layout[s, 1], layout[t, 1], None]
            except ImportError:
                pass
        
        fig_mst = go.Figure()
        
        if _edge_x:
            fig_mst.add_trace(go.Scatter(
                x=_edge_x, y=_edge_y,
                mode='lines',
                line=dict(width=1.5, color='rgba(100,100,100,0.5)'),
                hoverinfo='skip',
                showlegend=False,
            ))
        
        for mc_id in range(n_meta):
            _mask = metaclustering_nodes == mc_id
            if _mask.sum() == 0:
                continue
            _indices = np.where(_mask)[0]
            fig_mst.add_trace(go.Scatter(
                x=layout[_indices, 0],
                y=layout[_indices, 1],
                mode='markers+text',
                marker=dict(
                    size=_bubble_sizes[_indices],
                    color=_mc_palette[mc_id % len(_mc_palette)],
                    line=dict(width=1.5, color='white'),
                    opacity=0.9,
                ),
                text=[str(int(mc_id)) for _ in _indices],
                textfont=dict(size=9, color='white', family='Arial Black'),
                textposition='middle center',
                name=f"MC{mc_id} ({int(node_sizes[_indices].sum()):,} cells)",
                hovertemplate=[
                    f"<b>Node {ni}</b><br>"
                    f"MC {int(metaclustering_nodes[ni])}<br>"
                    f"Cellules: {int(node_sizes[ni]):,}<br>"
                    f"x: {layout[ni,0]:.2f}, y: {layout[ni,1]:.2f}<extra></extra>"
                    for ni in _indices
                ],
            ))
        
        fig_mst.update_layout(
            title=dict(
                text=f"<b>Arbre MST — {n_nodes} nodes, {n_meta} métaclusters</b><br>"
                     "<sup>Taille des bulles ∝ nombre de cellules · Cliquez la légende pour filtrer</sup>",
                font=dict(size=14),
            ),
            xaxis=dict(title="xNodes", showgrid=True, gridcolor="rgba(0,0,0,0.06)",
                       zeroline=False),
            yaxis=dict(title="yNodes", showgrid=True, gridcolor="rgba(0,0,0,0.06)",
                       zeroline=False, scaleanchor="x", scaleratio=1),
            height=750, width=900,
            paper_bgcolor="#fafafa", plot_bgcolor="#f5f5f5",
            legend=dict(
                title="Métacluster",
                font=dict(size=10),
                bgcolor="rgba(255,255,255,0.85)",
                bordercolor="#ccc", borderwidth=1,
                yanchor="top", y=1, xanchor="left", x=1.02,
            ),
            margin=dict(t=80, b=50, l=60, r=200),
        )
        fig_mst.show()
        
        print(f"[OK] Arbre MST — Matplotlib + Plotly interactif ({n_nodes} nodes, {n_meta} MC)")
    else:
        print("[!] Layout MST non disponible")
        
except Exception as e:
    import traceback
    print(f"[!] Erreur MST: {e}")
    traceback.print_exc()

In [ ]:
# =============================================================================
# DISTRIBUTION PAR CONDITION (Sain vs Pathologique)
# =============================================================================

print("Distribution des métaclusters par condition...")

metaclustering = cell_data.obs['metaclustering'].values
conditions = cell_data.obs['condition'].values

healthy_pcts = []
patho_pcts = []

for i in range(N_CLUSTERS):
    mask_cluster = metaclustering == i
    
    # Pourcentage dans Sain
    mask_healthy = (conditions == 'Sain') & mask_cluster
    total_healthy = (conditions == 'Sain').sum()
    healthy_pcts.append((mask_healthy.sum() / total_healthy * 100) if total_healthy > 0 else 0)
    
    # Pourcentage dans Pathologique
    mask_patho = (conditions == 'Pathologique') & mask_cluster
    total_patho = (conditions == 'Pathologique').sum()
    patho_pcts.append((mask_patho.sum() / total_patho * 100) if total_patho > 0 else 0)

# Créer le graphique
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(N_CLUSTERS)
width = 0.35

bars1 = ax.bar(x - width/2, healthy_pcts, width, label='Sain (NBM)', 
               color='#a6e3a1', edgecolor='white', linewidth=0.5)
bars2 = ax.bar(x + width/2, patho_pcts, width, label='Pathologique', 
               color='#f38ba8', edgecolor='white', linewidth=0.5)

ax.set_xlabel('Métacluster', fontsize=12)
ax.set_ylabel('Pourcentage (%)', fontsize=12)
ax.set_title('Distribution des Métaclusters par Condition', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels([f'MC{i}' for i in range(N_CLUSTERS)], fontsize=10)
ax.legend()
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Ajouter les valeurs sur les barres
for bar in bars1 + bars2:
    height = bar.get_height()
    if height > 1:  # N'afficher que si > 1%
        ax.annotate(f'{height:.1f}%',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3), textcoords="offset points",
                   ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# Tableau récapitulatif
print("\nTableau récapitulatif:")
print("-" * 50)
print(f"{'MC':>4} | {'Sain (%)':>10} | {'Patho (%)':>10} | {'Diff':>8}")
print("-" * 50)
for i in range(N_CLUSTERS):
    diff = patho_pcts[i] - healthy_pcts[i]
    print(f"{i:>4} | {healthy_pcts[i]:>10.2f} | {patho_pcts[i]:>10.2f} | {diff:>+8.2f}")
print("-" * 50)

## 11. Analyse Détaillée des Métaclusters

Statistiques approfondies par métacluster:
- Nombre de cellules
- MFI par marqueur
- Phénotype caractéristique

In [ ]:
# =============================================================================
# STATISTIQUES PAR MÉTACLUSTER
# =============================================================================

print(" STATISTIQUES PAR MÉTACLUSTER")
print("="*80)

# Créer un DataFrame de statistiques
stats_data = []

for i in range(N_CLUSTERS):
    mask = metaclustering == i
    n_cells = mask.sum()
    pct_total = n_cells / len(metaclustering) * 100
    
    # Calculer MFI pour chaque marqueur
    mfi = np.nanmean(X[mask][:, cols_to_use], axis=0) if n_cells > 0 else np.zeros(len(cols_to_use))
    
    # Top 3 marqueurs les plus exprimés
    top_indices = np.argsort(mfi)[::-1][:3]
    top_markers = [used_markers[idx] for idx in top_indices]
    
    stats_data.append({
        'Metacluster': i,
        'N_Cells': n_cells,
        'Pct_Total': pct_total,
        'Top_Markers': ', '.join(top_markers)
    })

df_stats = pd.DataFrame(stats_data)
print(df_stats.to_string(index=False))

# Graphique camembert
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart des tailles
ax = axes[0]
sizes = [s['N_Cells'] for s in stats_data]
labels = [f"MC{s['Metacluster']}" for s in stats_data]
colors = plt.cm.tab20(np.linspace(0, 1, N_CLUSTERS))

wedges, texts, autotexts = ax.pie(sizes, labels=labels, colors=colors, 
                                   autopct='%1.1f%%', pctdistance=0.8)
ax.set_title('Distribution des Cellules par Métacluster', fontsize=12, fontweight='bold')

# Bar chart des tailles
ax = axes[1]
ax.barh(range(N_CLUSTERS), sizes, color=colors, edgecolor='white')
ax.set_yticks(range(N_CLUSTERS))
ax.set_yticklabels(labels)
ax.set_xlabel('Nombre de cellules')
ax.set_title('Taille des Métaclusters', fontsize=12, fontweight='bold')
ax.grid(axis='x', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# PROFIL D'EXPRESSION DÉTAILLÉ PAR MÉTACLUSTER — SPIDER PLOT INTERACTIF
# =============================================================================

print("\n PROFIL D'EXPRESSION MOYEN PAR MÉTACLUSTER")
print("="*80)

# Créer un DataFrame avec MFI par marqueur et métacluster
mfi_matrix = np.zeros((N_CLUSTERS, len(used_markers)))

for i in range(N_CLUSTERS):
    mask = metaclustering == i
    if mask.sum() > 0:
        mfi_matrix[i] = np.nanmean(X[mask][:, cols_to_use], axis=0)

df_mfi = pd.DataFrame(mfi_matrix, 
                       columns=used_markers,
                       index=[f'MC{i}' for i in range(N_CLUSTERS)])

# Afficher le tableau formaté
print(df_mfi.round(2).to_string())

# =============================================================================
# SPIDER / RADAR PLOT INTERACTIF — TOUS LES CLUSTERS (Plotly)
# =============================================================================

import plotly.graph_objects as go
import plotly.colors as pc

# Palette de couleurs suffisante pour tous les clusters
if N_CLUSTERS <= 10:
    _radar_palette = pc.qualitative.Set3
elif N_CLUSTERS <= 20:
    _radar_palette = pc.qualitative.Alphabet
else:
    _radar_palette = [f"hsl({int(i*360/N_CLUSTERS)},70%,55%)" for i in range(N_CLUSTERS)]

fig_radar = go.Figure()

for cluster_id in range(N_CLUSTERS):
    values = mfi_matrix[cluster_id].copy()
    # Normaliser entre 0 et 1 pour la visualisation
    v_min, v_max = np.min(values), np.max(values)
    values_norm = (values - v_min) / (v_max - v_min + 1e-10)

    _c = _radar_palette[cluster_id % len(_radar_palette)]
    _n_cells = int((metaclustering == cluster_id).sum())

    # Construire une couleur de remplissage semi-transparente
    if 'rgb' in str(_c):
        _fill = _c.replace(')', ',0.08)').replace('rgb', 'rgba')
    else:
        _fill = f"rgba(128,128,128,0.05)"

    fig_radar.add_trace(go.Scatterpolar(
        r=np.append(values_norm, values_norm[0]),
        theta=used_markers + [used_markers[0]],
        fill='toself',
        fillcolor=_fill,
        opacity=0.85,
        name=f"MC{cluster_id}  ({_n_cells:,} cells)",
        line=dict(color=_c, width=2),
        marker=dict(size=5),
        customdata=np.stack([
            np.append(mfi_matrix[cluster_id], mfi_matrix[cluster_id][0]),
            np.append(values_norm, values_norm[0]),
        ], axis=-1),
        hovertemplate=(
            f"<b>MC{cluster_id}</b><br>"
            "Marqueur: %{theta}<br>"
            "MFI brute: %{customdata[0]:.2f}<br>"
            "Normalisé: %{customdata[1]:.3f}<extra></extra>"
        ),
    ))

fig_radar.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1.05],
            tickfont=dict(size=9),
            gridcolor="rgba(0,0,0,0.12)",
        ),
        angularaxis=dict(
            tickfont=dict(size=11),
            gridcolor="rgba(0,0,0,0.12)",
            rotation=90,
            direction="clockwise",
        ),
        bgcolor="rgba(250,250,250,0.5)",
    ),
    title=dict(
        text=f"<b>Profil d'Expression Normalisé — {N_CLUSTERS} Métaclusters</b><br>"
             "<sup>Cliquez sur la légende pour masquer/afficher un cluster</sup>",
        font=dict(size=15),
    ),
    legend=dict(
        title="Métacluster",
        font=dict(size=11),
        bgcolor="rgba(255,255,255,0.85)",
        bordercolor="#ccc", borderwidth=1,
        yanchor="top", y=1.0,
        xanchor="left", x=1.05,
    ),
    height=750,
    width=950,
    paper_bgcolor="#fafafa",
    margin=dict(t=90, b=40, l=80, r=200),
)

fig_radar.show()
print(f"\n[OK] Spider plot interactif — {N_CLUSTERS} métaclusters affichés")
print("     → Cliquez sur la légende pour isoler un métacluster")
print("     → Survolez les points pour voir les MFI brutes")

In [ ]:
# =============================================================================
# ANALYSE DES CLUSTERS EXCLUSIFS (mono-condition)
# =============================================================================
# Identification des clusters contenant UNIQUEMENT des cellules d'une condition
# Utile pour détecter les populations pathologiques spécifiques
# =============================================================================

print("="*70)
print(" ANALYSE DES CLUSTERS EXCLUSIFS PAR CONDITION")
print("="*70)

# Récupérer les conditions des cellules
cell_conditions = adata_flowsom.obs['condition'].values
unique_conditions = np.unique(cell_conditions)

print(f"\nConditions présentes: {list(unique_conditions)}")
print(f"Nombre de metaclusters: {n_meta}")

# Analyse par metacluster
clusters_patho_only = []
clusters_sain_only = []
clusters_mixed = []

print("\n" + "-"*70)
print(" METACLUSTERS EXCLUSIFS (100% d'une seule condition)")
print("-"*70)

for cluster_id in range(1, n_meta + 1):
    mask_cluster = (metaclustering_cells == cluster_id)
    n_cluster = mask_cluster.sum()
    
    if n_cluster == 0:
        continue
    
    # Compter les cellules par condition dans ce cluster
    conditions_in_cluster = cell_conditions[mask_cluster]
    
    # Calculer les proportions
    condition_counts = {}
    for cond in unique_conditions:
        count = (conditions_in_cluster == cond).sum()
        condition_counts[cond] = count
    
    # Vérifier si le cluster est exclusif à une condition
    total = sum(condition_counts.values())
    
    # Cluster 100% pathologique
    if "Pathologique" in condition_counts and condition_counts.get("Pathologique", 0) == total:
        clusters_patho_only.append((cluster_id, total))
        print(f"   [PATHO] Metacluster {cluster_id:2d}: {total:6,} cellules (100% Pathologique)")
    
    # Cluster 100% sain
    elif "Sain" in condition_counts and condition_counts.get("Sain", 0) == total:
        clusters_sain_only.append((cluster_id, total))
        print(f"   [SAIN]  Metacluster {cluster_id:2d}: {total:6,} cellules (100% Sain)")
    
    else:
        clusters_mixed.append(cluster_id)

# Résumé
print("\n" + "="*70)
print(" RÉSUMÉ")
print("="*70)

if clusters_patho_only:
    total_patho_exclusive = sum([c[1] for c in clusters_patho_only])
    print(f"\n[!] CLUSTERS 100% PATHOLOGIQUES: {len(clusters_patho_only)}")
    print(f"    Metaclusters: {[c[0] for c in clusters_patho_only]}")
    print(f"    Total cellules: {total_patho_exclusive:,}")
    print(f"    → Ces clusters représentent des populations UNIQUEMENT présentes chez le patient")
else:
    print(f"\n    Aucun cluster exclusivement pathologique détecté")

if clusters_sain_only:
    total_sain_exclusive = sum([c[1] for c in clusters_sain_only])
    print(f"\n[!] CLUSTERS 100% SAINS: {len(clusters_sain_only)}")
    print(f"    Metaclusters: {[c[0] for c in clusters_sain_only]}")
    print(f"    Total cellules: {total_sain_exclusive:,}")
    print(f"    → Ces clusters représentent des populations ABSENTES chez le patient")
else:
    print(f"\n    Aucun cluster exclusivement sain détecté")

print(f"\n    Clusters mixtes (partagés): {len(clusters_mixed)}")

# Visualisation si clusters exclusifs pathologiques
if clusters_patho_only and len(clusters_patho_only) > 0:
    print("\n" + "-"*70)
    print(" DÉTAIL DES CLUSTERS PATHOLOGIQUES EXCLUSIFS")
    print("-"*70)
    
    # Calculer le MFI des marqueurs pour ces clusters
    for cluster_id, n_cells in clusters_patho_only:
        mask_c = (metaclustering_cells == cluster_id)
        print(f"\n   Metacluster {cluster_id} ({n_cells:,} cellules):")
        
        # Top 3 marqueurs les plus exprimés
        mfi_cluster = adata_flowsom.X[mask_c].mean(axis=0)
        top_3_idx = np.argsort(mfi_cluster)[-3:][::-1]
        print(f"      Top marqueurs: ", end="")
        for idx in top_3_idx:
            marker_name = adata_flowsom.var_names[idx]
            print(f"{marker_name}({mfi_cluster[idx]:.2f}) ", end="")
        print()

## 12. Export des Résultats

Sauvegarde des résultats d'analyse:
- **CSV**: Tableau avec métaclusters assignés à chaque cellule
- **FCS**: Fichier FCS avec colonne métacluster ajoutée

In [ ]:
# =============================================================================
# EXPORT CSV/FCS AVEC COORDONNÉES SOM (style FlowSOM R EXACT)
# =============================================================================

import os
from datetime import datetime

# Créer le dossier de sortie
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(" PRÉPARATION DES DONNÉES POUR EXPORT (style FlowSOM R EXACT)")
print("="*70)

# =====================================================================
# PARAMÈTRES DE JITTER - STYLE FLOWSOM R EXACT
# Dans FlowSOM R, le jitter est CIRCULAIRE (pas carré!)
# La taille du cercle dépend du nombre de cellules dans le cluster
# Formule R: rnorm() * scale_factor * sqrt(node_size/max_size)
# =====================================================================
np.random.seed(SEED)  # Pour reproductibilité

# Paramètres FlowSOM R
MAX_NODE_SIZE = 0.45  # Rayon maximum du cercle (quand le node est le plus grand)
MIN_NODE_SIZE = 0.1   # Rayon minimum du cercle (pour éviter que les petits nodes disparaissent)

# Récupérer les coordonnées de grille et MST depuis cluster_data
grid_coords = cluster_data.obsm.get('grid', None)
layout_coords = cluster_data.obsm.get('layout', None)

# Récupérer le clustering pour mapper les coordonnées sur chaque cellule
clustering = cell_data.obs['clustering'].values
n_cells = len(clustering)
n_nodes = len(cluster_data)

# Calculer la taille de chaque node (nombre de cellules)
node_sizes = np.zeros(n_nodes, dtype=np.float32)
for i in range(n_nodes):
    node_sizes[i] = (clustering == i).sum()

max_size = node_sizes.max()
print(f"\n Taille des nodes:")
print(f"   Min: {node_sizes.min():.0f} cellules")
print(f"   Max: {max_size:.0f} cellules")
print(f"   Total: {n_cells} cellules")

# Créer un DataFrame avec toutes les données
df_export = pd.DataFrame(X, columns=var_names)

# MetaCluster avec +1 pour Kaluza (éviter le 0, commencer à 1)
df_export['FlowSOM_metacluster'] = metaclustering + 1

# FlowSOM cluster (nodes) avec +1
df_export['FlowSOM_cluster'] = clustering + 1

# Ajouter les métadonnées si disponibles
if 'condition' in cell_data.obs.columns:
    df_export['Condition'] = cell_data.obs['condition'].values
    df_export['Condition_Num'] = np.where(df_export['Condition'] == 'Sain', 1, 2)
if 'file_origin' in cell_data.obs.columns:
    df_export['File_Origin'] = cell_data.obs['file_origin'].values

# =====================================================================
# FONCTION JITTER CIRCULAIRE (style FlowSOM R exact)
# Génère des points distribués uniformément dans un disque
# Le rayon dépend de la taille du cluster
# =====================================================================
def circular_jitter(n_points, cluster_ids, node_sizes, max_radius=0.45, min_radius=0.1):
    """
    Génère un jitter circulaire style FlowSOM R.
    
    Dans FlowSOM R, les cellules sont distribuées dans des cercles
    dont le rayon dépend du nombre de cellules dans le node.
    Plus un node a de cellules, plus le cercle est grand.
    
    Méthode: 
    - Angle theta uniforme [0, 2*pi]
    - Rayon r = sqrt(u) * max_r (pour distribution uniforme dans le disque)
    - Le max_r dépend de la taille du node
    """
    # Angle uniforme autour du cercle
    theta = np.random.uniform(0, 2 * np.pi, n_points)
    
    # Rayon - distribution uniforme dans le disque (sqrt pour uniformité)
    u = np.random.uniform(0, 1, n_points)
    
    # Calculer le rayon pour chaque cellule selon la taille de son cluster
    # Dans FlowSOM R, le rayon est proportionnel à sqrt(node_size/max_size)
    max_size_val = node_sizes.max()
    
    # Rayon pour chaque cellule
    radii = np.zeros(n_points, dtype=np.float32)
    for i in range(n_points):
        node_id = int(cluster_ids[i])
        node_size = node_sizes[node_id]
        # Rayon proportionnel à sqrt(taille relative)
        size_ratio = np.sqrt(node_size / max_size_val)
        # Interpolation entre min et max radius
        node_radius = min_radius + (max_radius - min_radius) * size_ratio
        radii[i] = node_radius
    
    # Rayon final pour distribution uniforme dans le disque
    r = np.sqrt(u) * radii
    
    # Convertir en coordonnées cartésiennes
    jitter_x = r * np.cos(theta)
    jitter_y = r * np.sin(theta)
    
    return jitter_x.astype(np.float32), jitter_y.astype(np.float32)

# =====================================================================
# COORDONNÉES GRILLE SOM (xGrid, yGrid) - Style FlowSOM R
# =====================================================================
print(f"\n Application du jitter CIRCULAIRE style FlowSOM R")
print(f"   Rayon min: {MIN_NODE_SIZE}, Rayon max: {MAX_NODE_SIZE}")

if grid_coords is not None:
    # Générer jitter CIRCULAIRE dépendant de la taille du node
    jitter_x, jitter_y = circular_jitter(n_cells, clustering, node_sizes, 
                                          max_radius=MAX_NODE_SIZE, 
                                          min_radius=MIN_NODE_SIZE)
    
    # Mapper les coordonnées de grille sur chaque cellule
    xGrid_base = np.array([grid_coords[int(c), 0] for c in clustering], dtype=np.float32)
    yGrid_base = np.array([grid_coords[int(c), 1] for c in clustering], dtype=np.float32)
    
    # Appliquer le jitter circulaire
    xGrid_jittered = xGrid_base + jitter_x
    yGrid_jittered = yGrid_base + jitter_y
    
    # Décaler pour que les axes commencent à 1 (X ET Y)
    xGrid = xGrid_jittered - xGrid_jittered.min() + 1.0
    yGrid = yGrid_jittered - yGrid_jittered.min() + 1.0
    
    df_export['xGrid'] = xGrid.astype(np.float32)
    df_export['yGrid'] = yGrid.astype(np.float32)
    
    print(f"[OK] xGrid: [{xGrid.min():.3f} - {xGrid.max():.3f}]")
    print(f"[OK] yGrid: [{yGrid.min():.3f} - {yGrid.max():.3f}]")

# =====================================================================
# COORDONNÉES MST (xNodes, yNodes) - Style FlowSOM R
# =====================================================================
if layout_coords is not None:
    # Mapper les coordonnées MST sur chaque cellule
    xNodes_base = np.array([layout_coords[int(c), 0] for c in clustering], dtype=np.float32)
    yNodes_base = np.array([layout_coords[int(c), 1] for c in clustering], dtype=np.float32)
    
    # Calculer l'échelle pour le jitter MST (proportionnel à l'espacement moyen)
    x_range = xNodes_base.max() - xNodes_base.min()
    y_range = yNodes_base.max() - yNodes_base.min()
    mst_scale = min(x_range, y_range) / (XDIM * 2)  # Proportionnel à la grille
    
    # Jitter circulaire pour MST aussi
    mst_jitter_x, mst_jitter_y = circular_jitter(
        n_cells, clustering, node_sizes,
        max_radius=mst_scale * 0.8,  # Un peu moins que Grid car MST est plus espacé
        min_radius=mst_scale * 0.2
    )
    
    # Appliquer le jitter
    xNodes_jittered = xNodes_base + mst_jitter_x
    yNodes_jittered = yNodes_base + mst_jitter_y
    
    # Décaler pour que les axes commencent à 1 (X ET Y)
    xNodes = xNodes_jittered - xNodes_jittered.min() + 1.0
    yNodes = yNodes_jittered - yNodes_jittered.min() + 1.0
    
    df_export['xNodes'] = xNodes.astype(np.float32)
    df_export['yNodes'] = yNodes.astype(np.float32)
    
    print(f"[OK] xNodes: [{xNodes.min():.3f} - {xNodes.max():.3f}]")
    print(f"[OK] yNodes: [{yNodes.min():.3f} - {yNodes.max():.3f}]")

# =====================================================================
# TAILLE DES NODES (pour chaque cellule)
# =====================================================================
size_col = np.array([node_sizes[int(c)] for c in clustering], dtype=np.float32)
df_export['size'] = size_col
print(f"[OK] size: [{size_col.min():.0f} - {size_col.max():.0f}]")

# =====================================================================
# EXPORT CSV
# =====================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = os.path.join(OUTPUT_DIR, f"flowsom_results_{timestamp}.csv")
df_export.to_csv(csv_path, index=False)

print(f"\n[OK] CSV exporté: {csv_path}")
print(f"   Shape: {df_export.shape}")

# =====================================================================
# EXPORT FCS COMPATIBLE KALUZA
# =====================================================================
print("\n" + "="*70)
print("📄 EXPORT FCS COMPATIBLE KALUZA")
print("="*70)

def export_to_fcs_kaluza(df, output_path):
    """Export FCS compatible Kaluza avec toutes les coordonnées positives."""
    try:
        import fcswrite
        
        numeric_df = df.select_dtypes(include=[np.number])
        data = numeric_df.values.astype(np.float32)
        channels = list(numeric_df.columns)
        
        # Nettoyer NaN/Inf
        data = np.nan_to_num(data, nan=0.0, posinf=1e6, neginf=0.0)
        
        print(f"   {data.shape[0]:,} events, {data.shape[1]} canaux")
        
        fcswrite.write_fcs(output_path, channels, data, compat_chn_names=True)
        return True
        
    except ImportError:
        print("   [!] fcswrite non disponible (pip install fcswrite)")
        return False
    except Exception as e:
        print(f"   [!] Erreur: {e}")
        return False

# Préparer le DataFrame FCS
df_fcs = df_export.select_dtypes(include=[np.number]).copy()

# Vérifier les ranges
print(f"\n Colonnes exportées vers FCS:")
for col in ['FlowSOM_metacluster', 'FlowSOM_cluster', 'xGrid', 'yGrid', 'xNodes', 'yNodes', 'size', 'Condition_Num']:
    if col in df_fcs.columns:
        print(f"   [OK] {col:25s}: [{df_fcs[col].min():10.2f}, {df_fcs[col].max():10.2f}]")

# Export
fcs_path = os.path.join(OUTPUT_DIR, f"flowsom_results_{timestamp}.fcs")
if export_to_fcs_kaluza(df_fcs, fcs_path):
    print(f"\n[OK] FCS exporté: {fcs_path}")
    print(f"\n Dans Kaluza/FlowJo:")
    print(f"   - xGrid vs yGrid → visualisation grille SOM (cercles style FlowSOM R)")
    print(f"   - xNodes vs yNodes → visualisation arbre MST")
    print(f"   - La taille des cercles = proportionnelle au nombre de cellules")
    print(f"   - Colorez par FlowSOM_metacluster")

In [ ]:
# =============================================================================
# EXPORT DU RAPPORT DE STATISTIQUES
# =============================================================================

# Sauvegarder le rapport de statistiques
stats_path = os.path.join(OUTPUT_DIR, f"flowsom_statistics_{timestamp}.csv")
df_stats.to_csv(stats_path, index=False)
print(f"[OK] Statistiques exportées: {stats_path}")

# Sauvegarder la matrice MFI
mfi_path = os.path.join(OUTPUT_DIR, f"flowsom_mfi_matrix_{timestamp}.csv")
df_mfi.to_csv(mfi_path)
print(f"[OK] Matrice MFI exportée: {mfi_path}")

# Résumé final
print("\n" + "="*80)
print(" RÉSUMÉ DE L'ANALYSE FLOWSOM")
print("="*80)
print(f"   Fichiers analysés: {len(all_adatas)}")
print(f"   Cellules totales: {len(cell_data)}")
print(f"   Marqueurs utilisés: {len(used_markers)}")
print(f"   Nombre de métaclusters: {N_CLUSTERS}")
print(f"   Transformation: {TRANSFORM_TYPE}")
print(f"   Cofacteur: {COFACTOR}")
print("="*80)
print("[OK] Analyse FlowSOM terminée avec succès!")

In [ ]:
# INSPECTION DES VARIABLES DU NOTEBOOK


print("="*70)
print(" LISTE DES OBJETS AnnData")
print("="*70)
%who AnnData

print("\n" + "="*70)
print(" LISTE DES DataFrames")
print("="*70)
%who DataFrame

print("\n" + "="*70)
print(" DÉTAILS DE adata_flowsom")
print("="*70)
print(adata_flowsom)

print("\n- Shape:", adata_flowsom.shape)
print("- Variables (colonnes):", list(adata_flowsom.var_names))
print("- Observations (fichiers):", adata_flowsom.obs.columns.tolist() if len(adata_flowsom.obs.columns) > 0 else "Aucune annotation")

print("\n" + "-"*70)
print("- Statistiques de la matrice X:")
print("-"*70)
import pandas as pd
stats = pd.DataFrame({
    'Colonne': adata_flowsom.var_names,
    'Min': adata_flowsom.X.min(axis=0),
    'Max': adata_flowsom.X.max(axis=0),
    'Moyenne': adata_flowsom.X.mean(axis=0),
    'Std': adata_flowsom.X.std(axis=0)
})
display(stats)

print("\n" + "-"*70)
print("- Aperçu des observations (.obs):")
print("-"*70)
if adata_flowsom.obs.shape[1] > 0:
    display(adata_flowsom.obs.head(10))
else:
    print("Aucune annotation dans .obs")

print("\n" + "-"*70)
print("- Toutes les variables disponibles:")
print("-"*70)
%whos

In [ ]:
# =============================================================================
# FLOWSOM SUMMARY - RAPPORT PDF COMPLET
# =============================================================================
# Génération automatique d'un PDF récapitulatif avec toutes les visualisations
# Documentation: https://flowsom.readthedocs.io/en/stable/generated/flowsom.pl.FlowSOMmary.html
# =============================================================================

import os

# Chemin du fichier PDF de sortie
summary_pdf_path = os.path.join(OUTPUT_DIR, f"FlowSOMmary_{timestamp}.pdf")

print("="*70)
print(" GÉNÉRATION DU RAPPORT FlowSOMmary (PDF)")
print("="*70)

try:
    # Génération du rapport FlowSOMmary
    # Inclut: marker_diff, cluster_profiles, grid, mst
    # Exclut: UMAP (non demandé)
    fs.pl.FlowSOMmary(
        fsom,
        plot_file=summary_pdf_path
    )
    
    print(f"\n[OK] Rapport PDF généré avec succès!")
    print(f"    Fichier: {summary_pdf_path}")
    print(f"\n    Contenu du rapport:")
    print(f"    - Star plots (profils MFI par cluster)")
    print(f"    - Grid SOM avec metaclusters")
    print(f"    - Arbre MST avec metaclusters")
    print(f"    - Heatmap des marqueurs par cluster")
    print(f"    - Distribution des tailles de clusters")
    
except Exception as e:
    print(f"\n[!] Erreur lors de la génération du FlowSOMmary:")
    print(f"    {str(e)}")
    print(f"\n    Tentative de génération manuelle des graphiques...")
    
    # Alternative: générer les graphiques individuellement
    try:
        from matplotlib.backends.backend_pdf import PdfPages
        
        with PdfPages(summary_pdf_path) as pdf:
            # 1. Star plots par metacluster
            print("    - Génération des star plots...")
            fig_stars = fs.pl.plot_stars(fsom, background_values=fsom.get_cluster_data().obs["metaclustering"])
            pdf.savefig(fig_stars, bbox_inches='tight')
            plt.close(fig_stars)
            
            # 2. Grid SOM
            print("    - Génération de la grille SOM...")
            fig_grid = fs.pl.plot_grid(fsom, background_values=fsom.get_cluster_data().obs["metaclustering"])
            pdf.savefig(fig_grid, bbox_inches='tight')
            plt.close(fig_grid)
            
            # 3. Arbre MST
            print("    - Génération de l'arbre MST...")
            fig_mst = fs.pl.plot_mst(fsom, background_values=fsom.get_cluster_data().obs["metaclustering"])
            pdf.savefig(fig_mst, bbox_inches='tight')
            plt.close(fig_mst)
            
            # 4. Marker heatmap
            print("    - Génération de la heatmap des marqueurs...")
            fig_heatmap = fs.pl.plot_marker_heatmap(fsom)
            pdf.savefig(fig_heatmap, bbox_inches='tight')
            plt.close(fig_heatmap)
            
        print(f"\n[OK] Rapport PDF alternatif généré!")
        print(f"    Fichier: {summary_pdf_path}")
        
    except Exception as e2:
        print(f"\n[X] Impossible de générer le rapport: {str(e2)}")

print("\n" + "="*70)
print(" FIN DE L'ANALYSE FlowSOM")
print("="*70)